In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [1]:
import os
import datetime

st = datetime.datetime.now()

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# SVC class is too slow to train for large dataset


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [2]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper, sim=0.5, thre=0.840845057):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        
        ### constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        constraints.append(WordEmbeddingDistance(min_cos_sim=sim))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = thre,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [3]:
def runAttack(model_wrapper, n, path, target_ds, sim=0.5, thre=0.840845057):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper, sim, thre)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=50, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn

### BERT-based-uncased

In [6]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [7]:
bertagtrain = pd.read_csv('../dataset_embeddings/bertagnewstrain.txt', header=None)
bertagtest = pd.read_csv('../dataset_embeddings/bertagnewstest.txt', header=None)
lr_bert_ag, rf_bert_ag, knn_bert_ag = adaptation(bertagtrain, bertagtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8942105263157895
RF 0.7252631578947368
KNN 0.7902631578947369


In [8]:
# custom model wrapper for BERT
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [9]:
bert_model_wrapper_lr_ag = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_ag)
bert_model_wrapper_rf_ag = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_ag)
bert_model_wrapper_knn_ag = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_ag)

### DistilBERT-based-uncased

In [10]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
disbertagtrain = pd.read_csv('../dataset_embeddings/disbertagnewstrain.txt', header=None)
disbertagtest = pd.read_csv('../dataset_embeddings/disbertagnewstest.txt', header=None)
lr_disbert_ag, rf_disbert_ag, knn_disbert_ag = adaptation(disbertagtrain, disbertagtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.9148684210526316
RF 0.8802631578947369
KNN 0.9198684210526316


In [12]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [13]:
disbert_model_wrapper_lr_ag = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_ag)
disbert_model_wrapper_rf_ag = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_ag)
disbert_model_wrapper_knn_ag = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_ag)

### CLIP

In [14]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
clipagtrain = pd.read_csv('../dataset_embeddings/clipagnewstrain.txt', header=None)
clipagtest = pd.read_csv('../dataset_embeddings/clipagnewstest.txt', header=None)
lr_clip_ag, rf_clip_ag, knn_clip_ag = adaptation(clipagtrain, clipagtest)

Log Reg 0.915921052631579
RF 0.8751315789473684
KNN 0.9248684210526316


In [16]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [17]:
clip_model_wrapper_lr_ag = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_ag)
clip_model_wrapper_rf_ag = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_ag)
clip_model_wrapper_knn_ag = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_ag)

### Run Attacks

### Default parameters, threshold = 1 - 0.5/$\pi$ = 0.841, minSim = 0.5

In [4]:
target_ds_ag = textattack.datasets.HuggingFaceDataset('ag_news', split='test')

textattack: Loading datasets dataset ag_news, split test.


In [19]:
suffix = 'ep0841.csv'
n_samples = 500

In [20]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 43 / 2 / 5 / 50:  10%|██▎                    | 50/500 [08:39<1:17:52, 10.38s/it]textattack: Saving checkpoint under "checkpoints\1720266841671.ta.chkpt" at 2024-07-06 19:54:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 2 / 10 / 100:  20%|████                | 100/500 [16:50<1:07:20, 10.10s/it]textattack: Saving checkpoint under "checkpoints\1720267332696.ta.chkpt" at 2024-07-06 20:02:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 4 / 15 / 150:  30%|█████▋             | 150/500 [27:40<1:04:35, 11.07s/it]textattack: Saving checkpoint under "checkpoints\1720267983355.ta.chkpt" at 2024-07-06 20:13:03 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 307 / 6 / 37 / 350:  70%|█████████████▎     | 350/500 [1:07:16<28:50, 11.53s/it]textattack: Saving checkpoint under "checkpoints\1720270359405.ta.chkpt" at 2024-07-06 20:52:39 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 352 / 7 / 41 / 400:  80%|███████████████▏   | 400/500 [1:14:37<18:39, 11.19s/it]textattack: Saving checkpoint under "checkpoints\1720270799772.ta.chkpt" at 2024-07-06 20:59:59 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 390 / 7 / 53 / 450:  90%|█████████████████  | 450/500 [1:21:49<09:05, 10.91s/it]textattack: Saving checkpoint under "checkpoints\1720271231992.ta.chkpt" at 2024-07-06 21:07:11 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 434 / 9 / 57 / 500: 100%|███████████████████| 500/500 [1:31:41<00:00, 11.00s/it]textattack: Saving checkpoint under "checkpoints\1720271823539.ta.chkpt" at 2024-07-06 21:17:03 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 434 / 9 / 57 / 500: 100%|███████████████████| 500/500 [1:31:41<00:00, 11.00s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 434    |
| Number of failed attacks:     | 9      |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 1.8%   |
| Attack success rate:          | 97.97% |
| Average perturbed word %:     | 15.49% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 189.61 |
+-------------------------------+--------+


In [21]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 33 / 2 / 15 / 50:  10%|██▍                     | 50/500 [05:12<46:55,  6.26s/it]textattack: Saving checkpoint under "checkpoints\1720272136711.ta.chkpt" at 2024-07-06 21:22:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 3 / 31 / 100:  20%|████▍                 | 100/500 [10:09<40:38,  6.10s/it]textattack: Saving checkpoint under "checkpoints\1720272433494.ta.chkpt" at 2024-07-06 21:27:13 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 66 / 3 / 32 / 101:  20%|████▍                 | 101/500 [10:09<40:08,  6.04s/it]

[Succeeded / Failed / Skipped / Total] 103 / 5 / 42 / 150:  30%|██████▎              | 150/500 [16:21<38:11,  6.55s/it]textattack: Saving checkpoint under "checkpoints\1720272805876.ta.chkpt" at 2024-07-06 21:33:25 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 241 / 8 / 101 / 350:  70%|██████████████      | 350/500 [41:26<17:45,  7.10s/it]textattack: Saving checkpoint under "checkpoints\1720274310198.ta.chkpt" at 2024-07-06 21:58:30 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 279 / 9 / 112 / 400:  80%|████████████████    | 400/500 [46:10<11:32,  6.93s/it]textattack: Saving checkpoint under "checkpoints\1720274594557.ta.chkpt" at 2024-07-06 22:03:14 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 310 / 11 / 129 / 450:  90%|█████████████████  | 450/500 [51:57<05:46,  6.93s/it]textattack: Saving checkpoint under "checkpoints\1720274940963.ta.chkpt" at 2024-07-06 22:09:00 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 346 / 13 / 141 / 500: 100%|███████████████████| 500/500 [59:08<00:00,  7.10s/it]textattack: Saving checkpoint under "checkpoints\1720275372275.ta.chkpt" at 2024-07-06 22:16:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 346 / 13 / 141 / 500: 100%|███████████████████| 500/500 [59:08<00:00,  7.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 346    |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 2.6%   |
| Attack success rate:          | 96.38% |
| Average perturbed word %:     | 9.3%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 145.09 |
+-------------------------------+--------+


In [22]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 32 / 7 / 11 / 50:  10%|██▏                   | 50/500 [10:15<1:32:17, 12.31s/it]textattack: Saving checkpoint under "checkpoints\1720275987874.ta.chkpt" at 2024-07-06 22:26:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 10 / 20 / 100:  20%|███▊               | 100/500 [20:00<1:20:02, 12.01s/it]textattack: Saving checkpoint under "checkpoints\1720276573250.ta.chkpt" at 2024-07-06 22:36:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 100 / 18 / 32 / 150:  30%|█████▍            | 150/500 [32:16<1:15:19, 12.91s/it]textattack: Saving checkpoint under "checkpoints\1720277309451.ta.chkpt" at 2024-07-06 22:48:29 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 232 / 45 / 73 / 350:  70%|████████████▌     | 350/500 [1:15:49<32:29, 13.00s/it]textattack: Saving checkpoint under "checkpoints\1720279921707.ta.chkpt" at 2024-07-06 23:32:01 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 263 / 52 / 85 / 400:  80%|██████████████▍   | 400/500 [1:24:57<21:14, 12.74s/it]textattack: Saving checkpoint under "checkpoints\1720280470311.ta.chkpt" at 2024-07-06 23:41:10 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 292 / 59 / 99 / 450:  90%|████████████████▏ | 450/500 [1:35:00<10:33, 12.67s/it]textattack: Saving checkpoint under "checkpoints\1720281072754.ta.chkpt" at 2024-07-06 23:51:12 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 325 / 66 / 109 / 500: 100%|█████████████████| 500/500 [1:45:00<00:00, 12.60s/it]textattack: Saving checkpoint under "checkpoints\1720281672986.ta.chkpt" at 2024-07-07 00:01:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 325 / 66 / 109 / 500: 100%|█████████████████| 500/500 [1:45:00<00:00, 12.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 325    |
| Number of failed attacks:     | 66     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 13.2%  |
| Attack success rate:          | 83.12% |
| Average perturbed word %:     | 4.94%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 200.58 |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  10%|██▌                      | 50/500 [06:14<56:07,  7.48s/it]textattack: Saving checkpoint under "checkpoints\1720282047288.ta.chkpt" at 2024-07-07 00:07:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 4 / 8 / 100:  20%|████▌                  | 100/500 [11:59<47:57,  7.19s/it]textattack: Saving checkpoint under "checkpoints\1720282392418.ta.chkpt" at 2024-07-07 00:13:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 9 / 10 / 150:  30%|██████▎              | 150/500 [18:47<43:50,  7.52s/it]textattack: Saving checkpoint under "checkpoints\1720282800609.ta.chkpt" at 2024-07-07 00:20:00 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 304 / 15 / 31 / 350:  70%|██████████████      | 350/500 [46:27<19:54,  7.96s/it]textattack: Saving checkpoint under "checkpoints\1720284460177.ta.chkpt" at 2024-07-07 00:47:40 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 350 / 16 / 34 / 400:  80%|████████████████    | 400/500 [51:48<12:57,  7.77s/it]textattack: Saving checkpoint under "checkpoints\1720284781842.ta.chkpt" at 2024-07-07 00:53:01 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 391 / 18 / 41 / 450:  90%|██████████████████  | 450/500 [57:29<06:23,  7.67s/it]textattack: Saving checkpoint under "checkpoints\1720285123047.ta.chkpt" at 2024-07-07 00:58:43 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 436 / 20 / 44 / 500: 100%|██████████████████| 500/500 [1:03:24<00:00,  7.61s/it]textattack: Saving checkpoint under "checkpoints\1720285478099.ta.chkpt" at 2024-07-07 01:04:38 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 436 / 20 / 44 / 500: 100%|██████████████████| 500/500 [1:03:25<00:00,  7.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 436    |
| Number of failed attacks:     | 20     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 4.0%   |
| Attack success rate:          | 95.61% |
| Average perturbed word %:     | 20.28% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 236.67 |
+-------------------------------+--------+


In [24]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 39 / 7 / 4 / 50:  10%|██▎                    | 50/500 [07:56<1:11:31,  9.54s/it]textattack: Saving checkpoint under "checkpoints\1720285955056.ta.chkpt" at 2024-07-07 01:12:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 72 / 15 / 13 / 100:  20%|███▊               | 100/500 [15:55<1:03:42,  9.56s/it]textattack: Saving checkpoint under "checkpoints\1720286433914.ta.chkpt" at 2024-07-07 01:20:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 111 / 19 / 20 / 150:  30%|██████              | 150/500 [23:54<55:47,  9.56s/it]textattack: Saving checkpoint under "checkpoints\1720286912899.ta.chkpt" at 2024-07-07 01:28:32 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 257 / 47 / 46 / 350:  70%|██████████████      | 350/500 [59:56<25:41, 10.28s/it]textattack: Saving checkpoint under "checkpoints\1720289074857.ta.chkpt" at 2024-07-07 02:04:34 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 53 / 50 / 400:  80%|██████████████▍   | 400/500 [1:07:17<16:49, 10.09s/it]textattack: Saving checkpoint under "checkpoints\1720289515407.ta.chkpt" at 2024-07-07 02:11:55 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 331 / 60 / 59 / 450:  90%|████████████████▏ | 450/500 [1:16:08<08:27, 10.15s/it]textattack: Saving checkpoint under "checkpoints\1720290046337.ta.chkpt" at 2024-07-07 02:20:46 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 366 / 71 / 63 / 500: 100%|██████████████████| 500/500 [1:24:13<00:00, 10.11s/it]textattack: Saving checkpoint under "checkpoints\1720290531852.ta.chkpt" at 2024-07-07 02:28:51 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 366 / 71 / 63 / 500: 100%|██████████████████| 500/500 [1:24:13<00:00, 10.11s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 366    |
| Number of failed attacks:     | 71     |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 14.2%  |
| Attack success rate:          | 83.75% |
| Average perturbed word %:     | 23.15% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 324.56 |
+-------------------------------+--------+


In [25]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 8 / 39 / 3 / 50:  10%|██▎                    | 50/500 [17:46<2:40:00, 21.33s/it]textattack: Saving checkpoint under "checkpoints\1720291598679.ta.chkpt" at 2024-07-07 02:46:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 76 / 8 / 100:  20%|████                | 100/500 [36:35<2:26:20, 21.95s/it]textattack: Saving checkpoint under "checkpoints\1720292727111.ta.chkpt" at 2024-07-07 03:05:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 113 / 11 / 150:  30%|█████▍            | 150/500 [55:21<2:09:09, 22.14s/it]textattack: Saving checkpoint under "checkpoints\1720293853161.ta.chkpt" at 2024-07-07 03:24:13 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 50 / 265 / 35 / 350:  70%|████████████▌     | 350/500 [2:15:10<57:55, 23.17s/it]textattack: Saving checkpoint under "checkpoints\1720298642032.ta.chkpt" at 2024-07-07 04:44:02 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 56 / 304 / 40 / 400:  80%|██████████████▍   | 400/500 [2:33:08<38:17, 22.97s/it]textattack: Saving checkpoint under "checkpoints\1720299720645.ta.chkpt" at 2024-07-07 05:02:00 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 340 / 43 / 450:  90%|████████████████▏ | 450/500 [2:51:14<19:01, 22.83s/it]textattack: Saving checkpoint under "checkpoints\1720300806658.ta.chkpt" at 2024-07-07 05:20:06 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 380 / 45 / 500: 100%|██████████████████| 500/500 [3:08:19<00:00, 22.60s/it]textattack: Saving checkpoint under "checkpoints\1720301831155.ta.chkpt" at 2024-07-07 05:37:11 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 75 / 380 / 45 / 500: 100%|██████████████████| 500/500 [3:08:19<00:00, 22.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 75     |
| Number of failed attacks:     | 380    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 76.0%  |
| Attack success rate:          | 16.48% |
| Average perturbed word %:     | 6.21%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 523.8  |
+-------------------------------+--------+


In [26]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 43 / 3 / 4 / 50:  10%|██▎                    | 50/500 [08:34<1:17:14, 10.30s/it]textattack: Saving checkpoint under "checkpoints\1720302346259.ta.chkpt" at 2024-07-07 05:45:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 5 / 9 / 100:  20%|████▏                | 100/500 [15:24<1:01:36,  9.24s/it]textattack: Saving checkpoint under "checkpoints\1720302755495.ta.chkpt" at 2024-07-07 05:52:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 130 / 6 / 14 / 150:  30%|██████▎              | 150/500 [22:29<52:28,  9.00s/it]textattack: Saving checkpoint under "checkpoints\1720303180628.ta.chkpt" at 2024-07-07 05:59:40 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 308 / 9 / 33 / 350:  70%|██████████████▋      | 350/500 [51:17<21:59,  8.79s/it]textattack: Saving checkpoint under "checkpoints\1720304909263.ta.chkpt" at 2024-07-07 06:28:29 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 353 / 10 / 37 / 400:  80%|████████████████    | 400/500 [56:38<14:09,  8.50s/it]textattack: Saving checkpoint under "checkpoints\1720305229747.ta.chkpt" at 2024-07-07 06:33:49 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 393 / 12 / 45 / 450:  90%|████████████████▏ | 450/500 [1:03:02<07:00,  8.41s/it]textattack: Saving checkpoint under "checkpoints\1720305613643.ta.chkpt" at 2024-07-07 06:40:13 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 441 / 13 / 46 / 500: 100%|██████████████████| 500/500 [1:10:37<00:00,  8.48s/it]textattack: Saving checkpoint under "checkpoints\1720306069173.ta.chkpt" at 2024-07-07 06:47:49 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 441 / 13 / 46 / 500: 100%|██████████████████| 500/500 [1:10:37<00:00,  8.48s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 441    |
| Number of failed attacks:     | 13     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 2.6%   |
| Attack success rate:          | 97.14% |
| Average perturbed word %:     | 19.47% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 226.45 |
+-------------------------------+--------+


In [27]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 33 / 13 / 4 / 50:  10%|██▏                   | 50/500 [09:45<1:27:48, 11.71s/it]textattack: Saving checkpoint under "checkpoints\1720306654739.ta.chkpt" at 2024-07-07 06:57:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 18 / 13 / 100:  20%|███▊               | 100/500 [18:19<1:13:19, 11.00s/it]textattack: Saving checkpoint under "checkpoints\1720307169184.ta.chkpt" at 2024-07-07 07:06:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 100 / 30 / 20 / 150:  30%|█████▍            | 150/500 [28:04<1:05:30, 11.23s/it]textattack: Saving checkpoint under "checkpoints\1720307753931.ta.chkpt" at 2024-07-07 07:15:53 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 228 / 70 / 52 / 350:  70%|████████████▌     | 350/500 [1:03:49<27:21, 10.94s/it]textattack: Saving checkpoint under "checkpoints\1720309898779.ta.chkpt" at 2024-07-07 07:51:38 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 260 / 81 / 59 / 400:  80%|██████████████▍   | 400/500 [1:11:40<17:55, 10.75s/it]textattack: Saving checkpoint under "checkpoints\1720310370211.ta.chkpt" at 2024-07-07 07:59:30 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 286 / 93 / 71 / 450:  90%|████████████████▏ | 450/500 [1:21:01<09:00, 10.80s/it]textattack: Saving checkpoint under "checkpoints\1720310930811.ta.chkpt" at 2024-07-07 08:08:50 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 323 / 104 / 73 / 500: 100%|█████████████████| 500/500 [1:31:25<00:00, 10.97s/it]textattack: Saving checkpoint under "checkpoints\1720311554781.ta.chkpt" at 2024-07-07 08:19:14 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 323 / 104 / 73 / 500: 100%|█████████████████| 500/500 [1:31:25<00:00, 10.97s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 323    |
| Number of failed attacks:     | 104    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 20.8%  |
| Attack success rate:          | 75.64% |
| Average perturbed word %:     | 18.25% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 311.06 |
+-------------------------------+--------+


In [28]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 44 / 1 / 50:  10%|██▎                    | 50/500 [20:02<3:00:25, 24.06s/it]textattack: Saving checkpoint under "checkpoints\1720312757912.ta.chkpt" at 2024-07-07 08:39:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 78 / 4 / 100:  20%|████                | 100/500 [40:01<2:40:04, 24.01s/it]textattack: Saving checkpoint under "checkpoints\1720313956148.ta.chkpt" at 2024-07-07 08:59:16 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 118 / 7 / 150:  30%|█████            | 150/500 [1:01:02<2:22:24, 24.41s/it]textattack: Saving checkpoint under "checkpoints\1720315217151.ta.chkpt" at 2024-07-07 09:20:17 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 66 / 258 / 26 / 350:  70%|████████████▌     | 350/500 [2:16:16<58:24, 23.36s/it]textattack: Saving checkpoint under "checkpoints\1720319731685.ta.chkpt" at 2024-07-07 10:35:31 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 297 / 28 / 400:  80%|██████████████▍   | 400/500 [2:34:17<38:34, 23.14s/it]textattack: Saving checkpoint under "checkpoints\1720320812636.ta.chkpt" at 2024-07-07 10:53:32 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 333 / 35 / 450:  90%|████████████████▏ | 450/500 [2:53:22<19:15, 23.12s/it]textattack: Saving checkpoint under "checkpoints\1720321957691.ta.chkpt" at 2024-07-07 11:12:37 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 378 / 35 / 500: 100%|██████████████████| 500/500 [3:14:21<00:00, 23.32s/it]textattack: Saving checkpoint under "checkpoints\1720323216542.ta.chkpt" at 2024-07-07 11:33:36 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 87 / 378 / 35 / 500: 100%|██████████████████| 500/500 [3:14:21<00:00, 23.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 87     |
| Number of failed attacks:     | 378    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 75.6%  |
| Attack success rate:          | 18.71% |
| Average perturbed word %:     | 5.7%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 515.21 |
+-------------------------------+--------+


In [29]:
et = datetime.datetime.now()
et - st

datetime.timedelta(seconds=58631, microseconds=298655)

### Threshold = 1 - 0.4/$\pi$ = 0.873

In [30]:
suffix = 'ep0873.csv'
thre = 1-0.4/np.pi

In [31]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 42 / 3 / 5 / 50:  10%|██▎                    | 50/500 [07:55<1:11:19,  9.51s/it]textattack: Saving checkpoint under "checkpoints\1720323692388.ta.chkpt" at 2024-07-07 11:41:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 3 / 10 / 100:  20%|████                | 100/500 [16:00<1:04:03,  9.61s/it]textattack: Saving checkpoint under "checkpoints\1720324177808.ta.chkpt" at 2024-07-07 11:49:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 128 / 7 / 15 / 150:  30%|█████▋             | 150/500 [26:21<1:01:30, 10.54s/it]textattack: Saving checkpoint under "checkpoints\1720324798463.ta.chkpt" at 2024-07-07 11:59:58 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 299 / 14 / 37 / 350:  70%|████████████▌     | 350/500 [1:03:56<27:24, 10.96s/it]textattack: Saving checkpoint under "checkpoints\1720327053376.ta.chkpt" at 2024-07-07 12:37:33 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 344 / 15 / 41 / 400:  80%|██████████████▍   | 400/500 [1:11:10<17:47, 10.68s/it]textattack: Saving checkpoint under "checkpoints\1720327486935.ta.chkpt" at 2024-07-07 12:44:46 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 382 / 15 / 53 / 450:  90%|████████████████▏ | 450/500 [1:18:16<08:41, 10.44s/it]textattack: Saving checkpoint under "checkpoints\1720327913139.ta.chkpt" at 2024-07-07 12:51:53 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 427 / 16 / 57 / 500: 100%|██████████████████| 500/500 [1:27:33<00:00, 10.51s/it]textattack: Saving checkpoint under "checkpoints\1720328470823.ta.chkpt" at 2024-07-07 13:01:10 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 427 / 16 / 57 / 500: 100%|██████████████████| 500/500 [1:27:33<00:00, 10.51s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 427    |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 3.2%   |
| Attack success rate:          | 96.39% |
| Average perturbed word %:     | 16.29% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 181.91 |
+-------------------------------+--------+


In [32]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 33 / 2 / 15 / 50:  10%|██▍                     | 50/500 [04:32<40:50,  5.44s/it]textattack: Saving checkpoint under "checkpoints\1720328743168.ta.chkpt" at 2024-07-07 13:05:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 65 / 4 / 31 / 100:  20%|████▍                 | 100/500 [09:11<36:45,  5.51s/it]textattack: Saving checkpoint under "checkpoints\1720329022370.ta.chkpt" at 2024-07-07 13:10:22 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 65 / 4 / 32 / 101:  20%|████▍                 | 101/500 [09:11<36:18,  5.46s/it]

[Succeeded / Failed / Skipped / Total] 102 / 6 / 42 / 150:  30%|██████▎              | 150/500 [15:10<35:24,  6.07s/it]textattack: Saving checkpoint under "checkpoints\1720329381437.ta.chkpt" at 2024-07-07 13:16:21 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 237 / 12 / 101 / 350:  70%|█████████████▎     | 350/500 [39:09<16:46,  6.71s/it]textattack: Saving checkpoint under "checkpoints\1720330820161.ta.chkpt" at 2024-07-07 13:40:20 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 275 / 13 / 112 / 400:  80%|███████████████▏   | 400/500 [44:16<11:04,  6.64s/it]textattack: Saving checkpoint under "checkpoints\1720331127913.ta.chkpt" at 2024-07-07 13:45:27 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 306 / 15 / 129 / 450:  90%|█████████████████  | 450/500 [50:23<05:35,  6.72s/it]textattack: Saving checkpoint under "checkpoints\1720331493982.ta.chkpt" at 2024-07-07 13:51:33 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 341 / 18 / 141 / 500: 100%|███████████████████| 500/500 [57:11<00:00,  6.86s/it]textattack: Saving checkpoint under "checkpoints\1720331902735.ta.chkpt" at 2024-07-07 13:58:22 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 341 / 18 / 141 / 500: 100%|███████████████████| 500/500 [57:11<00:00,  6.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 341    |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 3.6%   |
| Attack success rate:          | 94.99% |
| Average perturbed word %:     | 9.39%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 136.5  |
+-------------------------------+--------+


In [33]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 32 / 7 / 11 / 50:  10%|██▏                   | 50/500 [09:39<1:26:51, 11.58s/it]textattack: Saving checkpoint under "checkpoints\1720332481857.ta.chkpt" at 2024-07-07 14:08:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 10 / 20 / 100:  20%|███▊               | 100/500 [18:44<1:14:57, 11.24s/it]textattack: Saving checkpoint under "checkpoints\1720333027193.ta.chkpt" at 2024-07-07 14:17:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 99 / 19 / 32 / 150:  30%|█████▋             | 150/500 [29:11<1:08:06, 11.68s/it]textattack: Saving checkpoint under "checkpoints\1720333654347.ta.chkpt" at 2024-07-07 14:27:34 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 237 / 40 / 73 / 350:  70%|████████████▌     | 350/500 [1:07:58<29:07, 11.65s/it]textattack: Saving checkpoint under "checkpoints\1720335980945.ta.chkpt" at 2024-07-07 15:06:20 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 267 / 48 / 85 / 400:  80%|██████████████▍   | 400/500 [1:16:35<19:08, 11.49s/it]textattack: Saving checkpoint under "checkpoints\1720336497870.ta.chkpt" at 2024-07-07 15:14:57 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 54 / 99 / 450:  90%|████████████████▏ | 450/500 [1:25:17<09:28, 11.37s/it]textattack: Saving checkpoint under "checkpoints\1720337020407.ta.chkpt" at 2024-07-07 15:23:40 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 328 / 63 / 109 / 500: 100%|█████████████████| 500/500 [1:35:06<00:00, 11.41s/it]textattack: Saving checkpoint under "checkpoints\1720337609653.ta.chkpt" at 2024-07-07 15:33:29 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 328 / 63 / 109 / 500: 100%|█████████████████| 500/500 [1:35:06<00:00, 11.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 328    |
| Number of failed attacks:     | 63     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 12.6%  |
| Attack success rate:          | 83.89% |
| Average perturbed word %:     | 5.14%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 180.09 |
+-------------------------------+--------+


In [34]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 43 / 3 / 4 / 50:  10%|██▌                      | 50/500 [06:36<59:32,  7.94s/it]textattack: Saving checkpoint under "checkpoints\1720338006732.ta.chkpt" at 2024-07-07 15:40:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 6 / 8 / 100:  20%|████▌                  | 100/500 [12:43<50:54,  7.64s/it]textattack: Saving checkpoint under "checkpoints\1720338373366.ta.chkpt" at 2024-07-07 15:46:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 129 / 11 / 10 / 150:  30%|██████              | 150/500 [19:25<45:18,  7.77s/it]textattack: Saving checkpoint under "checkpoints\1720338774842.ta.chkpt" at 2024-07-07 15:52:54 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 290 / 29 / 31 / 350:  70%|██████████████      | 350/500 [47:05<20:10,  8.07s/it]textattack: Saving checkpoint under "checkpoints\1720340434902.ta.chkpt" at 2024-07-07 16:20:34 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 334 / 32 / 34 / 400:  80%|████████████████    | 400/500 [52:27<13:06,  7.87s/it]textattack: Saving checkpoint under "checkpoints\1720340757191.ta.chkpt" at 2024-07-07 16:25:57 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 374 / 35 / 41 / 450:  90%|██████████████████  | 450/500 [58:33<06:30,  7.81s/it]textattack: Saving checkpoint under "checkpoints\1720341123025.ta.chkpt" at 2024-07-07 16:32:03 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 417 / 39 / 44 / 500: 100%|██████████████████| 500/500 [1:05:24<00:00,  7.85s/it]textattack: Saving checkpoint under "checkpoints\1720341533929.ta.chkpt" at 2024-07-07 16:38:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 417 / 39 / 44 / 500: 100%|██████████████████| 500/500 [1:05:24<00:00,  7.85s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 417    |
| Number of failed attacks:     | 39     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 7.8%   |
| Attack success rate:          | 91.45% |
| Average perturbed word %:     | 21.89% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 228.86 |
+-------------------------------+--------+


In [35]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 15 / 4 / 50:  10%|██▏                   | 50/500 [07:56<1:11:31,  9.54s/it]textattack: Saving checkpoint under "checkpoints\1720342010848.ta.chkpt" at 2024-07-07 16:46:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 24 / 13 / 100:  20%|███▊               | 100/500 [15:52<1:03:30,  9.53s/it]textattack: Saving checkpoint under "checkpoints\1720342486650.ta.chkpt" at 2024-07-07 16:54:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 90 / 40 / 20 / 150:  30%|██████▎              | 150/500 [24:01<56:02,  9.61s/it]textattack: Saving checkpoint under "checkpoints\1720342975042.ta.chkpt" at 2024-07-07 17:02:55 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 218 / 86 / 46 / 350:  70%|██████████████      | 350/500 [58:06<24:54,  9.96s/it]textattack: Saving checkpoint under "checkpoints\1720345020176.ta.chkpt" at 2024-07-07 17:37:00 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 250 / 100 / 50 / 400:  80%|█████████████▌   | 400/500 [1:05:18<16:19,  9.80s/it]textattack: Saving checkpoint under "checkpoints\1720345452991.ta.chkpt" at 2024-07-07 17:44:12 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 281 / 110 / 59 / 450:  90%|███████████████▎ | 450/500 [1:13:33<08:10,  9.81s/it]textattack: Saving checkpoint under "checkpoints\1720345947073.ta.chkpt" at 2024-07-07 17:52:27 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 312 / 125 / 63 / 500: 100%|█████████████████| 500/500 [1:21:30<00:00,  9.78s/it]textattack: Saving checkpoint under "checkpoints\1720346424122.ta.chkpt" at 2024-07-07 18:00:24 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 312 / 125 / 63 / 500: 100%|█████████████████| 500/500 [1:21:30<00:00,  9.78s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 312    |
| Number of failed attacks:     | 125    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 71.4%  |
| Average perturbed word %:     | 23.26% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 307.03 |
+-------------------------------+--------+


In [36]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 42 / 3 / 50:  10%|██▎                    | 50/500 [15:06<2:16:00, 18.13s/it]textattack: Saving checkpoint under "checkpoints\1720347330924.ta.chkpt" at 2024-07-07 18:15:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 78 / 8 / 100:  20%|████                | 100/500 [31:28<2:05:53, 18.88s/it]textattack: Saving checkpoint under "checkpoints\1720348312622.ta.chkpt" at 2024-07-07 18:31:52 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 117 / 11 / 150:  30%|█████▍            | 150/500 [48:39<1:53:32, 19.46s/it]textattack: Saving checkpoint under "checkpoints\1720349343906.ta.chkpt" at 2024-07-07 18:49:03 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 46 / 269 / 35 / 350:  70%|████████████▌     | 350/500 [1:56:01<49:43, 19.89s/it]textattack: Saving checkpoint under "checkpoints\1720353385723.ta.chkpt" at 2024-07-07 19:56:25 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 309 / 40 / 400:  80%|██████████████▍   | 400/500 [2:11:46<32:56, 19.77s/it]textattack: Saving checkpoint under "checkpoints\1720354330807.ta.chkpt" at 2024-07-07 20:12:10 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 346 / 43 / 450:  90%|████████████████▏ | 450/500 [2:27:56<16:26, 19.73s/it]textattack: Saving checkpoint under "checkpoints\1720355300782.ta.chkpt" at 2024-07-07 20:28:20 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 386 / 45 / 500: 100%|██████████████████| 500/500 [2:43:14<00:00, 19.59s/it]textattack: Saving checkpoint under "checkpoints\1720356218914.ta.chkpt" at 2024-07-07 20:43:38 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 69 / 386 / 45 / 500: 100%|██████████████████| 500/500 [2:43:14<00:00, 19.59s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 69     |
| Number of failed attacks:     | 386    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 77.2%  |
| Attack success rate:          | 15.16% |
| Average perturbed word %:     | 6.46%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 451.73 |
+-------------------------------+--------+


In [37]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 41 / 5 / 4 / 50:  10%|██▎                    | 50/500 [08:09<1:13:25,  9.79s/it]textattack: Saving checkpoint under "checkpoints\1720356708491.ta.chkpt" at 2024-07-07 20:51:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 84 / 7 / 9 / 100:  20%|████▌                  | 100/500 [14:36<58:27,  8.77s/it]textattack: Saving checkpoint under "checkpoints\1720357095949.ta.chkpt" at 2024-07-07 20:58:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 126 / 10 / 14 / 150:  30%|██████              | 150/500 [21:55<51:10,  8.77s/it]textattack: Saving checkpoint under "checkpoints\1720357534789.ta.chkpt" at 2024-07-07 21:05:34 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 295 / 22 / 33 / 350:  70%|██████████████      | 350/500 [50:40<21:42,  8.69s/it]textattack: Saving checkpoint under "checkpoints\1720359259097.ta.chkpt" at 2024-07-07 21:34:19 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 340 / 23 / 37 / 400:  80%|████████████████    | 400/500 [56:02<14:00,  8.41s/it]textattack: Saving checkpoint under "checkpoints\1720359581216.ta.chkpt" at 2024-07-07 21:39:41 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 380 / 25 / 45 / 450:  90%|████████████████▏ | 450/500 [1:02:42<06:58,  8.36s/it]textattack: Saving checkpoint under "checkpoints\1720359981695.ta.chkpt" at 2024-07-07 21:46:21 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 425 / 29 / 46 / 500: 100%|██████████████████| 500/500 [1:10:16<00:00,  8.43s/it]textattack: Saving checkpoint under "checkpoints\1720360435424.ta.chkpt" at 2024-07-07 21:53:55 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 425 / 29 / 46 / 500: 100%|██████████████████| 500/500 [1:10:16<00:00,  8.43s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 425    |
| Number of failed attacks:     | 29     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 5.8%   |
| Attack success rate:          | 93.61% |
| Average perturbed word %:     | 21.24% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 221.35 |
+-------------------------------+--------+


In [38]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 15 / 4 / 50:  10%|██▏                   | 50/500 [08:27<1:16:06, 10.15s/it]textattack: Saving checkpoint under "checkpoints\1720360942961.ta.chkpt" at 2024-07-07 22:02:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 24 / 13 / 100:  20%|███▊               | 100/500 [16:55<1:07:42, 10.16s/it]textattack: Saving checkpoint under "checkpoints\1720361451213.ta.chkpt" at 2024-07-07 22:10:51 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 92 / 38 / 20 / 150:  30%|█████▋             | 150/500 [26:43<1:02:20, 10.69s/it]textattack: Saving checkpoint under "checkpoints\1720362038608.ta.chkpt" at 2024-07-07 22:20:38 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 208 / 90 / 52 / 350:  70%|████████████▌     | 350/500 [1:01:33<26:23, 10.55s/it]textattack: Saving checkpoint under "checkpoints\1720364129351.ta.chkpt" at 2024-07-07 22:55:29 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 234 / 107 / 59 / 400:  80%|█████████████▌   | 400/500 [1:09:00<17:15, 10.35s/it]textattack: Saving checkpoint under "checkpoints\1720364576037.ta.chkpt" at 2024-07-07 23:02:56 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 260 / 119 / 71 / 450:  90%|███████████████▎ | 450/500 [1:17:46<08:38, 10.37s/it]textattack: Saving checkpoint under "checkpoints\1720365101886.ta.chkpt" at 2024-07-07 23:11:41 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 293 / 134 / 73 / 500: 100%|█████████████████| 500/500 [1:27:59<00:00, 10.56s/it]textattack: Saving checkpoint under "checkpoints\1720365715056.ta.chkpt" at 2024-07-07 23:21:55 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 293 / 134 / 73 / 500: 100%|█████████████████| 500/500 [1:27:59<00:00, 10.56s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 293    |
| Number of failed attacks:     | 134    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 26.8%  |
| Attack success rate:          | 68.62% |
| Average perturbed word %:     | 18.84% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 292.24 |
+-------------------------------+--------+


In [39]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 46 / 1 / 50:  10%|██▎                    | 50/500 [17:43<2:39:31, 21.27s/it]textattack: Saving checkpoint under "checkpoints\1720366778602.ta.chkpt" at 2024-07-07 23:39:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 81 / 4 / 100:  20%|████                | 100/500 [35:26<2:21:46, 21.27s/it]textattack: Saving checkpoint under "checkpoints\1720367841743.ta.chkpt" at 2024-07-07 23:57:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 121 / 7 / 150:  30%|█████▋             | 150/500 [53:40<2:05:14, 21.47s/it]textattack: Saving checkpoint under "checkpoints\1720368935837.ta.chkpt" at 2024-07-08 00:15:35 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 59 / 265 / 26 / 350:  70%|████████████▌     | 350/500 [2:01:30<52:04, 20.83s/it]textattack: Saving checkpoint under "checkpoints\1720373005713.ta.chkpt" at 2024-07-08 01:23:25 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 304 / 28 / 400:  80%|██████████████▍   | 400/500 [2:17:04<34:16, 20.56s/it]textattack: Saving checkpoint under "checkpoints\1720373939922.ta.chkpt" at 2024-07-08 01:38:59 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 342 / 35 / 450:  90%|████████████████▏ | 450/500 [2:34:09<17:07, 20.56s/it]textattack: Saving checkpoint under "checkpoints\1720374965077.ta.chkpt" at 2024-07-08 01:56:05 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 76 / 389 / 35 / 500: 100%|██████████████████| 500/500 [2:52:18<00:00, 20.68s/it]textattack: Saving checkpoint under "checkpoints\1720376053881.ta.chkpt" at 2024-07-08 02:14:13 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 76 / 389 / 35 / 500: 100%|██████████████████| 500/500 [2:52:18<00:00, 20.68s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 76     |
| Number of failed attacks:     | 389    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 77.8%  |
| Attack success rate:          | 16.34% |
| Average perturbed word %:     | 5.23%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 445.18 |
+-------------------------------+--------+


In [40]:
et2 = datetime.datetime.now()
et2 - et

datetime.timedelta(seconds=52837, microseconds=308499)

### Threshold = 0.9

In [41]:
suffix = 'ep09.csv'
thre = 0.9

In [42]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 40 / 5 / 5 / 50:  10%|██▎                    | 50/500 [08:16<1:14:24,  9.92s/it]textattack: Saving checkpoint under "checkpoints\1720376550199.ta.chkpt" at 2024-07-08 02:22:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 81 / 9 / 10 / 100:  20%|████                | 100/500 [16:19<1:05:16,  9.79s/it]textattack: Saving checkpoint under "checkpoints\1720377033422.ta.chkpt" at 2024-07-08 02:30:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 119 / 16 / 15 / 150:  30%|██████              | 150/500 [25:10<58:44, 10.07s/it]textattack: Saving checkpoint under "checkpoints\1720377564794.ta.chkpt" at 2024-07-08 02:39:24 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 278 / 35 / 37 / 350:  70%|████████████▌     | 350/500 [1:00:48<26:03, 10.43s/it]textattack: Saving checkpoint under "checkpoints\1720379702998.ta.chkpt" at 2024-07-08 03:15:02 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 318 / 41 / 41 / 400:  80%|██████████████▍   | 400/500 [1:07:32<16:53, 10.13s/it]textattack: Saving checkpoint under "checkpoints\1720380106570.ta.chkpt" at 2024-07-08 03:21:46 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 353 / 44 / 53 / 450:  90%|████████████████▏ | 450/500 [1:14:45<08:18,  9.97s/it]textattack: Saving checkpoint under "checkpoints\1720380539209.ta.chkpt" at 2024-07-08 03:28:59 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 391 / 52 / 57 / 500: 100%|██████████████████| 500/500 [1:23:30<00:00, 10.02s/it]textattack: Saving checkpoint under "checkpoints\1720381064322.ta.chkpt" at 2024-07-08 03:37:44 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 391 / 52 / 57 / 500: 100%|██████████████████| 500/500 [1:23:30<00:00, 10.02s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 391    |
| Number of failed attacks:     | 52     |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 10.4%  |
| Attack success rate:          | 88.26% |
| Average perturbed word %:     | 18.2%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 162.19 |
+-------------------------------+--------+


In [43]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 32 / 3 / 15 / 50:  10%|██▍                     | 50/500 [04:26<40:00,  5.33s/it]textattack: Saving checkpoint under "checkpoints\1720381331143.ta.chkpt" at 2024-07-08 03:42:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 6 / 31 / 100:  20%|████▍                 | 100/500 [09:03<36:13,  5.43s/it]textattack: Saving checkpoint under "checkpoints\1720381607806.ta.chkpt" at 2024-07-08 03:46:47 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 63 / 6 / 32 / 101:  20%|████▍                 | 101/500 [09:03<35:46,  5.38s/it]

[Succeeded / Failed / Skipped / Total] 98 / 10 / 42 / 150:  30%|██████▎              | 150/500 [13:49<32:15,  5.53s/it]textattack: Saving checkpoint under "checkpoints\1720381893959.ta.chkpt" at 2024-07-08 03:51:33 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 230 / 19 / 101 / 350:  70%|█████████████▎     | 350/500 [34:33<14:48,  5.93s/it]textattack: Saving checkpoint under "checkpoints\1720383138380.ta.chkpt" at 2024-07-08 04:12:18 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 268 / 20 / 112 / 400:  80%|███████████████▏   | 400/500 [38:43<09:40,  5.81s/it]textattack: Saving checkpoint under "checkpoints\1720383387499.ta.chkpt" at 2024-07-08 04:16:27 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 299 / 22 / 129 / 450:  90%|█████████████████  | 450/500 [43:54<04:52,  5.86s/it]textattack: Saving checkpoint under "checkpoints\1720383699269.ta.chkpt" at 2024-07-08 04:21:39 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 331 / 28 / 141 / 500: 100%|███████████████████| 500/500 [49:34<00:00,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1720384038516.ta.chkpt" at 2024-07-08 04:27:18 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 331 / 28 / 141 / 500: 100%|███████████████████| 500/500 [49:34<00:00,  5.95s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 331    |
| Number of failed attacks:     | 28     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 5.6%   |
| Attack success rate:          | 92.2%  |
| Average perturbed word %:     | 10.31% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 118.11 |
+-------------------------------+--------+


In [44]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 28 / 11 / 11 / 50:  10%|██                   | 50/500 [07:47<1:10:05,  9.35s/it]textattack: Saving checkpoint under "checkpoints\1720384505935.ta.chkpt" at 2024-07-08 04:35:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 16 / 20 / 100:  20%|████▏                | 100/500 [14:48<59:14,  8.89s/it]textattack: Saving checkpoint under "checkpoints\1720384927364.ta.chkpt" at 2024-07-08 04:42:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 27 / 32 / 150:  30%|██████▎              | 150/500 [23:23<54:34,  9.36s/it]textattack: Saving checkpoint under "checkpoints\1720385442114.ta.chkpt" at 2024-07-08 04:50:42 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 216 / 61 / 73 / 350:  70%|██████████████      | 350/500 [58:58<25:16, 10.11s/it]textattack: Saving checkpoint under "checkpoints\1720387576984.ta.chkpt" at 2024-07-08 05:26:16 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 242 / 73 / 85 / 400:  80%|██████████████▍   | 400/500 [1:06:36<16:39,  9.99s/it]textattack: Saving checkpoint under "checkpoints\1720388035272.ta.chkpt" at 2024-07-08 05:33:55 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 267 / 84 / 99 / 450:  90%|████████████████▏ | 450/500 [1:15:03<08:20, 10.01s/it]textattack: Saving checkpoint under "checkpoints\1720388542264.ta.chkpt" at 2024-07-08 05:42:22 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 298 / 93 / 109 / 500: 100%|█████████████████| 500/500 [1:22:33<00:00,  9.91s/it]textattack: Saving checkpoint under "checkpoints\1720388992139.ta.chkpt" at 2024-07-08 05:49:52 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 298 / 93 / 109 / 500: 100%|█████████████████| 500/500 [1:22:33<00:00,  9.91s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 298    |
| Number of failed attacks:     | 93     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 18.6%  |
| Attack success rate:          | 76.21% |
| Average perturbed word %:     | 5.2%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 149.41 |
+-------------------------------+--------+


In [45]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 27 / 19 / 4 / 50:  10%|██▍                     | 50/500 [06:28<58:16,  7.77s/it]textattack: Saving checkpoint under "checkpoints\1720389381232.ta.chkpt" at 2024-07-08 05:56:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 28 / 8 / 100:  20%|████▍                 | 100/500 [12:43<50:53,  7.63s/it]textattack: Saving checkpoint under "checkpoints\1720389756174.ta.chkpt" at 2024-07-08 06:02:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 98 / 42 / 10 / 150:  30%|██████▎              | 150/500 [18:59<44:18,  7.60s/it]textattack: Saving checkpoint under "checkpoints\1720390132219.ta.chkpt" at 2024-07-08 06:08:52 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 229 / 90 / 31 / 350:  70%|██████████████      | 350/500 [44:32<19:05,  7.64s/it]textattack: Saving checkpoint under "checkpoints\1720391665186.ta.chkpt" at 2024-07-08 06:34:25 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 263 / 103 / 34 / 400:  80%|███████████████▏   | 400/500 [49:51<12:27,  7.48s/it]textattack: Saving checkpoint under "checkpoints\1720391983821.ta.chkpt" at 2024-07-08 06:39:43 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 299 / 110 / 41 / 450:  90%|█████████████████  | 450/500 [55:19<06:08,  7.38s/it]textattack: Saving checkpoint under "checkpoints\1720392312684.ta.chkpt" at 2024-07-08 06:45:12 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 329 / 127 / 44 / 500: 100%|█████████████████| 500/500 [1:01:15<00:00,  7.35s/it]textattack: Saving checkpoint under "checkpoints\1720392668440.ta.chkpt" at 2024-07-08 06:51:08 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 329 / 127 / 44 / 500: 100%|█████████████████| 500/500 [1:01:15<00:00,  7.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 329    |
| Number of failed attacks:     | 127    |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 25.4%  |
| Attack success rate:          | 72.15% |
| Average perturbed word %:     | 23.34% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 196.23 |
+-------------------------------+--------+


In [46]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 16 / 30 / 4 / 50:  10%|██▏                   | 50/500 [06:40<1:00:06,  8.01s/it]textattack: Saving checkpoint under "checkpoints\1720393069274.ta.chkpt" at 2024-07-08 06:57:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 49 / 13 / 100:  20%|████▏                | 100/500 [13:59<55:56,  8.39s/it]textattack: Saving checkpoint under "checkpoints\1720393507633.ta.chkpt" at 2024-07-08 07:05:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 62 / 68 / 20 / 150:  30%|██████▎              | 150/500 [21:03<49:07,  8.42s/it]textattack: Saving checkpoint under "checkpoints\1720393931551.ta.chkpt" at 2024-07-08 07:12:11 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 144 / 160 / 46 / 350:  70%|█████████████▎     | 350/500 [50:22<21:35,  8.63s/it]textattack: Saving checkpoint under "checkpoints\1720395690726.ta.chkpt" at 2024-07-08 07:41:30 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 166 / 184 / 50 / 400:  80%|███████████████▏   | 400/500 [56:15<14:03,  8.44s/it]textattack: Saving checkpoint under "checkpoints\1720396044515.ta.chkpt" at 2024-07-08 07:47:24 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 184 / 207 / 59 / 450:  90%|███████████████▎ | 450/500 [1:02:57<06:59,  8.39s/it]textattack: Saving checkpoint under "checkpoints\1720396445935.ta.chkpt" at 2024-07-08 07:54:05 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 205 / 232 / 63 / 500: 100%|█████████████████| 500/500 [1:09:27<00:00,  8.33s/it]textattack: Saving checkpoint under "checkpoints\1720396835739.ta.chkpt" at 2024-07-08 08:00:35 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 205 / 232 / 63 / 500: 100%|█████████████████| 500/500 [1:09:27<00:00,  8.33s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 205    |
| Number of failed attacks:     | 232    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 46.4%  |
| Attack success rate:          | 46.91% |
| Average perturbed word %:     | 19.88% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 234.78 |
+-------------------------------+--------+


In [47]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 44 / 3 / 50:  10%|██▎                    | 50/500 [11:35<1:44:21, 13.91s/it]textattack: Saving checkpoint under "checkpoints\1720397531541.ta.chkpt" at 2024-07-08 08:12:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 80 / 8 / 100:  20%|████                | 100/500 [24:07<1:36:31, 14.48s/it]textattack: Saving checkpoint under "checkpoints\1720398283745.ta.chkpt" at 2024-07-08 08:24:43 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 121 / 11 / 150:  30%|█████▍            | 150/500 [37:46<1:28:07, 15.11s/it]textattack: Saving checkpoint under "checkpoints\1720399102116.ta.chkpt" at 2024-07-08 08:38:22 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 40 / 275 / 35 / 350:  70%|████████████▌     | 350/500 [1:28:25<37:53, 15.16s/it]textattack: Saving checkpoint under "checkpoints\1720402141251.ta.chkpt" at 2024-07-08 09:29:01 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 316 / 40 / 400:  80%|██████████████▍   | 400/500 [1:39:50<24:57, 14.98s/it]textattack: Saving checkpoint under "checkpoints\1720402826408.ta.chkpt" at 2024-07-08 09:40:26 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 353 / 43 / 450:  90%|████████████████▏ | 450/500 [1:52:25<12:29, 14.99s/it]textattack: Saving checkpoint under "checkpoints\1720403581642.ta.chkpt" at 2024-07-08 09:53:01 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 395 / 45 / 500: 100%|██████████████████| 500/500 [2:03:55<00:00, 14.87s/it]textattack: Saving checkpoint under "checkpoints\1720404271736.ta.chkpt" at 2024-07-08 10:04:31 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 60 / 395 / 45 / 500: 100%|██████████████████| 500/500 [2:03:55<00:00, 14.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 60     |
| Number of failed attacks:     | 395    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 79.0%  |
| Attack success rate:          | 13.19% |
| Average perturbed word %:     | 6.06%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 302.75 |
+-------------------------------+--------+


In [48]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 26 / 20 / 4 / 50:  10%|██▏                   | 50/500 [06:50<1:01:30,  8.20s/it]textattack: Saving checkpoint under "checkpoints\1720404681908.ta.chkpt" at 2024-07-08 10:11:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 28 / 9 / 100:  20%|████▍                 | 100/500 [13:31<54:07,  8.12s/it]textattack: Saving checkpoint under "checkpoints\1720405083655.ta.chkpt" at 2024-07-08 10:18:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 97 / 39 / 14 / 150:  30%|██████▎              | 150/500 [20:00<46:41,  8.01s/it]textattack: Saving checkpoint under "checkpoints\1720405472647.ta.chkpt" at 2024-07-08 10:24:32 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 235 / 82 / 33 / 350:  70%|██████████████      | 350/500 [47:02<20:09,  8.07s/it]textattack: Saving checkpoint under "checkpoints\1720407094653.ta.chkpt" at 2024-07-08 10:51:34 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 269 / 94 / 37 / 400:  80%|████████████████    | 400/500 [53:05<13:16,  7.96s/it]textattack: Saving checkpoint under "checkpoints\1720407457624.ta.chkpt" at 2024-07-08 10:57:37 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 296 / 109 / 45 / 450:  90%|█████████████████  | 450/500 [59:41<06:37,  7.96s/it]textattack: Saving checkpoint under "checkpoints\1720407853423.ta.chkpt" at 2024-07-08 11:04:13 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 328 / 126 / 46 / 500: 100%|█████████████████| 500/500 [1:06:21<00:00,  7.96s/it]textattack: Saving checkpoint under "checkpoints\1720408253210.ta.chkpt" at 2024-07-08 11:10:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 328 / 126 / 46 / 500: 100%|█████████████████| 500/500 [1:06:21<00:00,  7.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 328    |
| Number of failed attacks:     | 126    |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 25.2%  |
| Attack success rate:          | 72.25% |
| Average perturbed word %:     | 22.65% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 193.38 |
+-------------------------------+--------+


In [49]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 20 / 26 / 4 / 50:  10%|██▏                   | 50/500 [07:40<1:09:06,  9.21s/it]textattack: Saving checkpoint under "checkpoints\1720408714053.ta.chkpt" at 2024-07-08 11:18:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 46 / 13 / 100:  20%|███▊               | 100/500 [15:20<1:01:23,  9.21s/it]textattack: Saving checkpoint under "checkpoints\1720409174113.ta.chkpt" at 2024-07-08 11:26:14 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 70 / 20 / 150:  30%|██████▎              | 150/500 [24:03<56:07,  9.62s/it]textattack: Saving checkpoint under "checkpoints\1720409696654.ta.chkpt" at 2024-07-08 11:34:56 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 152 / 146 / 52 / 350:  70%|█████████████▎     | 350/500 [52:43<22:35,  9.04s/it]textattack: Saving checkpoint under "checkpoints\1720411416348.ta.chkpt" at 2024-07-08 12:03:36 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 171 / 170 / 59 / 400:  80%|███████████████▏   | 400/500 [58:58<14:44,  8.85s/it]textattack: Saving checkpoint under "checkpoints\1720411792155.ta.chkpt" at 2024-07-08 12:09:52 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 189 / 190 / 71 / 450:  90%|███████████████▎ | 450/500 [1:06:08<07:20,  8.82s/it]textattack: Saving checkpoint under "checkpoints\1720412221860.ta.chkpt" at 2024-07-08 12:17:01 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 209 / 218 / 73 / 500: 100%|█████████████████| 500/500 [1:14:10<00:00,  8.90s/it]textattack: Saving checkpoint under "checkpoints\1720412703770.ta.chkpt" at 2024-07-08 12:25:03 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 209 / 218 / 73 / 500: 100%|█████████████████| 500/500 [1:14:10<00:00,  8.90s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 209    |
| Number of failed attacks:     | 218    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 43.6%  |
| Attack success rate:          | 48.95% |
| Average perturbed word %:     | 16.7%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 224.84 |
+-------------------------------+--------+


In [50]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 2 / 47 / 1 / 50:  10%|██▎                    | 50/500 [12:27<1:52:03, 14.94s/it]textattack: Saving checkpoint under "checkpoints\1720413450885.ta.chkpt" at 2024-07-08 12:37:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 86 / 4 / 100:  20%|████                | 100/500 [26:11<1:44:46, 15.72s/it]textattack: Saving checkpoint under "checkpoints\1720414275542.ta.chkpt" at 2024-07-08 12:51:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 129 / 7 / 150:  30%|█████▋             | 150/500 [39:26<1:32:02, 15.78s/it]textattack: Saving checkpoint under "checkpoints\1720415070543.ta.chkpt" at 2024-07-08 13:04:30 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 44 / 280 / 26 / 350:  70%|████████████▌     | 350/500 [1:30:16<38:41, 15.47s/it]textattack: Saving checkpoint under "checkpoints\1720418119980.ta.chkpt" at 2024-07-08 13:55:19 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 322 / 28 / 400:  80%|██████████████▍   | 400/500 [1:42:08<25:32, 15.32s/it]textattack: Saving checkpoint under "checkpoints\1720418831931.ta.chkpt" at 2024-07-08 14:07:11 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 360 / 35 / 450:  90%|████████████████▏ | 450/500 [1:54:42<12:44, 15.29s/it]textattack: Saving checkpoint under "checkpoints\1720419586380.ta.chkpt" at 2024-07-08 14:19:46 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 407 / 35 / 500: 100%|██████████████████| 500/500 [2:07:41<00:00, 15.32s/it]textattack: Saving checkpoint under "checkpoints\1720420365081.ta.chkpt" at 2024-07-08 14:32:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 58 / 407 / 35 / 500: 100%|██████████████████| 500/500 [2:07:41<00:00, 15.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 58     |
| Number of failed attacks:     | 407    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 81.4%  |
| Attack success rate:          | 12.47% |
| Average perturbed word %:     | 5.49%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 300.24 |
+-------------------------------+--------+


In [51]:
et3 = datetime.datetime.now()
et3 - et2

datetime.timedelta(seconds=44311, microseconds=199429)

### Threshold = 0.95

In [52]:
suffix = 'ep095.csv'
thre = 0.95

In [53]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  10%|██▌                      | 50/500 [03:45<33:45,  4.50s/it]textattack: Saving checkpoint under "checkpoints\1720420590387.ta.chkpt" at 2024-07-08 14:36:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 87 / 10 / 100:  20%|████▍                 | 100/500 [07:50<31:22,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720420835997.ta.chkpt" at 2024-07-08 14:40:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 129 / 15 / 150:  30%|██████▎              | 150/500 [11:55<27:49,  4.77s/it]textattack: Saving checkpoint under "checkpoints\1720421080969.ta.chkpt" at 2024-07-08 14:44:40 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 13 / 300 / 37 / 350:  70%|██████████████      | 350/500 [28:26<12:11,  4.88s/it]textattack: Saving checkpoint under "checkpoints\1720422071620.ta.chkpt" at 2024-07-08 15:01:11 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 344 / 41 / 400:  80%|████████████████    | 400/500 [32:02<08:00,  4.81s/it]textattack: Saving checkpoint under "checkpoints\1720422288107.ta.chkpt" at 2024-07-08 15:04:48 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 381 / 53 / 450:  90%|██████████████████  | 450/500 [35:27<03:56,  4.73s/it]textattack: Saving checkpoint under "checkpoints\1720422492655.ta.chkpt" at 2024-07-08 15:08:12 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 426 / 57 / 500: 100%|████████████████████| 500/500 [39:02<00:00,  4.68s/it]textattack: Saving checkpoint under "checkpoints\1720422707353.ta.chkpt" at 2024-07-08 15:11:47 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 426 / 57 / 500: 100%|████████████████████| 500/500 [39:02<00:00,  4.68s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 17    |
| Number of failed attacks:     | 426   |
| Number of skipped attacks:    | 57    |
| Original accuracy:            | 88.6% |
| Accuracy under attack:        | 85.2% |
| Attack success rate:          | 3.84% |
| Average perturbed word %:     | 7.28% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 37.04 |
+-------------------------------+-------+


In [54]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 10 / 25 / 15 / 50:  10%|██▎                    | 50/500 [02:30<22:31,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1720422857657.ta.chkpt" at 2024-07-08 15:14:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 54 / 31 / 100:  20%|████▏                | 100/500 [05:21<21:24,  3.21s/it]textattack: Saving checkpoint under "checkpoints\1720423028598.ta.chkpt" at 2024-07-08 15:17:08 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 15 / 54 / 32 / 101:  20%|████▏                | 101/500 [05:21<21:09,  3.18s/it]

[Succeeded / Failed / Skipped / Total] 20 / 88 / 42 / 150:  30%|██████▎              | 150/500 [08:52<20:42,  3.55s/it]textattack: Saving checkpoint under "checkpoints\1720423240173.ta.chkpt" at 2024-07-08 15:20:40 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 43 / 206 / 101 / 350:  70%|█████████████▎     | 350/500 [21:21<09:09,  3.66s/it]textattack: Saving checkpoint under "checkpoints\1720423988855.ta.chkpt" at 2024-07-08 15:33:08 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 243 / 112 / 400:  80%|███████████████▏   | 400/500 [24:25<06:06,  3.66s/it]textattack: Saving checkpoint under "checkpoints\1720424172666.ta.chkpt" at 2024-07-08 15:36:12 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 49 / 272 / 129 / 450:  90%|█████████████████  | 450/500 [27:13<03:01,  3.63s/it]textattack: Saving checkpoint under "checkpoints\1720424340967.ta.chkpt" at 2024-07-08 15:39:00 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 307 / 141 / 500: 100%|███████████████████| 500/500 [30:11<00:00,  3.62s/it]textattack: Saving checkpoint under "checkpoints\1720424519371.ta.chkpt" at 2024-07-08 15:41:59 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 52 / 307 / 141 / 500: 100%|███████████████████| 500/500 [30:11<00:00,  3.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 52     |
| Number of failed attacks:     | 307    |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 61.4%  |
| Attack success rate:          | 14.48% |
| Average perturbed word %:     | 5.24%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 36.28  |
+-------------------------------+--------+


In [55]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 6 / 33 / 11 / 50:  10%|██▍                     | 50/500 [03:40<33:08,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1720424740438.ta.chkpt" at 2024-07-08 15:45:40 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 62 / 20 / 100:  20%|████▏                | 100/500 [07:34<30:17,  4.54s/it]textattack: Saving checkpoint under "checkpoints\1720424973910.ta.chkpt" at 2024-07-08 15:49:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 90 / 32 / 150:  30%|██████▎              | 150/500 [11:24<26:36,  4.56s/it]textattack: Saving checkpoint under "checkpoints\1720425203697.ta.chkpt" at 2024-07-08 15:53:23 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 52 / 225 / 73 / 350:  70%|██████████████      | 350/500 [28:24<12:10,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1720426224345.ta.chkpt" at 2024-07-08 16:10:24 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 258 / 85 / 400:  80%|████████████████    | 400/500 [31:43<07:55,  4.76s/it]textattack: Saving checkpoint under "checkpoints\1720426423128.ta.chkpt" at 2024-07-08 16:13:43 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 290 / 99 / 450:  90%|██████████████████  | 450/500 [35:32<03:56,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1720426651841.ta.chkpt" at 2024-07-08 16:17:31 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 65 / 326 / 109 / 500: 100%|███████████████████| 500/500 [39:19<00:00,  4.72s/it]textattack: Saving checkpoint under "checkpoints\1720426879421.ta.chkpt" at 2024-07-08 16:21:19 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 65 / 326 / 109 / 500: 100%|███████████████████| 500/500 [39:19<00:00,  4.72s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 65     |
| Number of failed attacks:     | 326    |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 65.2%  |
| Attack success rate:          | 16.62% |
| Average perturbed word %:     | 3.8%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 36.82  |
+-------------------------------+--------+


In [56]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  10%|██▌                      | 50/500 [03:05<27:53,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1720427065530.ta.chkpt" at 2024-07-08 16:24:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 91 / 8 / 100:  20%|████▌                  | 100/500 [06:30<26:02,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1720427270055.ta.chkpt" at 2024-07-08 16:27:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 135 / 10 / 150:  30%|██████▎              | 150/500 [09:52<23:02,  3.95s/it]textattack: Saving checkpoint under "checkpoints\1720427472178.ta.chkpt" at 2024-07-08 16:31:12 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 8 / 311 / 31 / 350:  70%|██████████████▋      | 350/500 [23:07<09:54,  3.96s/it]textattack: Saving checkpoint under "checkpoints\1720428266698.ta.chkpt" at 2024-07-08 16:44:26 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 358 / 34 / 400:  80%|████████████████▊    | 400/500 [26:08<06:32,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720428448417.ta.chkpt" at 2024-07-08 16:47:28 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 400 / 41 / 450:  90%|██████████████████▉  | 450/500 [29:11<03:14,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1720428630867.ta.chkpt" at 2024-07-08 16:50:30 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 446 / 44 / 500: 100%|████████████████████| 500/500 [32:08<00:00,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1720428807923.ta.chkpt" at 2024-07-08 16:53:27 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 10 / 446 / 44 / 500: 100%|████████████████████| 500/500 [32:08<00:00,  3.86s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 10    |
| Number of failed attacks:     | 446   |
| Number of skipped attacks:    | 44    |
| Original accuracy:            | 91.2% |
| Accuracy under attack:        | 89.2% |
| Attack success rate:          | 2.19% |
| Average perturbed word %:     | 5.86% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 37.27 |
+-------------------------------+-------+


In [57]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  10%|██▌                      | 50/500 [03:02<27:20,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1720428990377.ta.chkpt" at 2024-07-08 16:56:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 87 / 13 / 100:  20%|████▍                 | 100/500 [06:04<24:16,  3.64s/it]textattack: Saving checkpoint under "checkpoints\1720429172218.ta.chkpt" at 2024-07-08 16:59:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 129 / 20 / 150:  30%|██████▎              | 150/500 [09:13<21:31,  3.69s/it]textattack: Saving checkpoint under "checkpoints\1720429361683.ta.chkpt" at 2024-07-08 17:02:41 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 6 / 298 / 46 / 350:  70%|██████████████▋      | 350/500 [22:07<09:28,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1720430135130.ta.chkpt" at 2024-07-08 17:15:35 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 341 / 50 / 400:  80%|████████████████▊    | 400/500 [24:57<06:14,  3.74s/it]textattack: Saving checkpoint under "checkpoints\1720430305815.ta.chkpt" at 2024-07-08 17:18:25 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 381 / 59 / 450:  90%|██████████████████  | 450/500 [27:51<03:05,  3.71s/it]textattack: Saving checkpoint under "checkpoints\1720430479312.ta.chkpt" at 2024-07-08 17:21:19 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 425 / 63 / 500: 100%|████████████████████| 500/500 [30:42<00:00,  3.68s/it]textattack: Saving checkpoint under "checkpoints\1720430650198.ta.chkpt" at 2024-07-08 17:24:10 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 12 / 425 / 63 / 500: 100%|████████████████████| 500/500 [30:42<00:00,  3.68s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 12    |
| Number of failed attacks:     | 425   |
| Number of skipped attacks:    | 63    |
| Original accuracy:            | 87.4% |
| Accuracy under attack:        | 85.0% |
| Attack success rate:          | 2.75% |
| Average perturbed word %:     | 5.96% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 37.03 |
+-------------------------------+-------+


In [58]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 46 / 3 / 50:  10%|██▌                      | 50/500 [03:44<33:37,  4.48s/it]textattack: Saving checkpoint under "checkpoints\1720430874470.ta.chkpt" at 2024-07-08 17:27:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 90 / 8 / 100:  20%|████▌                  | 100/500 [07:39<30:37,  4.59s/it]textattack: Saving checkpoint under "checkpoints\1720431109625.ta.chkpt" at 2024-07-08 17:31:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 136 / 11 / 150:  30%|██████▎              | 150/500 [11:56<27:52,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1720431367254.ta.chkpt" at 2024-07-08 17:36:07 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 9 / 306 / 35 / 350:  70%|██████████████▋      | 350/500 [27:54<11:57,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1720432324439.ta.chkpt" at 2024-07-08 17:52:04 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 350 / 40 / 400:  80%|████████████████    | 400/500 [31:22<07:50,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720432533283.ta.chkpt" at 2024-07-08 17:55:33 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 395 / 43 / 450:  90%|██████████████████  | 450/500 [35:26<03:56,  4.72s/it]textattack: Saving checkpoint under "checkpoints\1720432776425.ta.chkpt" at 2024-07-08 17:59:36 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 441 / 45 / 500: 100%|████████████████████| 500/500 [39:07<00:00,  4.69s/it]textattack: Saving checkpoint under "checkpoints\1720432997507.ta.chkpt" at 2024-07-08 18:03:17 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 14 / 441 / 45 / 500: 100%|████████████████████| 500/500 [39:07<00:00,  4.69s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 14    |
| Number of failed attacks:     | 441   |
| Number of skipped attacks:    | 45    |
| Original accuracy:            | 91.0% |
| Accuracy under attack:        | 88.2% |
| Attack success rate:          | 3.08% |
| Average perturbed word %:     | 4.28% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 37.3  |
+-------------------------------+-------+


In [59]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  10%|██▌                      | 50/500 [03:14<29:11,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1720433192264.ta.chkpt" at 2024-07-08 18:06:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  20%|████▌                  | 100/500 [06:43<26:52,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1720433400861.ta.chkpt" at 2024-07-08 18:10:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  30%|██████▎              | 150/500 [10:15<23:55,  4.10s/it]textattack: Saving checkpoint under "checkpoints\1720433613018.ta.chkpt" at 2024-07-08 18:13:33 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 4 / 313 / 33 / 350:  70%|██████████████▋      | 350/500 [24:19<10:25,  4.17s/it]textattack: Saving checkpoint under "checkpoints\1720434456902.ta.chkpt" at 2024-07-08 18:27:36 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 359 / 37 / 400:  80%|████████████████▊    | 400/500 [27:24<06:51,  4.11s/it]textattack: Saving checkpoint under "checkpoints\1720434642414.ta.chkpt" at 2024-07-08 18:30:42 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 400 / 45 / 450:  90%|██████████████████▉  | 450/500 [30:35<03:23,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1720434832898.ta.chkpt" at 2024-07-08 18:33:52 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 449 / 46 / 500: 100%|█████████████████████| 500/500 [33:52<00:00,  4.07s/it]textattack: Saving checkpoint under "checkpoints\1720435030380.ta.chkpt" at 2024-07-08 18:37:10 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 449 / 46 / 500: 100%|█████████████████████| 500/500 [33:52<00:00,  4.07s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 5     |
| Number of failed attacks:     | 449   |
| Number of skipped attacks:    | 46    |
| Original accuracy:            | 90.8% |
| Accuracy under attack:        | 89.8% |
| Attack success rate:          | 1.1%  |
| Average perturbed word %:     | 6.67% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 37.13 |
+-------------------------------+-------+


In [60]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 2 / 44 / 4 / 50:  10%|██▌                      | 50/500 [03:11<28:40,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1720435221660.ta.chkpt" at 2024-07-08 18:40:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 84 / 13 / 100:  20%|████▍                 | 100/500 [06:24<25:38,  3.85s/it]textattack: Saving checkpoint under "checkpoints\1720435415251.ta.chkpt" at 2024-07-08 18:43:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 126 / 20 / 150:  30%|██████▎              | 150/500 [09:44<22:43,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1720435615090.ta.chkpt" at 2024-07-08 18:46:55 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 14 / 284 / 52 / 350:  70%|██████████████      | 350/500 [22:18<09:33,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1720436368580.ta.chkpt" at 2024-07-08 18:59:28 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 325 / 59 / 400:  80%|████████████████    | 400/500 [25:07<06:16,  3.77s/it]textattack: Saving checkpoint under "checkpoints\1720436538000.ta.chkpt" at 2024-07-08 19:02:18 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 362 / 71 / 450:  90%|██████████████████  | 450/500 [27:57<03:06,  3.73s/it]textattack: Saving checkpoint under "checkpoints\1720436708525.ta.chkpt" at 2024-07-08 19:05:08 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 409 / 73 / 500: 100%|████████████████████| 500/500 [31:11<00:00,  3.74s/it]textattack: Saving checkpoint under "checkpoints\1720436902305.ta.chkpt" at 2024-07-08 19:08:22 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 18 / 409 / 73 / 500: 100%|████████████████████| 500/500 [31:11<00:00,  3.74s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 18    |
| Number of failed attacks:     | 409   |
| Number of skipped attacks:    | 73    |
| Original accuracy:            | 85.4% |
| Accuracy under attack:        | 81.8% |
| Attack success rate:          | 4.22% |
| Average perturbed word %:     | 4.51% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 36.46 |
+-------------------------------+-------+


In [61]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  10%|██▌                      | 50/500 [03:49<34:28,  4.60s/it]textattack: Saving checkpoint under "checkpoints\1720437132318.ta.chkpt" at 2024-07-08 19:12:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 95 / 4 / 100:  20%|████▌                  | 100/500 [07:54<31:36,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1720437376565.ta.chkpt" at 2024-07-08 19:16:16 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 140 / 7 / 150:  30%|██████▌               | 150/500 [11:57<27:53,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1720437619465.ta.chkpt" at 2024-07-08 19:20:19 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 6 / 318 / 26 / 350:  70%|██████████████▋      | 350/500 [28:22<12:09,  4.86s/it]textattack: Saving checkpoint under "checkpoints\1720438604645.ta.chkpt" at 2024-07-08 19:36:44 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 366 / 28 / 400:  80%|████████████████▊    | 400/500 [32:18<08:04,  4.85s/it]textattack: Saving checkpoint under "checkpoints\1720438841392.ta.chkpt" at 2024-07-08 19:40:41 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 408 / 35 / 450:  90%|██████████████████▉  | 450/500 [36:09<04:01,  4.82s/it]textattack: Saving checkpoint under "checkpoints\1720439072169.ta.chkpt" at 2024-07-08 19:44:32 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 458 / 35 / 500: 100%|█████████████████████| 500/500 [40:09<00:00,  4.82s/it]textattack: Saving checkpoint under "checkpoints\1720439311569.ta.chkpt" at 2024-07-08 19:48:31 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 7 / 458 / 35 / 500: 100%|█████████████████████| 500/500 [40:09<00:00,  4.82s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 7     |
| Number of failed attacks:     | 458   |
| Number of skipped attacks:    | 35    |
| Original accuracy:            | 93.0% |
| Accuracy under attack:        | 91.6% |
| Attack success rate:          | 1.51% |
| Average perturbed word %:     | 4.89% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 36.95 |
+-------------------------------+-------+


In [62]:
et4 = datetime.datetime.now()
et4 - et3

datetime.timedelta(seconds=18946, microseconds=527459)

### Threshold = 1 - 0.6/$\pi$ = 0.809

In [63]:
suffix = 'ep0809.csv'
thre = 1-0.6/np.pi

In [64]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 43 / 2 / 5 / 50:  10%|██▎                    | 50/500 [07:55<1:11:22,  9.52s/it]textattack: Saving checkpoint under "checkpoints\1720439787849.ta.chkpt" at 2024-07-08 19:56:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 2 / 10 / 100:  20%|████                | 100/500 [15:55<1:03:42,  9.56s/it]textattack: Saving checkpoint under "checkpoints\1720440267729.ta.chkpt" at 2024-07-08 20:04:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 133 / 2 / 15 / 150:  30%|█████▋             | 150/500 [26:46<1:02:29, 10.71s/it]textattack: Saving checkpoint under "checkpoints\1720440918973.ta.chkpt" at 2024-07-08 20:15:18 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 310 / 3 / 37 / 350:  70%|█████████████▎     | 350/500 [1:04:46<27:45, 11.10s/it]textattack: Saving checkpoint under "checkpoints\1720443198319.ta.chkpt" at 2024-07-08 20:53:18 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 355 / 4 / 41 / 400:  80%|███████████████▏   | 400/500 [1:12:05<18:01, 10.81s/it]textattack: Saving checkpoint under "checkpoints\1720443637220.ta.chkpt" at 2024-07-08 21:00:37 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 393 / 4 / 53 / 450:  90%|█████████████████  | 450/500 [1:20:07<08:54, 10.68s/it]textattack: Saving checkpoint under "checkpoints\1720444119753.ta.chkpt" at 2024-07-08 21:08:39 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 437 / 6 / 57 / 500: 100%|███████████████████| 500/500 [1:30:56<00:00, 10.91s/it]textattack: Saving checkpoint under "checkpoints\1720444768065.ta.chkpt" at 2024-07-08 21:19:28 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 437 / 6 / 57 / 500: 100%|███████████████████| 500/500 [1:30:56<00:00, 10.91s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 437    |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 1.2%   |
| Attack success rate:          | 98.65% |
| Average perturbed word %:     | 15.41% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 190.24 |
+-------------------------------+--------+


In [65]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 34 / 1 / 15 / 50:  10%|██▍                     | 50/500 [06:07<55:10,  7.36s/it]textattack: Saving checkpoint under "checkpoints\1720445135973.ta.chkpt" at 2024-07-08 21:25:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 2 / 31 / 100:  20%|████▍                 | 100/500 [12:05<48:22,  7.26s/it]textattack: Saving checkpoint under "checkpoints\1720445493861.ta.chkpt" at 2024-07-08 21:31:33 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 67 / 2 / 32 / 101:  20%|████▍                 | 101/500 [12:05<47:47,  7.19s/it]

[Succeeded / Failed / Skipped / Total] 105 / 3 / 42 / 150:  30%|██████▎              | 150/500 [19:16<44:59,  7.71s/it]textattack: Saving checkpoint under "checkpoints\1720445925152.ta.chkpt" at 2024-07-08 21:38:45 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 283 / 5 / 112 / 400:  80%|████████████████    | 400/500 [51:42<12:55,  7.76s/it]textattack: Saving checkpoint under "checkpoints\1720447871044.ta.chkpt" at 2024-07-08 22:11:11 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 315 / 6 / 129 / 450:  90%|██████████████████  | 450/500 [58:25<06:29,  7.79s/it]textattack: Saving checkpoint under "checkpoints\1720448273977.ta.chkpt" at 2024-07-08 22:17:53 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 351 / 8 / 141 / 500: 100%|██████████████████| 500/500 [1:06:11<00:00,  7.94s/it]textattack: Saving checkpoint under "checkpoints\1720448739201.ta.chkpt" at 2024-07-08 22:25:39 after 500 attacks.


[Succeeded / Failed / Skipped / Total] 351 / 8 / 141 / 500: 100%|██████████████████| 500/500 [1:06:11<00:00,  7.94s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 351    |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 1.6%   |
| Attack success rate:          | 97.77% |
| Average perturbed word %:     | 9.54%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 144.81 |
+-------------------------------+--------+


In [66]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 32 / 7 / 11 / 50:  10%|██▏                   | 50/500 [10:54<1:38:12, 13.10s/it]textattack: Saving checkpoint under "checkpoints\1720449394240.ta.chkpt" at 2024-07-08 22:36:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 71 / 9 / 20 / 100:  20%|████                | 100/500 [21:15<1:25:00, 12.75s/it]textattack: Saving checkpoint under "checkpoints\1720450014716.ta.chkpt" at 2024-07-08 22:46:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 101 / 17 / 32 / 150:  30%|█████▍            | 150/500 [34:53<1:21:25, 13.96s/it]textattack: Saving checkpoint under "checkpoints\1720450833089.ta.chkpt" at 2024-07-08 23:00:33 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 233 / 44 / 73 / 350:  70%|████████████▌     | 350/500 [1:21:47<35:03, 14.02s/it]textattack: Saving checkpoint under "checkpoints\1720453646597.ta.chkpt" at 2024-07-08 23:47:26 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 265 / 50 / 85 / 400:  80%|██████████████▍   | 400/500 [1:31:37<22:54, 13.74s/it]textattack: Saving checkpoint under "checkpoints\1720454237094.ta.chkpt" at 2024-07-08 23:57:17 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 293 / 58 / 99 / 450:  90%|████████████████▏ | 450/500 [1:42:44<11:24, 13.70s/it]textattack: Saving checkpoint under "checkpoints\1720454904164.ta.chkpt" at 2024-07-09 00:08:24 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 326 / 65 / 109 / 500: 100%|█████████████████| 500/500 [1:53:33<00:00, 13.63s/it]textattack: Saving checkpoint under "checkpoints\1720455552964.ta.chkpt" at 2024-07-09 00:19:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 326 / 65 / 109 / 500: 100%|█████████████████| 500/500 [1:53:33<00:00, 13.63s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 326    |
| Number of failed attacks:     | 65     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 13.0%  |
| Attack success rate:          | 83.38% |
| Average perturbed word %:     | 4.98%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 206.06 |
+-------------------------------+--------+


In [67]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  10%|██▎                    | 50/500 [08:22<1:15:21, 10.05s/it]textattack: Saving checkpoint under "checkpoints\1720456055472.ta.chkpt" at 2024-07-09 00:27:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 4 / 8 / 100:  20%|████▌                  | 100/500 [14:47<59:08,  8.87s/it]textattack: Saving checkpoint under "checkpoints\1720456440183.ta.chkpt" at 2024-07-09 00:34:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 132 / 8 / 10 / 150:  30%|██████▎              | 150/500 [22:15<51:56,  8.90s/it]textattack: Saving checkpoint under "checkpoints\1720456888528.ta.chkpt" at 2024-07-09 00:41:28 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 305 / 14 / 31 / 350:  70%|██████████████      | 350/500 [51:16<21:58,  8.79s/it]textattack: Saving checkpoint under "checkpoints\1720458629721.ta.chkpt" at 2024-07-09 01:10:29 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 350 / 16 / 34 / 400:  80%|████████████████    | 400/500 [57:02<14:15,  8.56s/it]textattack: Saving checkpoint under "checkpoints\1720458975526.ta.chkpt" at 2024-07-09 01:16:15 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 391 / 18 / 41 / 450:  90%|████████████████▏ | 450/500 [1:03:01<07:00,  8.40s/it]textattack: Saving checkpoint under "checkpoints\1720459334823.ta.chkpt" at 2024-07-09 01:22:14 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 437 / 19 / 44 / 500: 100%|██████████████████| 500/500 [1:09:40<00:00,  8.36s/it]textattack: Saving checkpoint under "checkpoints\1720459733151.ta.chkpt" at 2024-07-09 01:28:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 437 / 19 / 44 / 500: 100%|██████████████████| 500/500 [1:09:40<00:00,  8.36s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 437    |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 3.8%   |
| Attack success rate:          | 95.83% |
| Average perturbed word %:     | 19.5%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 236.38 |
+-------------------------------+--------+


In [68]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 39 / 7 / 4 / 50:  10%|██▎                    | 50/500 [08:28<1:16:19, 10.18s/it]textattack: Saving checkpoint under "checkpoints\1720460242157.ta.chkpt" at 2024-07-09 01:37:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 74 / 13 / 13 / 100:  20%|███▊               | 100/500 [16:52<1:07:28, 10.12s/it]textattack: Saving checkpoint under "checkpoints\1720460745427.ta.chkpt" at 2024-07-09 01:45:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 111 / 19 / 20 / 150:  30%|██████              | 150/500 [25:17<58:59, 10.11s/it]textattack: Saving checkpoint under "checkpoints\1720461250380.ta.chkpt" at 2024-07-09 01:54:10 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 256 / 48 / 46 / 350:  70%|████████████▌     | 350/500 [1:03:24<27:10, 10.87s/it]textattack: Saving checkpoint under "checkpoints\1720463538274.ta.chkpt" at 2024-07-09 02:32:18 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 53 / 50 / 400:  80%|██████████████▍   | 400/500 [1:11:03<17:45, 10.66s/it]textattack: Saving checkpoint under "checkpoints\1720463996632.ta.chkpt" at 2024-07-09 02:39:56 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 334 / 57 / 59 / 450:  90%|████████████████▏ | 450/500 [1:19:57<08:53, 10.66s/it]textattack: Saving checkpoint under "checkpoints\1720464531234.ta.chkpt" at 2024-07-09 02:48:51 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 370 / 67 / 63 / 500: 100%|██████████████████| 500/500 [1:28:34<00:00, 10.63s/it]textattack: Saving checkpoint under "checkpoints\1720465047424.ta.chkpt" at 2024-07-09 02:57:27 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 370 / 67 / 63 / 500: 100%|██████████████████| 500/500 [1:28:34<00:00, 10.63s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 370    |
| Number of failed attacks:     | 67     |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 13.4%  |
| Attack success rate:          | 84.67% |
| Average perturbed word %:     | 22.62% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 329.13 |
+-------------------------------+--------+


In [69]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 39 / 3 / 50:  10%|██▎                    | 50/500 [19:25<2:54:50, 23.31s/it]textattack: Saving checkpoint under "checkpoints\1720466213155.ta.chkpt" at 2024-07-09 03:16:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 74 / 8 / 100:  20%|████                | 100/500 [39:16<2:37:04, 23.56s/it]textattack: Saving checkpoint under "checkpoints\1720467403638.ta.chkpt" at 2024-07-09 03:36:43 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 111 / 11 / 150:  30%|████▊           | 150/500 [1:00:06<2:20:15, 24.05s/it]textattack: Saving checkpoint under "checkpoints\1720468654375.ta.chkpt" at 2024-07-09 03:57:34 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 53 / 262 / 35 / 350:  70%|███████████▏    | 350/500 [2:24:03<1:01:44, 24.70s/it]textattack: Saving checkpoint under "checkpoints\1720473690922.ta.chkpt" at 2024-07-09 05:21:30 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 301 / 40 / 400:  80%|██████████████▍   | 400/500 [2:43:13<40:48, 24.48s/it]textattack: Saving checkpoint under "checkpoints\1720474841145.ta.chkpt" at 2024-07-09 05:40:41 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 337 / 43 / 450:  90%|████████████████▏ | 450/500 [3:02:26<20:16, 24.32s/it]textattack: Saving checkpoint under "checkpoints\1720475993745.ta.chkpt" at 2024-07-09 05:59:53 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 376 / 45 / 500: 100%|██████████████████| 500/500 [3:20:49<00:00, 24.10s/it]textattack: Saving checkpoint under "checkpoints\1720477097037.ta.chkpt" at 2024-07-09 06:18:17 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 79 / 376 / 45 / 500: 100%|██████████████████| 500/500 [3:20:49<00:00, 24.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 79     |
| Number of failed attacks:     | 376    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 75.2%  |
| Attack success rate:          | 17.36% |
| Average perturbed word %:     | 6.22%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 540.99 |
+-------------------------------+--------+


In [70]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  10%|██▎                    | 50/500 [09:11<1:22:41, 11.03s/it]textattack: Saving checkpoint under "checkpoints\1720477648467.ta.chkpt" at 2024-07-09 06:27:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 4 / 9 / 100:  20%|████▏                | 100/500 [16:10<1:04:42,  9.71s/it]textattack: Saving checkpoint under "checkpoints\1720478067848.ta.chkpt" at 2024-07-09 06:34:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 5 / 14 / 150:  30%|██████▎              | 150/500 [23:48<55:33,  9.53s/it]textattack: Saving checkpoint under "checkpoints\1720478525989.ta.chkpt" at 2024-07-09 06:42:05 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 309 / 8 / 33 / 350:  70%|██████████████▋      | 350/500 [53:53<23:05,  9.24s/it]textattack: Saving checkpoint under "checkpoints\1720480330180.ta.chkpt" at 2024-07-09 07:12:10 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 354 / 9 / 37 / 400:  80%|████████████████▊    | 400/500 [59:29<14:52,  8.92s/it]textattack: Saving checkpoint under "checkpoints\1720480666557.ta.chkpt" at 2024-07-09 07:17:46 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 395 / 10 / 45 / 450:  90%|████████████████▏ | 450/500 [1:06:15<07:21,  8.83s/it]textattack: Saving checkpoint under "checkpoints\1720481072516.ta.chkpt" at 2024-07-09 07:24:32 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 443 / 11 / 46 / 500: 100%|██████████████████| 500/500 [1:14:04<00:00,  8.89s/it]textattack: Saving checkpoint under "checkpoints\1720481541541.ta.chkpt" at 2024-07-09 07:32:21 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 443 / 11 / 46 / 500: 100%|██████████████████| 500/500 [1:14:04<00:00,  8.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 443    |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 2.2%   |
| Attack success rate:          | 97.58% |
| Average perturbed word %:     | 18.8%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 225.12 |
+-------------------------------+--------+


In [71]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 35 / 11 / 4 / 50:  10%|██▏                   | 50/500 [10:28<1:34:12, 12.56s/it]textattack: Saving checkpoint under "checkpoints\1720482169700.ta.chkpt" at 2024-07-09 07:42:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 14 / 13 / 100:  20%|███▊               | 100/500 [19:34<1:18:16, 11.74s/it]textattack: Saving checkpoint under "checkpoints\1720482715877.ta.chkpt" at 2024-07-09 07:51:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 25 / 20 / 150:  30%|█████▍            | 150/500 [29:46<1:09:27, 11.91s/it]textattack: Saving checkpoint under "checkpoints\1720483327728.ta.chkpt" at 2024-07-09 08:02:07 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 233 / 65 / 52 / 350:  70%|████████████▌     | 350/500 [1:09:09<29:38, 11.86s/it]textattack: Saving checkpoint under "checkpoints\1720485691256.ta.chkpt" at 2024-07-09 08:41:31 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 266 / 75 / 59 / 400:  80%|██████████████▍   | 400/500 [1:17:47<19:26, 11.67s/it]textattack: Saving checkpoint under "checkpoints\1720486209064.ta.chkpt" at 2024-07-09 08:50:09 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 295 / 84 / 71 / 450:  90%|████████████████▏ | 450/500 [1:27:25<09:42, 11.66s/it]textattack: Saving checkpoint under "checkpoints\1720486787213.ta.chkpt" at 2024-07-09 08:59:47 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 332 / 95 / 73 / 500: 100%|██████████████████| 500/500 [1:37:43<00:00, 11.73s/it]textattack: Saving checkpoint under "checkpoints\1720487404864.ta.chkpt" at 2024-07-09 09:10:04 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 332 / 95 / 73 / 500: 100%|██████████████████| 500/500 [1:37:43<00:00, 11.73s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 332    |
| Number of failed attacks:     | 95     |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 19.0%  |
| Attack success rate:          | 77.75% |
| Average perturbed word %:     | 17.91% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 313.17 |
+-------------------------------+--------+


In [72]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 44 / 1 / 50:  10%|██▎                    | 50/500 [21:47<3:16:03, 26.14s/it]textattack: Saving checkpoint under "checkpoints\1720488711971.ta.chkpt" at 2024-07-09 09:31:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 77 / 4 / 100:  20%|████                | 100/500 [43:22<2:53:29, 26.02s/it]textattack: Saving checkpoint under "checkpoints\1720490007359.ta.chkpt" at 2024-07-09 09:53:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 116 / 7 / 150:  30%|█████            | 150/500 [1:05:20<2:32:28, 26.14s/it]textattack: Saving checkpoint under "checkpoints\1720491325788.ta.chkpt" at 2024-07-09 10:15:25 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 69 / 255 / 26 / 350:  70%|███████████▏    | 350/500 [2:26:09<1:02:38, 25.06s/it]textattack: Saving checkpoint under "checkpoints\1720496174768.ta.chkpt" at 2024-07-09 11:36:14 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 294 / 28 / 400:  80%|██████████████▍   | 400/500 [2:45:30<41:22, 24.83s/it]textattack: Saving checkpoint under "checkpoints\1720497335356.ta.chkpt" at 2024-07-09 11:55:35 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 329 / 35 / 450:  90%|████████████████▏ | 450/500 [3:05:34<20:37, 24.74s/it]textattack: Saving checkpoint under "checkpoints\1720498539671.ta.chkpt" at 2024-07-09 12:15:39 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 92 / 373 / 35 / 500: 100%|██████████████████| 500/500 [3:27:44<00:00, 24.93s/it]textattack: Saving checkpoint under "checkpoints\1720499869835.ta.chkpt" at 2024-07-09 12:37:49 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 92 / 373 / 35 / 500: 100%|██████████████████| 500/500 [3:27:44<00:00, 24.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 92     |
| Number of failed attacks:     | 373    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 74.6%  |
| Attack success rate:          | 19.78% |
| Average perturbed word %:     | 5.49%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 532.03 |
+-------------------------------+--------+


In [73]:
et5 = datetime.datetime.now()
et5 - et4

datetime.timedelta(seconds=60558, microseconds=240028)

### MinSim 0.6, threshold 0.5

In [74]:
suffix = 'sim06.csv'
thre = 1-0.5/np.pi

In [75]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 43 / 2 / 5 / 50:  10%|██▎                    | 50/500 [06:49<1:01:27,  8.19s/it]textattack: Saving checkpoint under "checkpoints\1720500280066.ta.chkpt" at 2024-07-09 12:44:40 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 2 / 10 / 100:  20%|████▍                 | 100/500 [14:31<58:05,  8.71s/it]textattack: Saving checkpoint under "checkpoints\1720500741651.ta.chkpt" at 2024-07-09 12:52:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 130 / 5 / 15 / 150:  30%|██████▎              | 150/500 [23:16<54:17,  9.31s/it]textattack: Saving checkpoint under "checkpoints\1720501266613.ta.chkpt" at 2024-07-09 13:01:06 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 300 / 13 / 37 / 350:  70%|██████████████      | 350/500 [56:44<24:19,  9.73s/it]textattack: Saving checkpoint under "checkpoints\1720503275262.ta.chkpt" at 2024-07-09 13:34:35 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 344 / 15 / 41 / 400:  80%|██████████████▍   | 400/500 [1:03:15<15:48,  9.49s/it]textattack: Saving checkpoint under "checkpoints\1720503666019.ta.chkpt" at 2024-07-09 13:41:06 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 382 / 15 / 53 / 450:  90%|████████████████▏ | 450/500 [1:09:56<07:46,  9.33s/it]textattack: Saving checkpoint under "checkpoints\1720504066878.ta.chkpt" at 2024-07-09 13:47:46 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 425 / 18 / 57 / 500: 100%|██████████████████| 500/500 [1:18:24<00:00,  9.41s/it]textattack: Saving checkpoint under "checkpoints\1720504574536.ta.chkpt" at 2024-07-09 13:56:14 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 425 / 18 / 57 / 500: 100%|██████████████████| 500/500 [1:18:24<00:00,  9.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 425    |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 3.6%   |
| Attack success rate:          | 95.94% |
| Average perturbed word %:     | 16.92% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 154.92 |
+-------------------------------+--------+


In [76]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 4 / 15 / 50:  10%|██▍                     | 50/500 [04:31<40:39,  5.42s/it]textattack: Saving checkpoint under "checkpoints\1720504845740.ta.chkpt" at 2024-07-09 14:00:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 65 / 4 / 31 / 100:  20%|████▍                 | 100/500 [08:27<33:50,  5.08s/it]textattack: Saving checkpoint under "checkpoints\1720505082451.ta.chkpt" at 2024-07-09 14:04:42 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 65 / 4 / 32 / 101:  20%|████▍                 | 101/500 [08:27<33:26,  5.03s/it]

[Succeeded / Failed / Skipped / Total] 103 / 5 / 42 / 150:  30%|██████▎              | 150/500 [13:34<31:40,  5.43s/it]textattack: Saving checkpoint under "checkpoints\1720505389327.ta.chkpt" at 2024-07-09 14:09:49 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 236 / 13 / 101 / 350:  70%|█████████████▎     | 350/500 [35:11<15:04,  6.03s/it]textattack: Saving checkpoint under "checkpoints\1720506685999.ta.chkpt" at 2024-07-09 14:31:25 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 274 / 14 / 112 / 400:  80%|███████████████▏   | 400/500 [39:26<09:51,  5.92s/it]textattack: Saving checkpoint under "checkpoints\1720506941217.ta.chkpt" at 2024-07-09 14:35:41 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 306 / 15 / 129 / 450:  90%|█████████████████  | 450/500 [43:48<04:52,  5.84s/it]textattack: Saving checkpoint under "checkpoints\1720507203464.ta.chkpt" at 2024-07-09 14:40:03 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 341 / 18 / 141 / 500: 100%|███████████████████| 500/500 [49:29<00:00,  5.94s/it]textattack: Saving checkpoint under "checkpoints\1720507544514.ta.chkpt" at 2024-07-09 14:45:44 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 341 / 18 / 141 / 500: 100%|███████████████████| 500/500 [49:29<00:00,  5.94s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 341    |
| Number of failed attacks:     | 18     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 3.6%   |
| Attack success rate:          | 94.99% |
| Average perturbed word %:     | 10.19% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 118.21 |
+-------------------------------+--------+


In [77]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 29 / 10 / 11 / 50:  10%|██                   | 50/500 [08:31<1:16:41, 10.23s/it]textattack: Saving checkpoint under "checkpoints\1720508055953.ta.chkpt" at 2024-07-09 14:54:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 65 / 15 / 20 / 100:  20%|███▊               | 100/500 [16:44<1:06:56, 10.04s/it]textattack: Saving checkpoint under "checkpoints\1720508548668.ta.chkpt" at 2024-07-09 15:02:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 95 / 23 / 32 / 150:  30%|██████▎              | 150/500 [25:29<59:27, 10.19s/it]textattack: Saving checkpoint under "checkpoints\1720509073794.ta.chkpt" at 2024-07-09 15:11:13 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 223 / 54 / 73 / 350:  70%|████████████▌     | 350/500 [1:00:26<25:54, 10.36s/it]textattack: Saving checkpoint under "checkpoints\1720511170684.ta.chkpt" at 2024-07-09 15:46:10 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 254 / 61 / 85 / 400:  80%|██████████████▍   | 400/500 [1:07:47<16:56, 10.17s/it]textattack: Saving checkpoint under "checkpoints\1720511612234.ta.chkpt" at 2024-07-09 15:53:32 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 281 / 70 / 99 / 450:  90%|████████████████▏ | 450/500 [1:16:05<08:27, 10.15s/it]textattack: Saving checkpoint under "checkpoints\1720512109939.ta.chkpt" at 2024-07-09 16:01:49 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 314 / 77 / 109 / 500: 100%|█████████████████| 500/500 [1:24:05<00:00, 10.09s/it]textattack: Saving checkpoint under "checkpoints\1720512590441.ta.chkpt" at 2024-07-09 16:09:50 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 314 / 77 / 109 / 500: 100%|█████████████████| 500/500 [1:24:05<00:00, 10.09s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 314    |
| Number of failed attacks:     | 77     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 15.4%  |
| Attack success rate:          | 80.31% |
| Average perturbed word %:     | 5.32%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 153.76 |
+-------------------------------+--------+


In [78]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 42 / 4 / 4 / 50:  10%|██▎                    | 50/500 [06:59<1:02:55,  8.39s/it]textattack: Saving checkpoint under "checkpoints\1720513010051.ta.chkpt" at 2024-07-09 16:16:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 6 / 8 / 100:  20%|████▌                  | 100/500 [12:16<49:04,  7.36s/it]textattack: Saving checkpoint under "checkpoints\1720513326672.ta.chkpt" at 2024-07-09 16:22:06 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 128 / 12 / 10 / 150:  30%|██████              | 150/500 [17:55<41:50,  7.17s/it]textattack: Saving checkpoint under "checkpoints\1720513666333.ta.chkpt" at 2024-07-09 16:27:46 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 287 / 32 / 31 / 350:  70%|██████████████      | 350/500 [41:55<17:58,  7.19s/it]textattack: Saving checkpoint under "checkpoints\1720515105946.ta.chkpt" at 2024-07-09 16:51:45 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 329 / 37 / 34 / 400:  80%|████████████████    | 400/500 [46:33<11:38,  6.98s/it]textattack: Saving checkpoint under "checkpoints\1720515383861.ta.chkpt" at 2024-07-09 16:56:23 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 371 / 38 / 41 / 450:  90%|██████████████████  | 450/500 [51:48<05:45,  6.91s/it]textattack: Saving checkpoint under "checkpoints\1720515699481.ta.chkpt" at 2024-07-09 17:01:39 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 412 / 44 / 44 / 500: 100%|████████████████████| 500/500 [57:23<00:00,  6.89s/it]textattack: Saving checkpoint under "checkpoints\1720516033956.ta.chkpt" at 2024-07-09 17:07:13 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 412 / 44 / 44 / 500: 100%|████████████████████| 500/500 [57:23<00:00,  6.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 412    |
| Number of failed attacks:     | 44     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 8.8%   |
| Attack success rate:          | 90.35% |
| Average perturbed word %:     | 22.04% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 190.33 |
+-------------------------------+--------+


In [79]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 30 / 16 / 4 / 50:  10%|██▏                   | 50/500 [07:12<1:04:51,  8.65s/it]textattack: Saving checkpoint under "checkpoints\1720516466496.ta.chkpt" at 2024-07-09 17:14:26 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 27 / 13 / 100:  20%|████▏                | 100/500 [14:03<56:13,  8.43s/it]textattack: Saving checkpoint under "checkpoints\1720516877478.ta.chkpt" at 2024-07-09 17:21:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 39 / 20 / 150:  30%|██████▎              | 150/500 [20:49<48:34,  8.33s/it]textattack: Saving checkpoint under "checkpoints\1720517283229.ta.chkpt" at 2024-07-09 17:28:03 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 215 / 89 / 46 / 350:  70%|██████████████      | 350/500 [50:01<21:26,  8.58s/it]textattack: Saving checkpoint under "checkpoints\1720519035837.ta.chkpt" at 2024-07-09 17:57:15 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 252 / 98 / 50 / 400:  80%|████████████████    | 400/500 [56:18<14:04,  8.45s/it]textattack: Saving checkpoint under "checkpoints\1720519412424.ta.chkpt" at 2024-07-09 18:03:32 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 283 / 108 / 59 / 450:  90%|███████████████▎ | 450/500 [1:03:13<07:01,  8.43s/it]textattack: Saving checkpoint under "checkpoints\1720519827627.ta.chkpt" at 2024-07-09 18:10:27 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 316 / 121 / 63 / 500: 100%|█████████████████| 500/500 [1:10:20<00:00,  8.44s/it]textattack: Saving checkpoint under "checkpoints\1720520254586.ta.chkpt" at 2024-07-09 18:17:34 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 316 / 121 / 63 / 500: 100%|█████████████████| 500/500 [1:10:20<00:00,  8.44s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 316    |
| Number of failed attacks:     | 121    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 24.2%  |
| Attack success rate:          | 72.31% |
| Average perturbed word %:     | 22.82% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 248.51 |
+-------------------------------+--------+


In [80]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 43 / 3 / 50:  10%|██▎                    | 50/500 [13:58<2:05:43, 16.76s/it]textattack: Saving checkpoint under "checkpoints\1720521092909.ta.chkpt" at 2024-07-09 18:31:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 79 / 8 / 100:  20%|████                | 100/500 [28:03<1:52:13, 16.83s/it]textattack: Saving checkpoint under "checkpoints\1720521938129.ta.chkpt" at 2024-07-09 18:45:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 116 / 11 / 150:  30%|█████▍            | 150/500 [42:53<1:40:03, 17.15s/it]textattack: Saving checkpoint under "checkpoints\1720522827703.ta.chkpt" at 2024-07-09 19:00:27 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 49 / 266 / 35 / 350:  70%|████████████▌     | 350/500 [1:41:12<43:22, 17.35s/it]textattack: Saving checkpoint under "checkpoints\1720526326827.ta.chkpt" at 2024-07-09 19:58:46 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 306 / 40 / 400:  80%|██████████████▍   | 400/500 [1:54:44<28:41, 17.21s/it]textattack: Saving checkpoint under "checkpoints\1720527138952.ta.chkpt" at 2024-07-09 20:12:18 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 343 / 43 / 450:  90%|████████████████▏ | 450/500 [2:08:28<14:16, 17.13s/it]textattack: Saving checkpoint under "checkpoints\1720527963517.ta.chkpt" at 2024-07-09 20:26:03 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 72 / 383 / 45 / 500: 100%|██████████████████| 500/500 [2:21:19<00:00, 16.96s/it]textattack: Saving checkpoint under "checkpoints\1720528733797.ta.chkpt" at 2024-07-09 20:38:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 72 / 383 / 45 / 500: 100%|██████████████████| 500/500 [2:21:19<00:00, 16.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 72     |
| Number of failed attacks:     | 383    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 76.6%  |
| Attack success rate:          | 15.82% |
| Average perturbed word %:     | 6.21%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 354.39 |
+-------------------------------+--------+


In [81]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 39 / 7 / 4 / 50:  10%|██▎                    | 50/500 [07:39<1:08:52,  9.18s/it]textattack: Saving checkpoint under "checkpoints\1720529193083.ta.chkpt" at 2024-07-09 20:46:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 9 / 9 / 100:  20%|████▌                  | 100/500 [13:58<55:52,  8.38s/it]textattack: Saving checkpoint under "checkpoints\1720529571943.ta.chkpt" at 2024-07-09 20:52:51 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 123 / 13 / 14 / 150:  30%|██████              | 150/500 [20:12<47:09,  8.08s/it]textattack: Saving checkpoint under "checkpoints\1720529946438.ta.chkpt" at 2024-07-09 20:59:06 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 287 / 30 / 33 / 350:  70%|██████████████      | 350/500 [45:54<19:40,  7.87s/it]textattack: Saving checkpoint under "checkpoints\1720531488569.ta.chkpt" at 2024-07-09 21:24:48 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 330 / 33 / 37 / 400:  80%|████████████████    | 400/500 [50:45<12:41,  7.61s/it]textattack: Saving checkpoint under "checkpoints\1720531779655.ta.chkpt" at 2024-07-09 21:29:39 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 369 / 36 / 45 / 450:  90%|██████████████████  | 450/500 [56:50<06:18,  7.58s/it]textattack: Saving checkpoint under "checkpoints\1720532144406.ta.chkpt" at 2024-07-09 21:35:44 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 413 / 41 / 46 / 500: 100%|██████████████████| 500/500 [1:03:45<00:00,  7.65s/it]textattack: Saving checkpoint under "checkpoints\1720532559815.ta.chkpt" at 2024-07-09 21:42:39 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 413 / 41 / 46 / 500: 100%|██████████████████| 500/500 [1:03:45<00:00,  7.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 413    |
| Number of failed attacks:     | 41     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 8.2%   |
| Attack success rate:          | 90.97% |
| Average perturbed word %:     | 21.74% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 188.97 |
+-------------------------------+--------+


In [82]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 27 / 19 / 4 / 50:  10%|██▏                   | 50/500 [07:51<1:10:43,  9.43s/it]textattack: Saving checkpoint under "checkpoints\1720533031381.ta.chkpt" at 2024-07-09 21:50:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 30 / 13 / 100:  20%|███▊               | 100/500 [15:56<1:03:45,  9.56s/it]textattack: Saving checkpoint under "checkpoints\1720533516420.ta.chkpt" at 2024-07-09 21:58:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 84 / 46 / 20 / 150:  30%|██████▎              | 150/500 [25:27<59:25, 10.19s/it]textattack: Saving checkpoint under "checkpoints\1720534087845.ta.chkpt" at 2024-07-09 22:08:07 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 197 / 101 / 52 / 350:  70%|█████████████▎     | 350/500 [57:15<24:32,  9.82s/it]textattack: Saving checkpoint under "checkpoints\1720535995916.ta.chkpt" at 2024-07-09 22:39:55 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 225 / 116 / 59 / 400:  80%|█████████████▌   | 400/500 [1:04:11<16:02,  9.63s/it]textattack: Saving checkpoint under "checkpoints\1720536410991.ta.chkpt" at 2024-07-09 22:46:50 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 251 / 128 / 71 / 450:  90%|███████████████▎ | 450/500 [1:11:14<07:54,  9.50s/it]textattack: Saving checkpoint under "checkpoints\1720536834788.ta.chkpt" at 2024-07-09 22:53:54 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 284 / 143 / 73 / 500: 100%|█████████████████| 500/500 [1:18:51<00:00,  9.46s/it]textattack: Saving checkpoint under "checkpoints\1720537291846.ta.chkpt" at 2024-07-09 23:01:31 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 284 / 143 / 73 / 500: 100%|█████████████████| 500/500 [1:18:51<00:00,  9.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 284    |
| Number of failed attacks:     | 143    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 28.6%  |
| Attack success rate:          | 66.51% |
| Average perturbed word %:     | 18.58% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 238.62 |
+-------------------------------+--------+


In [83]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.6, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 47 / 1 / 50:  10%|██▎                    | 50/500 [15:29<2:19:29, 18.60s/it]textattack: Saving checkpoint under "checkpoints\1720538221987.ta.chkpt" at 2024-07-09 23:17:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 82 / 4 / 100:  20%|████                | 100/500 [31:00<2:04:00, 18.60s/it]textattack: Saving checkpoint under "checkpoints\1720539152140.ta.chkpt" at 2024-07-09 23:32:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 121 / 7 / 150:  30%|█████▋             | 150/500 [46:18<1:48:03, 18.53s/it]textattack: Saving checkpoint under "checkpoints\1720540070798.ta.chkpt" at 2024-07-09 23:47:50 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 55 / 269 / 26 / 350:  70%|████████████▌     | 350/500 [1:44:55<44:58, 17.99s/it]textattack: Saving checkpoint under "checkpoints\1720543587525.ta.chkpt" at 2024-07-10 00:46:27 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 311 / 28 / 400:  80%|██████████████▍   | 400/500 [1:58:54<29:43, 17.84s/it]textattack: Saving checkpoint under "checkpoints\1720544426553.ta.chkpt" at 2024-07-10 01:00:26 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 347 / 35 / 450:  90%|████████████████▏ | 450/500 [2:13:13<14:48, 17.76s/it]textattack: Saving checkpoint under "checkpoints\1720545285277.ta.chkpt" at 2024-07-10 01:14:45 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 392 / 35 / 500: 100%|██████████████████| 500/500 [2:28:54<00:00, 17.87s/it]textattack: Saving checkpoint under "checkpoints\1720546226078.ta.chkpt" at 2024-07-10 01:30:26 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 392 / 35 / 500: 100%|██████████████████| 500/500 [2:28:54<00:00, 17.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 392    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 78.4%  |
| Attack success rate:          | 15.7%  |
| Average perturbed word %:     | 5.79%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 356.57 |
+-------------------------------+--------+


In [84]:
et6 = datetime.datetime.now()
et6 - et5

datetime.timedelta(seconds=46356, microseconds=253465)

### MinSim 0.7, threshold 0.5

In [85]:
suffix = 'sim07.csv'
thre = 1-0.5/np.pi

In [86]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 42 / 3 / 5 / 50:  10%|██▎                    | 50/500 [07:50<1:10:32,  9.41s/it]textattack: Saving checkpoint under "checkpoints\1720546696946.ta.chkpt" at 2024-07-10 01:38:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 83 / 7 / 10 / 100:  20%|████▍                 | 100/500 [14:46<59:06,  8.87s/it]textattack: Saving checkpoint under "checkpoints\1720547113174.ta.chkpt" at 2024-07-10 01:45:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 122 / 13 / 15 / 150:  30%|██████              | 150/500 [21:49<50:55,  8.73s/it]textattack: Saving checkpoint under "checkpoints\1720547536166.ta.chkpt" at 2024-07-10 01:52:16 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 278 / 35 / 37 / 350:  70%|██████████████      | 350/500 [49:45<21:19,  8.53s/it]textattack: Saving checkpoint under "checkpoints\1720549211838.ta.chkpt" at 2024-07-10 02:20:11 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 320 / 39 / 41 / 400:  80%|████████████████    | 400/500 [54:42<13:40,  8.21s/it]textattack: Saving checkpoint under "checkpoints\1720549509509.ta.chkpt" at 2024-07-10 02:25:09 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 355 / 42 / 53 / 450:  90%|████████████████▏ | 450/500 [1:00:34<06:43,  8.08s/it]textattack: Saving checkpoint under "checkpoints\1720549860808.ta.chkpt" at 2024-07-10 02:31:00 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 394 / 49 / 57 / 500: 100%|██████████████████| 500/500 [1:06:56<00:00,  8.03s/it]textattack: Saving checkpoint under "checkpoints\1720550243345.ta.chkpt" at 2024-07-10 02:37:23 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 394 / 49 / 57 / 500: 100%|██████████████████| 500/500 [1:06:56<00:00,  8.03s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 394    |
| Number of failed attacks:     | 49     |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 9.8%   |
| Attack success rate:          | 88.94% |
| Average perturbed word %:     | 18.96% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 119.47 |
+-------------------------------+--------+


In [87]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 30 / 5 / 15 / 50:  10%|██▍                     | 50/500 [03:44<33:37,  4.48s/it]textattack: Saving checkpoint under "checkpoints\1720550467702.ta.chkpt" at 2024-07-10 02:41:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 6 / 31 / 100:  20%|████▍                 | 100/500 [06:55<27:42,  4.16s/it]textattack: Saving checkpoint under "checkpoints\1720550659069.ta.chkpt" at 2024-07-10 02:44:19 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 63 / 6 / 32 / 101:  20%|████▍                 | 101/500 [06:55<27:22,  4.12s/it]

[Succeeded / Failed / Skipped / Total] 96 / 12 / 42 / 150:  30%|██████▎              | 150/500 [11:19<26:26,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1720550923338.ta.chkpt" at 2024-07-10 02:48:43 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 222 / 27 / 101 / 350:  70%|█████████████▎     | 350/500 [28:01<12:00,  4.81s/it]textattack: Saving checkpoint under "checkpoints\1720551925280.ta.chkpt" at 2024-07-10 03:05:25 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 259 / 29 / 112 / 400:  80%|███████████████▏   | 400/500 [31:23<07:50,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720552126612.ta.chkpt" at 2024-07-10 03:08:46 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 289 / 32 / 129 / 450:  90%|█████████████████  | 450/500 [35:07<03:54,  4.68s/it]textattack: Saving checkpoint under "checkpoints\1720552350910.ta.chkpt" at 2024-07-10 03:12:30 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 320 / 39 / 141 / 500: 100%|███████████████████| 500/500 [39:37<00:00,  4.75s/it]textattack: Saving checkpoint under "checkpoints\1720552620549.ta.chkpt" at 2024-07-10 03:17:00 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 320 / 39 / 141 / 500: 100%|███████████████████| 500/500 [39:37<00:00,  4.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 320    |
| Number of failed attacks:     | 39     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 7.8%   |
| Attack success rate:          | 89.14% |
| Average perturbed word %:     | 10.49% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 89.67  |
+-------------------------------+--------+


In [88]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 8 / 11 / 50:  10%|██▍                     | 50/500 [06:10<55:32,  7.41s/it]textattack: Saving checkpoint under "checkpoints\1720552990945.ta.chkpt" at 2024-07-10 03:23:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 17 / 20 / 100:  20%|████▏                | 100/500 [13:39<54:36,  8.19s/it]textattack: Saving checkpoint under "checkpoints\1720553439920.ta.chkpt" at 2024-07-10 03:30:39 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 89 / 29 / 32 / 150:  30%|██████▎              | 150/500 [20:47<48:30,  8.32s/it]textattack: Saving checkpoint under "checkpoints\1720553868186.ta.chkpt" at 2024-07-10 03:37:48 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 212 / 65 / 73 / 350:  70%|██████████████      | 350/500 [48:32<20:48,  8.32s/it]textattack: Saving checkpoint under "checkpoints\1720555533143.ta.chkpt" at 2024-07-10 04:05:33 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 237 / 78 / 85 / 400:  80%|████████████████    | 400/500 [55:25<13:51,  8.31s/it]textattack: Saving checkpoint under "checkpoints\1720555945828.ta.chkpt" at 2024-07-10 04:12:25 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 261 / 90 / 99 / 450:  90%|████████████████▏ | 450/500 [1:02:52<06:59,  8.38s/it]textattack: Saving checkpoint under "checkpoints\1720556393644.ta.chkpt" at 2024-07-10 04:19:53 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 293 / 98 / 109 / 500: 100%|█████████████████| 500/500 [1:09:46<00:00,  8.37s/it]textattack: Saving checkpoint under "checkpoints\1720556807626.ta.chkpt" at 2024-07-10 04:26:47 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 293 / 98 / 109 / 500: 100%|█████████████████| 500/500 [1:09:46<00:00,  8.37s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 293    |
| Number of failed attacks:     | 98     |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 19.6%  |
| Attack success rate:          | 74.94% |
| Average perturbed word %:     | 5.31%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 111.23 |
+-------------------------------+--------+


In [89]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 35 / 11 / 4 / 50:  10%|██▏                   | 50/500 [07:56<1:11:31,  9.54s/it]textattack: Saving checkpoint under "checkpoints\1720557287449.ta.chkpt" at 2024-07-10 04:34:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 14 / 8 / 100:  20%|████▍                 | 100/500 [13:49<55:16,  8.29s/it]textattack: Saving checkpoint under "checkpoints\1720557639628.ta.chkpt" at 2024-07-10 04:40:39 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 116 / 24 / 10 / 150:  30%|██████              | 150/500 [19:41<45:56,  7.88s/it]textattack: Saving checkpoint under "checkpoints\1720557992119.ta.chkpt" at 2024-07-10 04:46:32 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 259 / 60 / 31 / 350:  70%|██████████████      | 350/500 [42:10<18:04,  7.23s/it]textattack: Saving checkpoint under "checkpoints\1720559341543.ta.chkpt" at 2024-07-10 05:09:01 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 69 / 34 / 400:  80%|████████████████    | 400/500 [46:25<11:36,  6.96s/it]textattack: Saving checkpoint under "checkpoints\1720559596520.ta.chkpt" at 2024-07-10 05:13:16 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 332 / 77 / 41 / 450:  90%|██████████████████  | 450/500 [51:21<05:42,  6.85s/it]textattack: Saving checkpoint under "checkpoints\1720559891689.ta.chkpt" at 2024-07-10 05:18:11 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 370 / 86 / 44 / 500: 100%|████████████████████| 500/500 [56:16<00:00,  6.75s/it]textattack: Saving checkpoint under "checkpoints\1720560187248.ta.chkpt" at 2024-07-10 05:23:07 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 370 / 86 / 44 / 500: 100%|████████████████████| 500/500 [56:16<00:00,  6.75s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 370    |
| Number of failed attacks:     | 86     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 17.2%  |
| Attack success rate:          | 81.14% |
| Average perturbed word %:     | 23.89% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 138.97 |
+-------------------------------+--------+


In [90]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 26 / 20 / 4 / 50:  10%|██▍                     | 50/500 [06:06<55:00,  7.34s/it]textattack: Saving checkpoint under "checkpoints\1720560554376.ta.chkpt" at 2024-07-10 05:29:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 35 / 13 / 100:  20%|████▏                | 100/500 [11:56<47:45,  7.16s/it]textattack: Saving checkpoint under "checkpoints\1720560903892.ta.chkpt" at 2024-07-10 05:35:03 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 80 / 50 / 20 / 150:  30%|██████▎              | 150/500 [17:51<41:39,  7.14s/it]textattack: Saving checkpoint under "checkpoints\1720561258759.ta.chkpt" at 2024-07-10 05:40:58 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 179 / 125 / 46 / 350:  70%|█████████████▎     | 350/500 [40:01<17:09,  6.86s/it]textattack: Saving checkpoint under "checkpoints\1720562589575.ta.chkpt" at 2024-07-10 06:03:09 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 205 / 145 / 50 / 400:  80%|███████████████▏   | 400/500 [44:38<11:09,  6.70s/it]textattack: Saving checkpoint under "checkpoints\1720562866610.ta.chkpt" at 2024-07-10 06:07:46 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 223 / 168 / 59 / 450:  90%|█████████████████  | 450/500 [49:53<05:32,  6.65s/it]textattack: Saving checkpoint under "checkpoints\1720563180808.ta.chkpt" at 2024-07-10 06:13:00 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 250 / 187 / 63 / 500: 100%|███████████████████| 500/500 [55:03<00:00,  6.61s/it]textattack: Saving checkpoint under "checkpoints\1720563491466.ta.chkpt" at 2024-07-10 06:18:11 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 250 / 187 / 63 / 500: 100%|███████████████████| 500/500 [55:03<00:00,  6.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 250    |
| Number of failed attacks:     | 187    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 37.4%  |
| Attack success rate:          | 57.21% |
| Average perturbed word %:     | 21.93% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 169.19 |
+-------------------------------+--------+


In [91]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 40 / 3 / 50:  10%|██▎                    | 50/500 [10:12<1:31:53, 12.25s/it]textattack: Saving checkpoint under "checkpoints\1720564104207.ta.chkpt" at 2024-07-10 06:28:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 77 / 8 / 100:  20%|████                | 100/500 [20:29<1:21:57, 12.29s/it]textattack: Saving checkpoint under "checkpoints\1720564720991.ta.chkpt" at 2024-07-10 06:38:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 116 / 11 / 150:  30%|█████▍            | 150/500 [31:05<1:12:33, 12.44s/it]textattack: Saving checkpoint under "checkpoints\1720565357399.ta.chkpt" at 2024-07-10 06:49:17 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 44 / 271 / 35 / 350:  70%|████████████▌     | 350/500 [1:12:23<31:01, 12.41s/it]textattack: Saving checkpoint under "checkpoints\1720567835501.ta.chkpt" at 2024-07-10 07:30:35 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 313 / 40 / 400:  80%|██████████████▍   | 400/500 [1:22:06<20:31, 12.32s/it]textattack: Saving checkpoint under "checkpoints\1720568418362.ta.chkpt" at 2024-07-10 07:40:18 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 56 / 351 / 43 / 450:  90%|████████████████▏ | 450/500 [1:32:17<10:15, 12.31s/it]textattack: Saving checkpoint under "checkpoints\1720569029026.ta.chkpt" at 2024-07-10 07:50:29 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 391 / 45 / 500: 100%|██████████████████| 500/500 [1:41:26<00:00, 12.17s/it]textattack: Saving checkpoint under "checkpoints\1720569578495.ta.chkpt" at 2024-07-10 07:59:38 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 64 / 391 / 45 / 500: 100%|██████████████████| 500/500 [1:41:26<00:00, 12.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 64     |
| Number of failed attacks:     | 391    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 78.2%  |
| Attack success rate:          | 14.07% |
| Average perturbed word %:     | 6.26%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 212.53 |
+-------------------------------+--------+


In [92]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 15 / 4 / 50:  10%|██▍                     | 50/500 [06:12<55:50,  7.45s/it]textattack: Saving checkpoint under "checkpoints\1720569950936.ta.chkpt" at 2024-07-10 08:05:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 21 / 9 / 100:  20%|████▍                 | 100/500 [11:29<45:56,  6.89s/it]textattack: Saving checkpoint under "checkpoints\1720570267802.ta.chkpt" at 2024-07-10 08:11:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 31 / 14 / 150:  30%|██████              | 150/500 [16:30<38:30,  6.60s/it]textattack: Saving checkpoint under "checkpoints\1720570568939.ta.chkpt" at 2024-07-10 08:16:08 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 246 / 71 / 33 / 350:  70%|██████████████      | 350/500 [37:21<16:00,  6.40s/it]textattack: Saving checkpoint under "checkpoints\1720571819865.ta.chkpt" at 2024-07-10 08:36:59 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 287 / 76 / 37 / 400:  80%|████████████████    | 400/500 [41:34<10:23,  6.24s/it]textattack: Saving checkpoint under "checkpoints\1720572072733.ta.chkpt" at 2024-07-10 08:41:12 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 326 / 79 / 45 / 450:  90%|██████████████████  | 450/500 [46:40<05:11,  6.22s/it]textattack: Saving checkpoint under "checkpoints\1720572379504.ta.chkpt" at 2024-07-10 08:46:19 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 365 / 89 / 46 / 500: 100%|████████████████████| 500/500 [51:56<00:00,  6.23s/it]textattack: Saving checkpoint under "checkpoints\1720572695541.ta.chkpt" at 2024-07-10 08:51:35 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 365 / 89 / 46 / 500: 100%|████████████████████| 500/500 [51:56<00:00,  6.23s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 365    |
| Number of failed attacks:     | 89     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 17.8%  |
| Attack success rate:          | 80.4%  |
| Average perturbed word %:     | 22.87% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 139.68 |
+-------------------------------+--------+


In [93]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 24 / 4 / 50:  10%|██▍                     | 50/500 [06:09<55:26,  7.39s/it]textattack: Saving checkpoint under "checkpoints\1720573065340.ta.chkpt" at 2024-07-10 08:57:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 44 / 13 / 100:  20%|████▏                | 100/500 [12:12<48:49,  7.32s/it]textattack: Saving checkpoint under "checkpoints\1720573428079.ta.chkpt" at 2024-07-10 09:03:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 63 / 20 / 150:  30%|██████▎              | 150/500 [18:12<42:28,  7.28s/it]textattack: Saving checkpoint under "checkpoints\1720573787900.ta.chkpt" at 2024-07-10 09:09:47 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 159 / 139 / 52 / 350:  70%|█████████████▎     | 350/500 [40:34<17:23,  6.95s/it]textattack: Saving checkpoint under "checkpoints\1720575129915.ta.chkpt" at 2024-07-10 09:32:09 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 184 / 157 / 59 / 400:  80%|███████████████▏   | 400/500 [45:43<11:25,  6.86s/it]textattack: Saving checkpoint under "checkpoints\1720575438815.ta.chkpt" at 2024-07-10 09:37:18 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 207 / 172 / 71 / 450:  90%|█████████████████  | 450/500 [51:02<05:40,  6.81s/it]textattack: Saving checkpoint under "checkpoints\1720575758400.ta.chkpt" at 2024-07-10 09:42:38 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 232 / 195 / 73 / 500: 100%|███████████████████| 500/500 [57:21<00:00,  6.88s/it]textattack: Saving checkpoint under "checkpoints\1720576137526.ta.chkpt" at 2024-07-10 09:48:57 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 232 / 195 / 73 / 500: 100%|███████████████████| 500/500 [57:21<00:00,  6.88s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 232    |
| Number of failed attacks:     | 195    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 39.0%  |
| Attack success rate:          | 54.33% |
| Average perturbed word %:     | 18.45% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 165.69 |
+-------------------------------+--------+


In [94]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.7, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 47 / 1 / 50:  10%|██▎                    | 50/500 [10:45<1:36:50, 12.91s/it]textattack: Saving checkpoint under "checkpoints\1720576783301.ta.chkpt" at 2024-07-10 09:59:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 87 / 4 / 100:  20%|████▏                | 100/500 [21:38<1:26:35, 12.99s/it]textattack: Saving checkpoint under "checkpoints\1720577436463.ta.chkpt" at 2024-07-10 10:10:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 128 / 7 / 150:  30%|█████▋             | 150/500 [32:07<1:14:56, 12.85s/it]textattack: Saving checkpoint under "checkpoints\1720578064739.ta.chkpt" at 2024-07-10 10:21:04 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 42 / 282 / 26 / 350:  70%|████████████▌     | 350/500 [1:11:50<30:47, 12.31s/it]textattack: Saving checkpoint under "checkpoints\1720580447660.ta.chkpt" at 2024-07-10 11:00:47 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 325 / 28 / 400:  80%|██████████████▍   | 400/500 [1:21:42<20:25, 12.26s/it]textattack: Saving checkpoint under "checkpoints\1720581040277.ta.chkpt" at 2024-07-10 11:10:40 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 361 / 35 / 450:  90%|████████████████▏ | 450/500 [1:31:37<10:10, 12.22s/it]textattack: Saving checkpoint under "checkpoints\1720581634784.ta.chkpt" at 2024-07-10 11:20:34 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 408 / 35 / 500: 100%|██████████████████| 500/500 [1:42:11<00:00, 12.26s/it]textattack: Saving checkpoint under "checkpoints\1720582268708.ta.chkpt" at 2024-07-10 11:31:08 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 57 / 408 / 35 / 500: 100%|██████████████████| 500/500 [1:42:11<00:00, 12.26s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 57     |
| Number of failed attacks:     | 408    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 81.6%  |
| Attack success rate:          | 12.26% |
| Average perturbed word %:     | 5.1%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 213.81 |
+-------------------------------+--------+


In [95]:
et7 = datetime.datetime.now()
et7 - et6

datetime.timedelta(seconds=36042, microseconds=621717)

### MinSim 0.8, threshold 0.5

In [96]:
suffix = 'sim08.csv'
thre = 1-0.5/np.pi

In [97]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 14 / 5 / 50:  10%|██▍                     | 50/500 [05:52<52:52,  7.05s/it]textattack: Saving checkpoint under "checkpoints\1720582621810.ta.chkpt" at 2024-07-10 11:37:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 26 / 10 / 100:  20%|████▏                | 100/500 [11:07<44:31,  6.68s/it]textattack: Saving checkpoint under "checkpoints\1720582937156.ta.chkpt" at 2024-07-10 11:42:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 42 / 15 / 150:  30%|██████▎              | 150/500 [16:28<38:27,  6.59s/it]textattack: Saving checkpoint under "checkpoints\1720583258116.ta.chkpt" at 2024-07-10 11:47:38 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 200 / 113 / 37 / 350:  70%|█████████████▎     | 350/500 [38:50<16:38,  6.66s/it]textattack: Saving checkpoint under "checkpoints\1720584599803.ta.chkpt" at 2024-07-10 12:09:59 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 234 / 125 / 41 / 400:  80%|███████████████▏   | 400/500 [43:22<10:50,  6.51s/it]textattack: Saving checkpoint under "checkpoints\1720584871915.ta.chkpt" at 2024-07-10 12:14:31 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 258 / 139 / 53 / 450:  90%|█████████████████  | 450/500 [48:05<05:20,  6.41s/it]textattack: Saving checkpoint under "checkpoints\1720585155055.ta.chkpt" at 2024-07-10 12:19:15 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 286 / 157 / 57 / 500: 100%|███████████████████| 500/500 [53:09<00:00,  6.38s/it]textattack: Saving checkpoint under "checkpoints\1720585458813.ta.chkpt" at 2024-07-10 12:24:18 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 286 / 157 / 57 / 500: 100%|███████████████████| 500/500 [53:09<00:00,  6.38s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 286    |
| Number of failed attacks:     | 157    |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 31.4%  |
| Attack success rate:          | 64.56% |
| Average perturbed word %:     | 18.26% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 87.05  |
+-------------------------------+--------+


In [98]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 30 / 5 / 15 / 50:  10%|██▍                     | 50/500 [02:51<25:45,  3.43s/it]textattack: Saving checkpoint under "checkpoints\1720585630780.ta.chkpt" at 2024-07-10 12:27:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 12 / 31 / 100:  20%|████▏                | 100/500 [06:04<24:18,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1720585823596.ta.chkpt" at 2024-07-10 12:30:23 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 57 / 12 / 32 / 101:  20%|████▏                | 101/500 [06:04<24:00,  3.61s/it]

[Succeeded / Failed / Skipped / Total] 86 / 22 / 42 / 150:  30%|██████▎              | 150/500 [09:38<22:30,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1720586038033.ta.chkpt" at 2024-07-10 12:33:58 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 194 / 55 / 101 / 350:  70%|█████████████▎     | 350/500 [23:30<10:04,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1720586869533.ta.chkpt" at 2024-07-10 12:47:49 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 226 / 62 / 112 / 400:  80%|███████████████▏   | 400/500 [26:26<06:36,  3.97s/it]textattack: Saving checkpoint under "checkpoints\1720587045308.ta.chkpt" at 2024-07-10 12:50:45 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 254 / 67 / 129 / 450:  90%|█████████████████  | 450/500 [29:21<03:15,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1720587220656.ta.chkpt" at 2024-07-10 12:53:40 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 285 / 74 / 141 / 500: 100%|███████████████████| 500/500 [32:43<00:00,  3.93s/it]textattack: Saving checkpoint under "checkpoints\1720587422856.ta.chkpt" at 2024-07-10 12:57:02 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 285 / 74 / 141 / 500: 100%|███████████████████| 500/500 [32:43<00:00,  3.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 285    |
| Number of failed attacks:     | 74     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 14.8%  |
| Attack success rate:          | 79.39% |
| Average perturbed word %:     | 11.77% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 68.47  |
+-------------------------------+--------+


In [99]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 17 / 11 / 50:  10%|██▎                    | 50/500 [05:22<48:19,  6.44s/it]textattack: Saving checkpoint under "checkpoints\1720587745216.ta.chkpt" at 2024-07-10 13:02:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 30 / 20 / 100:  20%|████▏                | 100/500 [10:47<43:11,  6.48s/it]textattack: Saving checkpoint under "checkpoints\1720588070748.ta.chkpt" at 2024-07-10 13:07:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 74 / 44 / 32 / 150:  30%|██████▎              | 150/500 [15:58<37:15,  6.39s/it]textattack: Saving checkpoint under "checkpoints\1720588381277.ta.chkpt" at 2024-07-10 13:13:01 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 177 / 100 / 73 / 350:  70%|█████████████▎     | 350/500 [39:03<16:44,  6.70s/it]textattack: Saving checkpoint under "checkpoints\1720589766325.ta.chkpt" at 2024-07-10 13:36:06 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 201 / 114 / 85 / 400:  80%|███████████████▏   | 400/500 [43:45<10:56,  6.56s/it]textattack: Saving checkpoint under "checkpoints\1720590048745.ta.chkpt" at 2024-07-10 13:40:48 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 223 / 128 / 99 / 450:  90%|█████████████████  | 450/500 [49:14<05:28,  6.57s/it]textattack: Saving checkpoint under "checkpoints\1720590377528.ta.chkpt" at 2024-07-10 13:46:17 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 247 / 144 / 109 / 500: 100%|██████████████████| 500/500 [54:28<00:00,  6.54s/it]textattack: Saving checkpoint under "checkpoints\1720590691099.ta.chkpt" at 2024-07-10 13:51:31 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 247 / 144 / 109 / 500: 100%|██████████████████| 500/500 [54:28<00:00,  6.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 247    |
| Number of failed attacks:     | 144    |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 28.8%  |
| Attack success rate:          | 63.17% |
| Average perturbed word %:     | 5.2%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 77.07  |
+-------------------------------+--------+


In [100]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 19 / 27 / 4 / 50:  10%|██▍                     | 50/500 [05:01<45:10,  6.02s/it]textattack: Saving checkpoint under "checkpoints\1720590992401.ta.chkpt" at 2024-07-10 13:56:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 47 / 8 / 100:  20%|████▍                 | 100/500 [09:11<36:46,  5.52s/it]textattack: Saving checkpoint under "checkpoints\1720591242949.ta.chkpt" at 2024-07-10 14:00:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 70 / 10 / 150:  30%|██████▎              | 150/500 [13:08<30:39,  5.26s/it]textattack: Saving checkpoint under "checkpoints\1720591479768.ta.chkpt" at 2024-07-10 14:04:39 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 154 / 165 / 31 / 350:  70%|█████████████▎     | 350/500 [28:55<12:23,  4.96s/it]textattack: Saving checkpoint under "checkpoints\1720592426762.ta.chkpt" at 2024-07-10 14:20:26 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 179 / 187 / 34 / 400:  80%|███████████████▏   | 400/500 [32:22<08:05,  4.86s/it]textattack: Saving checkpoint under "checkpoints\1720592633714.ta.chkpt" at 2024-07-10 14:23:53 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 205 / 204 / 41 / 450:  90%|█████████████████  | 450/500 [35:42<03:58,  4.76s/it]textattack: Saving checkpoint under "checkpoints\1720592833665.ta.chkpt" at 2024-07-10 14:27:13 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 224 / 232 / 44 / 500: 100%|███████████████████| 500/500 [39:13<00:00,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720593045125.ta.chkpt" at 2024-07-10 14:30:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 224 / 232 / 44 / 500: 100%|███████████████████| 500/500 [39:13<00:00,  4.71s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 224    |
| Number of failed attacks:     | 232    |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 46.4%  |
| Attack success rate:          | 49.12% |
| Average perturbed word %:     | 21.58% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 97.63  |
+-------------------------------+--------+


In [101]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 13 / 33 / 4 / 50:  10%|██▍                     | 50/500 [03:59<35:56,  4.79s/it]textattack: Saving checkpoint under "checkpoints\1720593285019.ta.chkpt" at 2024-07-10 14:34:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 59 / 13 / 100:  20%|████▏                | 100/500 [07:51<31:24,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720593516598.ta.chkpt" at 2024-07-10 14:38:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 93 / 20 / 150:  30%|██████▎              | 150/500 [11:54<27:47,  4.76s/it]textattack: Saving checkpoint under "checkpoints\1720593759924.ta.chkpt" at 2024-07-10 14:42:39 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 99 / 205 / 46 / 350:  70%|██████████████      | 350/500 [27:58<11:59,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1720594723862.ta.chkpt" at 2024-07-10 14:58:43 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 114 / 236 / 50 / 400:  80%|███████████████▏   | 400/500 [31:23<07:50,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1720594929227.ta.chkpt" at 2024-07-10 15:02:09 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 128 / 263 / 59 / 450:  90%|█████████████████  | 450/500 [35:09<03:54,  4.69s/it]textattack: Saving checkpoint under "checkpoints\1720595155441.ta.chkpt" at 2024-07-10 15:05:55 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 141 / 296 / 63 / 500: 100%|███████████████████| 500/500 [38:48<00:00,  4.66s/it]textattack: Saving checkpoint under "checkpoints\1720595374128.ta.chkpt" at 2024-07-10 15:09:34 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 141 / 296 / 63 / 500: 100%|███████████████████| 500/500 [38:48<00:00,  4.66s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 141    |
| Number of failed attacks:     | 296    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 59.2%  |
| Attack success rate:          | 32.27% |
| Average perturbed word %:     | 17.37% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 104.78 |
+-------------------------------+--------+


In [102]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 42 / 3 / 50:  10%|██▎                    | 50/500 [07:08<1:04:15,  8.57s/it]textattack: Saving checkpoint under "checkpoints\1720595802611.ta.chkpt" at 2024-07-10 15:16:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 81 / 8 / 100:  20%|████▍                 | 100/500 [14:38<58:34,  8.79s/it]textattack: Saving checkpoint under "checkpoints\1720596252900.ta.chkpt" at 2024-07-10 15:24:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 124 / 11 / 150:  30%|██████              | 150/500 [22:14<51:53,  8.90s/it]textattack: Saving checkpoint under "checkpoints\1720596708796.ta.chkpt" at 2024-07-10 15:31:48 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 32 / 283 / 35 / 350:  70%|██████████████      | 350/500 [51:16<21:58,  8.79s/it]textattack: Saving checkpoint under "checkpoints\1720598451077.ta.chkpt" at 2024-07-10 16:00:51 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 325 / 40 / 400:  80%|████████████████    | 400/500 [57:58<14:29,  8.70s/it]textattack: Saving checkpoint under "checkpoints\1720598852864.ta.chkpt" at 2024-07-10 16:07:32 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 363 / 43 / 450:  90%|████████████████▏ | 450/500 [1:05:11<07:14,  8.69s/it]textattack: Saving checkpoint under "checkpoints\1720599285270.ta.chkpt" at 2024-07-10 16:14:45 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 405 / 45 / 500: 100%|██████████████████| 500/500 [1:11:51<00:00,  8.62s/it]textattack: Saving checkpoint under "checkpoints\1720599685315.ta.chkpt" at 2024-07-10 16:21:25 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 50 / 405 / 45 / 500: 100%|██████████████████| 500/500 [1:11:51<00:00,  8.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 50     |
| Number of failed attacks:     | 405    |
| Number of skipped attacks:    | 45     |
| Original accuracy:            | 91.0%  |
| Accuracy under attack:        | 81.0%  |
| Attack success rate:          | 10.99% |
| Average perturbed word %:     | 5.53%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 114.69 |
+-------------------------------+--------+


In [103]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 16 / 30 / 4 / 50:  10%|██▍                     | 50/500 [04:27<40:05,  5.34s/it]textattack: Saving checkpoint under "checkpoints\1720599952657.ta.chkpt" at 2024-07-10 16:25:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 57 / 9 / 100:  20%|████▍                 | 100/500 [09:01<36:07,  5.42s/it]textattack: Saving checkpoint under "checkpoints\1720600227396.ta.chkpt" at 2024-07-10 16:30:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 76 / 14 / 150:  30%|██████▎              | 150/500 [12:54<30:06,  5.16s/it]textattack: Saving checkpoint under "checkpoints\1720600459742.ta.chkpt" at 2024-07-10 16:34:19 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 146 / 171 / 33 / 350:  70%|█████████████▎     | 350/500 [29:32<12:39,  5.07s/it]textattack: Saving checkpoint under "checkpoints\1720601458177.ta.chkpt" at 2024-07-10 16:50:58 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 170 / 193 / 37 / 400:  80%|███████████████▏   | 400/500 [33:01<08:15,  4.95s/it]textattack: Saving checkpoint under "checkpoints\1720601667010.ta.chkpt" at 2024-07-10 16:54:27 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 196 / 209 / 45 / 450:  90%|█████████████████  | 450/500 [36:58<04:06,  4.93s/it]textattack: Saving checkpoint under "checkpoints\1720601903850.ta.chkpt" at 2024-07-10 16:58:23 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 211 / 243 / 46 / 500: 100%|███████████████████| 500/500 [41:20<00:00,  4.96s/it]textattack: Saving checkpoint under "checkpoints\1720602165470.ta.chkpt" at 2024-07-10 17:02:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 211 / 243 / 46 / 500: 100%|███████████████████| 500/500 [41:20<00:00,  4.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 211    |
| Number of failed attacks:     | 243    |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 48.6%  |
| Attack success rate:          | 46.48% |
| Average perturbed word %:     | 21.11% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 98.8   |
+-------------------------------+--------+


In [104]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 14 / 32 / 4 / 50:  10%|██▍                     | 50/500 [04:26<39:55,  5.32s/it]textattack: Saving checkpoint under "checkpoints\1720602431745.ta.chkpt" at 2024-07-10 17:07:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 60 / 13 / 100:  20%|████▏                | 100/500 [08:51<35:27,  5.32s/it]textattack: Saving checkpoint under "checkpoints\1720602697444.ta.chkpt" at 2024-07-10 17:11:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 42 / 88 / 20 / 150:  30%|██████▎              | 150/500 [13:08<30:39,  5.25s/it]textattack: Saving checkpoint under "checkpoints\1720602953809.ta.chkpt" at 2024-07-10 17:15:53 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 105 / 193 / 52 / 350:  70%|█████████████▎     | 350/500 [28:58<12:24,  4.97s/it]textattack: Saving checkpoint under "checkpoints\1720603903738.ta.chkpt" at 2024-07-10 17:31:43 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 118 / 223 / 59 / 400:  80%|███████████████▏   | 400/500 [32:27<08:06,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1720604112680.ta.chkpt" at 2024-07-10 17:35:12 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 135 / 244 / 71 / 450:  90%|█████████████████  | 450/500 [36:01<04:00,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1720604327405.ta.chkpt" at 2024-07-10 17:38:47 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 148 / 279 / 73 / 500: 100%|███████████████████| 500/500 [40:27<00:00,  4.85s/it]textattack: Saving checkpoint under "checkpoints\1720604592768.ta.chkpt" at 2024-07-10 17:43:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 148 / 279 / 73 / 500: 100%|███████████████████| 500/500 [40:27<00:00,  4.85s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 148    |
| Number of failed attacks:     | 279    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 55.8%  |
| Attack success rate:          | 34.66% |
| Average perturbed word %:     | 14.69% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 101.25 |
+-------------------------------+--------+


In [105]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.8, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 47 / 1 / 50:  10%|██▎                    | 50/500 [07:10<1:04:32,  8.61s/it]textattack: Saving checkpoint under "checkpoints\1720605023120.ta.chkpt" at 2024-07-10 17:50:23 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 88 / 4 / 100:  20%|████▌                  | 100/500 [14:30<58:01,  8.70s/it]textattack: Saving checkpoint under "checkpoints\1720605463337.ta.chkpt" at 2024-07-10 17:57:43 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 132 / 7 / 150:  30%|██████▎              | 150/500 [21:34<50:19,  8.63s/it]textattack: Saving checkpoint under "checkpoints\1720605886897.ta.chkpt" at 2024-07-10 18:04:46 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 28 / 296 / 26 / 350:  70%|██████████████      | 350/500 [49:02<21:01,  8.41s/it]textattack: Saving checkpoint under "checkpoints\1720607535346.ta.chkpt" at 2024-07-10 18:32:15 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 338 / 28 / 400:  80%|████████████████    | 400/500 [55:38<13:54,  8.35s/it]textattack: Saving checkpoint under "checkpoints\1720607931153.ta.chkpt" at 2024-07-10 18:38:51 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 377 / 35 / 450:  90%|████████████████▏ | 450/500 [1:02:30<06:56,  8.33s/it]textattack: Saving checkpoint under "checkpoints\1720608343395.ta.chkpt" at 2024-07-10 18:45:43 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 426 / 35 / 500: 100%|██████████████████| 500/500 [1:09:28<00:00,  8.34s/it]textattack: Saving checkpoint under "checkpoints\1720608761489.ta.chkpt" at 2024-07-10 18:52:41 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 39 / 426 / 35 / 500: 100%|██████████████████| 500/500 [1:09:28<00:00,  8.34s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 39     |
| Number of failed attacks:     | 426    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 93.0%  |
| Accuracy under attack:        | 85.2%  |
| Attack success rate:          | 8.39%  |
| Average perturbed word %:     | 5.31%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 115.35 |
+-------------------------------+--------+


In [106]:
et8 = datetime.datetime.now()
et8 - et7

datetime.timedelta(seconds=26492, microseconds=837670)

### MinSim 0.9, threshold 0.5

In [107]:
suffix = 'sim09.csv'
thre = 1-0.5/np.pi

In [108]:
runAttack(bert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/bertagattacklr'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 9 / 36 / 5 / 50:  10%|██▌                      | 50/500 [03:37<32:34,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1720608979103.ta.chkpt" at 2024-07-10 18:56:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 63 / 10 / 100:  20%|████▏                | 100/500 [07:31<30:04,  4.51s/it]textattack: Saving checkpoint under "checkpoints\1720609213051.ta.chkpt" at 2024-07-10 19:00:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 92 / 15 / 150:  30%|██████▎              | 150/500 [11:20<26:28,  4.54s/it]textattack: Saving checkpoint under "checkpoints\1720609442654.ta.chkpt" at 2024-07-10 19:04:02 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 82 / 231 / 37 / 350:  70%|██████████████      | 350/500 [27:40<11:51,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1720610422356.ta.chkpt" at 2024-07-10 19:20:22 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 268 / 41 / 400:  80%|████████████████    | 400/500 [31:08<07:47,  4.67s/it]textattack: Saving checkpoint under "checkpoints\1720610630361.ta.chkpt" at 2024-07-10 19:23:50 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 102 / 295 / 53 / 450:  90%|█████████████████  | 450/500 [34:27<03:49,  4.60s/it]textattack: Saving checkpoint under "checkpoints\1720610829769.ta.chkpt" at 2024-07-10 19:27:09 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 112 / 331 / 57 / 500: 100%|███████████████████| 500/500 [37:54<00:00,  4.55s/it]textattack: Saving checkpoint under "checkpoints\1720611036680.ta.chkpt" at 2024-07-10 19:30:36 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 112 / 331 / 57 / 500: 100%|███████████████████| 500/500 [37:54<00:00,  4.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 112    |
| Number of failed attacks:     | 331    |
| Number of skipped attacks:    | 57     |
| Original accuracy:            | 88.6%  |
| Accuracy under attack:        | 66.2%  |
| Attack success rate:          | 25.28% |
| Average perturbed word %:     | 13.02% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 57.44  |
+-------------------------------+--------+


In [109]:
runAttack(bert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/bertagattackrf'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 18 / 17 / 15 / 50:  10%|██▎                    | 50/500 [02:27<22:05,  2.95s/it]textattack: Saving checkpoint under "checkpoints\1720611184580.ta.chkpt" at 2024-07-10 19:33:04 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 36 / 31 / 100:  20%|████▏                | 100/500 [05:04<20:19,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1720611342287.ta.chkpt" at 2024-07-10 19:35:42 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 33 / 36 / 32 / 101:  20%|████▏                | 101/500 [05:05<20:05,  3.02s/it]

[Succeeded / Failed / Skipped / Total] 50 / 58 / 42 / 150:  30%|██████▎              | 150/500 [08:18<19:23,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1720611535734.ta.chkpt" at 2024-07-10 19:38:55 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 125 / 124 / 101 / 350:  70%|████████████▌     | 350/500 [19:31<08:22,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1720612208950.ta.chkpt" at 2024-07-10 19:50:08 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 143 / 145 / 112 / 400:  80%|██████████████▍   | 400/500 [22:16<05:34,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1720612374050.ta.chkpt" at 2024-07-10 19:52:54 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 158 / 163 / 129 / 450:  90%|████████████████▏ | 450/500 [25:05<02:47,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1720612543235.ta.chkpt" at 2024-07-10 19:55:43 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 170 / 189 / 141 / 500: 100%|██████████████████| 500/500 [27:51<00:00,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1720612708311.ta.chkpt" at 2024-07-10 19:58:28 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 170 / 189 / 141 / 500: 100%|██████████████████| 500/500 [27:51<00:00,  3.34s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 170    |
| Number of failed attacks:     | 189    |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 37.8%  |
| Attack success rate:          | 47.35% |
| Average perturbed word %:     | 9.76%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 51.58  |
+-------------------------------+--------+


In [110]:
runAttack(bert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/bertagattackknn'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 17 / 22 / 11 / 50:  10%|██▎                    | 50/500 [04:01<36:13,  4.83s/it]textattack: Saving checkpoint under "checkpoints\1720612949956.ta.chkpt" at 2024-07-10 20:02:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 40 / 20 / 100:  20%|████▏                | 100/500 [08:20<33:21,  5.00s/it]textattack: Saving checkpoint under "checkpoints\1720613208915.ta.chkpt" at 2024-07-10 20:06:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 61 / 32 / 150:  30%|██████▎              | 150/500 [12:18<28:43,  4.92s/it]textattack: Saving checkpoint under "checkpoints\1720613446918.ta.chkpt" at 2024-07-10 20:10:46 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 126 / 151 / 73 / 350:  70%|█████████████▎     | 350/500 [30:34<13:06,  5.24s/it]textattack: Saving checkpoint under "checkpoints\1720614542746.ta.chkpt" at 2024-07-10 20:29:02 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 144 / 171 / 85 / 400:  80%|███████████████▏   | 400/500 [34:19<08:34,  5.15s/it]textattack: Saving checkpoint under "checkpoints\1720614767745.ta.chkpt" at 2024-07-10 20:32:47 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 156 / 195 / 99 / 450:  90%|█████████████████  | 450/500 [38:54<04:19,  5.19s/it]textattack: Saving checkpoint under "checkpoints\1720615042559.ta.chkpt" at 2024-07-10 20:37:22 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 169 / 222 / 109 / 500: 100%|██████████████████| 500/500 [43:18<00:00,  5.20s/it]textattack: Saving checkpoint under "checkpoints\1720615307202.ta.chkpt" at 2024-07-10 20:41:47 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 169 / 222 / 109 / 500: 100%|██████████████████| 500/500 [43:18<00:00,  5.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 169    |
| Number of failed attacks:     | 222    |
| Number of skipped attacks:    | 109    |
| Original accuracy:            | 78.2%  |
| Accuracy under attack:        | 44.4%  |
| Attack success rate:          | 43.22% |
| Average perturbed word %:     | 4.7%   |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 52.21  |
+-------------------------------+--------+


In [111]:
runAttack(disbert_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/disbertagattacklr'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  10%|██▌                      | 50/500 [04:10<37:30,  5.00s/it]textattack: Saving checkpoint under "checkpoints\1720615557321.ta.chkpt" at 2024-07-10 20:45:57 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 79 / 8 / 100:  20%|████▍                 | 100/500 [07:17<29:08,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1720615744411.ta.chkpt" at 2024-07-10 20:49:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 115 / 10 / 150:  30%|██████              | 150/500 [10:20<24:08,  4.14s/it]textattack: Saving checkpoint under "checkpoints\1720615928168.ta.chkpt" at 2024-07-10 20:52:08 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 55 / 264 / 31 / 350:  70%|██████████████      | 350/500 [22:09<09:29,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1720616637035.ta.chkpt" at 2024-07-10 21:03:57 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 308 / 34 / 400:  80%|████████████████    | 400/500 [24:49<06:12,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1720616796475.ta.chkpt" at 2024-07-10 21:06:36 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 340 / 41 / 450:  90%|██████████████████  | 450/500 [27:23<03:02,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1720616950486.ta.chkpt" at 2024-07-10 21:09:10 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 383 / 44 / 500: 100%|████████████████████| 500/500 [29:49<00:00,  3.58s/it]textattack: Saving checkpoint under "checkpoints\1720617096853.ta.chkpt" at 2024-07-10 21:11:36 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 383 / 44 / 500: 100%|████████████████████| 500/500 [29:49<00:00,  3.58s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 383    |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 76.6%  |
| Attack success rate:          | 16.01% |
| Average perturbed word %:     | 12.51% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 58.45  |
+-------------------------------+--------+


In [112]:
runAttack(disbert_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/disbertagattackrf'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  10%|██▌                      | 50/500 [02:37<23:37,  3.15s/it]textattack: Saving checkpoint under "checkpoints\1720617254434.ta.chkpt" at 2024-07-10 21:14:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 80 / 13 / 100:  20%|████▍                 | 100/500 [05:18<21:13,  3.18s/it]textattack: Saving checkpoint under "checkpoints\1720617415318.ta.chkpt" at 2024-07-10 21:16:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 119 / 20 / 150:  30%|██████              | 150/500 [08:00<18:40,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1720617577002.ta.chkpt" at 2024-07-10 21:19:37 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 33 / 271 / 46 / 350:  70%|██████████████      | 350/500 [19:30<08:21,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1720618267353.ta.chkpt" at 2024-07-10 21:31:07 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 312 / 50 / 400:  80%|████████████████    | 400/500 [22:07<05:31,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1720618424111.ta.chkpt" at 2024-07-10 21:33:44 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 348 / 59 / 450:  90%|██████████████████  | 450/500 [24:37<02:44,  3.28s/it]textattack: Saving checkpoint under "checkpoints\1720618574476.ta.chkpt" at 2024-07-10 21:36:14 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 387 / 63 / 500: 100%|████████████████████| 500/500 [27:00<00:00,  3.24s/it]textattack: Saving checkpoint under "checkpoints\1720618717917.ta.chkpt" at 2024-07-10 21:38:37 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 50 / 387 / 63 / 500: 100%|████████████████████| 500/500 [27:01<00:00,  3.24s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 50     |
| Number of failed attacks:     | 387    |
| Number of skipped attacks:    | 63     |
| Original accuracy:            | 87.4%  |
| Accuracy under attack:        | 77.4%  |
| Attack success rate:          | 11.44% |
| Average perturbed word %:     | 11.0%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 58.99  |
+-------------------------------+--------+


In [113]:
runAttack(disbert_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/disbertagattackknn'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 46 / 3 / 50:  10%|██▌                      | 50/500 [04:56<44:31,  5.94s/it]textattack: Saving checkpoint under "checkpoints\1720619014846.ta.chkpt" at 2024-07-10 21:43:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 88 / 8 / 100:  20%|████▌                  | 100/500 [10:03<40:15,  6.04s/it]textattack: Saving checkpoint under "checkpoints\1720619321928.ta.chkpt" at 2024-07-10 21:48:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 133 / 11 / 150:  30%|██████▎              | 150/500 [14:57<34:54,  5.99s/it]textattack: Saving checkpoint under "checkpoints\1720619615845.ta.chkpt" at 2024-07-10 21:53:35 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 18 / 297 / 35 / 350:  70%|██████████████      | 350/500 [33:56<14:32,  5.82s/it]textattack: Saving checkpoint under "checkpoints\1720620754256.ta.chkpt" at 2024-07-10 22:12:34 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 339 / 40 / 400:  80%|████████████████    | 400/500 [38:18<09:34,  5.75s/it]textattack: Saving checkpoint under "checkpoints\1720621016047.ta.chkpt" at 2024-07-10 22:16:56 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 381 / 43 / 450:  90%|██████████████████  | 450/500 [43:09<04:47,  5.76s/it]textattack: Saving checkpoint under "checkpoints\1720621307861.ta.chkpt" at 2024-07-10 22:21:47 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 425 / 45 / 500: 100%|████████████████████| 500/500 [47:26<00:00,  5.69s/it]textattack: Saving checkpoint under "checkpoints\1720621564732.ta.chkpt" at 2024-07-10 22:26:04 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 30 / 425 / 45 / 500: 100%|████████████████████| 500/500 [47:26<00:00,  5.69s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 30    |
| Number of failed attacks:     | 425   |
| Number of skipped attacks:    | 45    |
| Original accuracy:            | 91.0% |
| Accuracy under attack:        | 85.0% |
| Attack success rate:          | 6.59% |
| Average perturbed word %:     | 3.98% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 60.25 |
+-------------------------------+-------+


In [114]:
runAttack(clip_model_wrapper_lr_ag, n_samples, '../attack_results_new/ag/clipagattacklr'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattacklrsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  10%|██▌                      | 50/500 [03:12<28:56,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1720621757784.ta.chkpt" at 2024-07-10 22:29:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  20%|████▌                  | 100/500 [06:22<25:29,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1720621947132.ta.chkpt" at 2024-07-10 22:32:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 121 / 14 / 150:  30%|██████              | 150/500 [09:26<22:02,  3.78s/it]textattack: Saving checkpoint under "checkpoints\1720622131615.ta.chkpt" at 2024-07-10 22:35:31 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 49 / 268 / 33 / 350:  70%|██████████████      | 350/500 [21:32<09:14,  3.69s/it]textattack: Saving checkpoint under "checkpoints\1720622857554.ta.chkpt" at 2024-07-10 22:47:37 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 308 / 37 / 400:  80%|████████████████    | 400/500 [24:20<06:05,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1720623025413.ta.chkpt" at 2024-07-10 22:50:25 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 346 / 45 / 450:  90%|██████████████████  | 450/500 [27:15<03:01,  3.63s/it]textattack: Saving checkpoint under "checkpoints\1720623200559.ta.chkpt" at 2024-07-10 22:53:20 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 390 / 46 / 500: 100%|████████████████████| 500/500 [30:08<00:00,  3.62s/it]textattack: Saving checkpoint under "checkpoints\1720623373419.ta.chkpt" at 2024-07-10 22:56:13 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 64 / 390 / 46 / 500: 100%|████████████████████| 500/500 [30:08<00:00,  3.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 64     |
| Number of failed attacks:     | 390    |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 90.8%  |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 14.1%  |
| Average perturbed word %:     | 13.09% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 59.09  |
+-------------------------------+--------+


In [115]:
runAttack(clip_model_wrapper_rf_ag, n_samples, '../attack_results_new/ag/clipagattackrf'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackrfsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  10%|██▌                      | 50/500 [02:55<26:17,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1720623548886.ta.chkpt" at 2024-07-10 22:59:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 78 / 13 / 100:  20%|████▍                 | 100/500 [05:51<23:27,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1720623725429.ta.chkpt" at 2024-07-10 23:02:05 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 117 / 20 / 150:  30%|██████              | 150/500 [08:46<20:28,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1720623900076.ta.chkpt" at 2024-07-10 23:05:00 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 46 / 252 / 52 / 350:  70%|██████████████      | 350/500 [19:39<08:25,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1720624553048.ta.chkpt" at 2024-07-10 23:15:53 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 288 / 59 / 400:  80%|████████████████    | 400/500 [22:13<05:33,  3.33s/it]textattack: Saving checkpoint under "checkpoints\1720624707381.ta.chkpt" at 2024-07-10 23:18:27 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 318 / 71 / 450:  90%|██████████████████  | 450/500 [24:57<02:46,  3.33s/it]textattack: Saving checkpoint under "checkpoints\1720624871203.ta.chkpt" at 2024-07-10 23:21:11 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 363 / 73 / 500: 100%|████████████████████| 500/500 [27:58<00:00,  3.36s/it]textattack: Saving checkpoint under "checkpoints\1720625052087.ta.chkpt" at 2024-07-10 23:24:12 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 64 / 363 / 73 / 500: 100%|████████████████████| 500/500 [27:58<00:00,  3.36s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 64     |
| Number of failed attacks:     | 363    |
| Number of skipped attacks:    | 73     |
| Original accuracy:            | 85.4%  |
| Accuracy under attack:        | 72.6%  |
| Attack success rate:          | 14.99% |
| Average perturbed word %:     | 8.78%  |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 57.93  |
+-------------------------------+--------+


In [116]:
runAttack(clip_model_wrapper_knn_ag, n_samples, '../attack_results_new/ag/clipagattackknn'+suffix, target_ds_ag, 0.9, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/clipagattackknnsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  10%|██▌                      | 50/500 [04:39<41:52,  5.58s/it]textattack: Saving checkpoint under "checkpoints\1720625331389.ta.chkpt" at 2024-07-10 23:28:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 93 / 4 / 100:  20%|████▌                  | 100/500 [09:28<37:53,  5.68s/it]textattack: Saving checkpoint under "checkpoints\1720625620600.ta.chkpt" at 2024-07-10 23:33:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 137 / 7 / 150:  30%|██████▌               | 150/500 [13:53<32:25,  5.56s/it]textattack: Saving checkpoint under "checkpoints\1720625886163.ta.chkpt" at 2024-07-10 23:38:06 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 11 / 313 / 26 / 350:  70%|██████████████      | 350/500 [32:02<13:43,  5.49s/it]textattack: Saving checkpoint under "checkpoints\1720626974667.ta.chkpt" at 2024-07-10 23:56:14 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 358 / 28 / 400:  80%|████████████████    | 400/500 [36:22<09:05,  5.46s/it]textattack: Saving checkpoint under "checkpoints\1720627234289.ta.chkpt" at 2024-07-11 00:00:34 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 400 / 35 / 450:  90%|██████████████████  | 450/500 [40:56<04:32,  5.46s/it]textattack: Saving checkpoint under "checkpoints\1720627508454.ta.chkpt" at 2024-07-11 00:05:08 after 450 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 450 / 35 / 500: 100%|████████████████████| 500/500 [45:18<00:00,  5.44s/it]textattack: Saving checkpoint under "checkpoints\1720627771000.ta.chkpt" at 2024-07-11 00:09:31 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 15 / 450 / 35 / 500: 100%|████████████████████| 500/500 [45:18<00:00,  5.44s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 15    |
| Number of failed attacks:     | 450   |
| Number of skipped attacks:    | 35    |
| Original accuracy:            | 93.0% |
| Accuracy under attack:        | 90.0% |
| Attack success rate:          | 3.23% |
| Average perturbed word %:     | 4.64% |
| Average num. words per input: | 38.43 |
| Avg num queries:              | 60.59 |
+-------------------------------+-------+


In [117]:
et9 = datetime.datetime.now()
et9 - et8

datetime.timedelta(seconds=19009, microseconds=494854)

### Pretrained BERT

In [5]:
# bert-based-uncased
pretrained_bert_path = 'textattack/bert-base-uncased-ag-news'

pretrained_model_bert = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_bert_path)
pretrained_tokenizer_bert = transformers.AutoTokenizer.from_pretrained(pretrained_bert_path)
pretrained_wrapper_bert = HuggingFaceModelWrapper(pretrained_model_bert, pretrained_tokenizer_bert)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
n_samples = 200

In [10]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptep0841.csv', target_ds_ag, 0.5, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']




  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|▍                                                                                 | 1/200 [00:01<05:18,  1.60s/it]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<05:23,  1.62s/it]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                         | 2/200 [03:18<5:28:13, 99.46s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   1%|▎                         | 2/200 [03:18<5:28:15, 99.47s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   2%|▍                        | 3/200 [06:30<7:07:35, 130.23s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▍                        | 3/200 [06:30<7:07:37, 130.24s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▍                       | 4/200 [15:19<12:30:36, 229.78s/it]

[Succeeded / Failed / Skipped 



[Succeeded / Failed / Skipped / Total] 38 / 10 / 2 / 50:  26%|████▌             | 51/200 [5:04:04<14:48:23, 357.74s/it]

[Succeeded / Failed / Skipped / Total] 39 / 10 / 2 / 51:  26%|████▌             | 51/200 [5:04:04<14:48:23, 357.74s/it]

[Succeeded / Failed / Skipped / Total] 39 / 10 / 2 / 51:  26%|████▋             | 52/200 [5:07:22<14:34:50, 354.66s/it]

[Succeeded / Failed / Skipped / Total] 40 / 10 / 2 / 52:  26%|████▋             | 52/200 [5:07:22<14:34:50, 354.66s/it]

[Succeeded / Failed / Skipped / Total] 40 / 10 / 2 / 52:  26%|████▊             | 53/200 [5:11:53<14:25:02, 353.08s/it]

[Succeeded / Failed / Skipped / Total] 41 / 10 / 2 / 53:  26%|████▊             | 53/200 [5:11:53<14:25:02, 353.08s/it]

[Succeeded / Failed / Skipped / Total] 41 / 10 / 2 / 53:  27%|████▊             | 54/200 [5:14:04<14:09:08, 348.96s/it]

[Succeeded / Failed / Skipped / Total] 42 / 10 / 2 / 54:  27%|████▊             | 54/200 [5:14:04<14:09:08, 348.96s/it]

[Succeeded / Failed / Skipped 



[Succeeded / Failed / Skipped / Total] 77 / 17 / 6 / 100:  50%|████████▌        | 101/200 [9:00:35<8:49:53, 321.15s/it]

[Succeeded / Failed / Skipped / Total] 78 / 17 / 6 / 101:  50%|████████▌        | 101/200 [9:00:35<8:49:53, 321.15s/it]

[Succeeded / Failed / Skipped / Total] 78 / 17 / 6 / 101:  51%|████████▋        | 102/200 [9:04:56<8:43:34, 320.55s/it]

[Succeeded / Failed / Skipped / Total] 79 / 17 / 6 / 102:  51%|████████▋        | 102/200 [9:04:56<8:43:34, 320.55s/it]

[Succeeded / Failed / Skipped / Total] 79 / 17 / 6 / 102:  52%|████████▊        | 103/200 [9:10:17<8:38:14, 320.56s/it]

[Succeeded / Failed / Skipped / Total] 80 / 17 / 6 / 103:  52%|████████▊        | 103/200 [9:10:17<8:38:14, 320.56s/it]

[Succeeded / Failed / Skipped / Total] 80 / 17 / 6 / 103:  52%|████████▊        | 104/200 [9:21:33<8:38:21, 323.97s/it]

[Succeeded / Failed / Skipped / Total] 81 / 17 / 6 / 104:  52%|████████▊        | 104/200 [9:21:33<8:38:21, 323.97s/it]

[Succeeded / Failed / Skipped 



[Succeeded / Failed / Skipped / Total] 115 / 27 / 8 / 150:  76%|███████████▎   | 151/200 [13:43:30<4:27:13, 327.22s/it]

[Succeeded / Failed / Skipped / Total] 116 / 27 / 8 / 151:  76%|███████████▎   | 151/200 [13:43:30<4:27:13, 327.22s/it]

[Succeeded / Failed / Skipped / Total] 116 / 27 / 8 / 151:  76%|███████████▍   | 152/200 [13:51:58<4:22:43, 328.41s/it]

[Succeeded / Failed / Skipped / Total] 117 / 27 / 8 / 152:  76%|███████████▍   | 152/200 [13:51:58<4:22:43, 328.41s/it]

[Succeeded / Failed / Skipped / Total] 117 / 27 / 8 / 152:  76%|███████████▍   | 153/200 [14:01:26<4:18:28, 329.97s/it]

[Succeeded / Failed / Skipped / Total] 117 / 28 / 8 / 153:  76%|███████████▍   | 153/200 [14:01:26<4:18:28, 329.97s/it]

[Succeeded / Failed / Skipped / Total] 117 / 28 / 8 / 153:  77%|███████████▌   | 154/200 [14:03:35<4:11:58, 328.67s/it]

[Succeeded / Failed / Skipped / Total] 118 / 28 / 8 / 154:  77%|███████████▌   | 154/200 [14:03:35<4:11:58, 328.67s/it]

[Succeeded / Failed / Skipped 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 149    |
| Number of failed attacks:     | 40     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 78.84% |
| Average perturbed word %:     | 22.58% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 325.2  |
+-------------------------------+--------+


In [11]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptep0873.csv', target_ds_ag, 0.5, 1-0.4/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 32 / 16 / 2 / 50:  25%|████▌             | 50/200 [4:28:35<13:25:46, 322.31s/it]textattack: Saving checkpoint under "checkpoints\1720793340274.ta.chkpt" at 2024-07-12 22:09:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 28 / 6 / 100:  50%|████████▌        | 100/200 [8:59:37<8:59:37, 323.77s/it]textattack: Saving checkpoint under "checkpoints\1720809601931.ta.chkpt" at 2024-07-13 02:40:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 103 / 39 / 8 / 150:  75%|███████████▎   | 150/200 [13:46:12<4:35:24, 330.48s/it]textattack: Saving checkpoint under "checkpoints\1720826797255.ta.chkpt" at 2024-07-13 07:26:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 136 / 53 / 11 / 200: 100%|████████████████| 200/200 [19:05:56<00:00, 343.78s/it]textattack: Saving checkpoint under "checkpoints\1720845981343.ta.chkpt" at 2024-07-13 12:46:21 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 136 / 53 / 11 / 200: 100%|████████████████| 200/200 [19:05:56<00:00, 343.78s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 136    |
| Number of failed attacks:     | 53     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 26.5%  |
| Attack success rate:          | 71.96% |
| Average perturbed word %:     | 24.18% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 306.38 |
+-------------------------------+--------+


In [12]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptep09.csv', target_ds_ag, 0.5, 0.9)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 13 / 35 / 2 / 50:  25%|████▌             | 50/200 [4:00:03<12:00:11, 288.08s/it]textattack: Saving checkpoint under "checkpoints\1720860385283.ta.chkpt" at 2024-07-13 16:46:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 54 / 6 / 100:  50%|████████▌        | 100/200 [8:00:46<8:00:46, 288.47s/it]textattack: Saving checkpoint under "checkpoints\1720874828062.ta.chkpt" at 2024-07-13 20:47:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 65 / 77 / 8 / 150:  75%|████████████    | 150/200 [11:57:44<3:59:14, 287.10s/it]textattack: Saving checkpoint under "checkpoints\1720889045789.ta.chkpt" at 2024-07-14 00:44:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 98 / 11 / 200: 100%|█████████████████| 200/200 [16:13:40<00:00, 292.10s/it]textattack: Saving checkpoint under "checkpoints\1720904402020.ta.chkpt" at 2024-07-14 05:00:02 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 91 / 98 / 11 / 200: 100%|█████████████████| 200/200 [16:13:40<00:00, 292.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 91     |
| Number of failed attacks:     | 98     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 49.0%  |
| Attack success rate:          | 48.15% |
| Average perturbed word %:     | 22.58% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 231.81 |
+-------------------------------+--------+


In [13]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptep095.csv', target_ds_ag, 0.5, 0.95)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  25%|█████▊                 | 50/200 [37:45<1:53:17, 45.31s/it]textattack: Saving checkpoint under "checkpoints\1720906667781.ta.chkpt" at 2024-07-14 05:37:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 93 / 6 / 100:  50%|█████████▌         | 100/200 [1:18:58<1:18:58, 47.38s/it]textattack: Saving checkpoint under "checkpoints\1720909140264.ta.chkpt" at 2024-07-14 06:19:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 140 / 8 / 150:  75%|███████████████     | 150/200 [2:02:32<40:50, 49.01s/it]textattack: Saving checkpoint under "checkpoints\1720911754124.ta.chkpt" at 2024-07-14 07:02:34 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200: 100%|███████████████████| 200/200 [2:44:58<00:00, 49.49s/it]textattack: Saving checkpoint under "checkpoints\1720914301082.ta.chkpt" at 2024-07-14 07:45:01 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 186 / 11 / 200: 100%|███████████████████| 200/200 [2:44:59<00:00, 49.50s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 3     |
| Number of failed attacks:     | 186   |
| Number of skipped attacks:    | 11    |
| Original accuracy:            | 94.5% |
| Accuracy under attack:        | 93.0% |
| Attack success rate:          | 1.59% |
| Average perturbed word %:     | 3.25% |
| Average num. words per input: | 38.66 |
| Avg num queries:              | 36.96 |
+-------------------------------+-------+


In [15]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptep0809.csv', target_ds_ag, 0.5, 1-0.6/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']




  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|▍                                                                                 | 1/200 [00:01<04:09,  1.25s/it]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<04:14,  1.28s/it]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                        | 2/200 [04:39<7:41:33, 139.86s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   1%|▎                        | 2/200 [04:39<7:41:35, 139.88s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   2%|▎                       | 3/200 [09:18<10:11:02, 186.10s/it]

[Succeeded / Failed / Skipped / Total] 25 / 7 / 1 / 33:  16%|██▋             | 33/200 [68:04:51<344:31:52, 7427.02s/it]
textattack: CSVLogger exiting without calling flush().


[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   2%|▍                       | 4/200 [20:



[Succeeded / Failed / Skipped / Total] 38 / 10 / 2 / 50:  26%|████▌             | 51/200 [6:18:22<18:25:26, 445.15s/it]

[Succeeded / Failed / Skipped / Total] 39 / 10 / 2 / 51:  26%|████▌             | 51/200 [6:18:22<18:25:27, 445.15s/it]

[Succeeded / Failed / Skipped / Total] 39 / 10 / 2 / 51:  26%|████▋             | 52/200 [6:22:08<18:07:36, 440.92s/it]

[Succeeded / Failed / Skipped / Total] 40 / 10 / 2 / 52:  26%|████▋             | 52/200 [6:22:08<18:07:36, 440.93s/it]

[Succeeded / Failed / Skipped / Total] 40 / 10 / 2 / 52:  26%|████▊             | 53/200 [6:28:08<17:56:32, 439.40s/it]

[Succeeded / Failed / Skipped / Total] 41 / 10 / 2 / 53:  26%|████▊             | 53/200 [6:28:08<17:56:32, 439.40s/it]

[Succeeded / Failed / Skipped / Total] 41 / 10 / 2 / 53:  27%|████▊             | 54/200 [6:31:10<17:37:35, 434.63s/it]

[Succeeded / Failed / Skipped / Total] 42 / 10 / 2 / 54:  27%|████▊             | 54/200 [6:31:10<17:37:36, 434.63s/it]

[Succeeded / Failed / Skipped 



[Succeeded / Failed / Skipped / Total] 79 / 15 / 6 / 100:  50%|███████▌       | 101/200 [11:07:20<10:54:07, 396.44s/it]

[Succeeded / Failed / Skipped / Total] 80 / 15 / 6 / 101:  50%|███████▌       | 101/200 [11:07:20<10:54:07, 396.44s/it]

[Succeeded / Failed / Skipped / Total] 80 / 15 / 6 / 101:  51%|███████▋       | 102/200 [11:12:34<10:46:12, 395.63s/it]

[Succeeded / Failed / Skipped / Total] 81 / 15 / 6 / 102:  51%|███████▋       | 102/200 [11:12:34<10:46:12, 395.63s/it]

[Succeeded / Failed / Skipped / Total] 81 / 15 / 6 / 102:  52%|███████▋       | 103/200 [11:18:51<10:39:18, 395.45s/it]

[Succeeded / Failed / Skipped / Total] 82 / 15 / 6 / 103:  52%|███████▋       | 103/200 [11:18:51<10:39:18, 395.45s/it]

[Succeeded / Failed / Skipped / Total] 82 / 15 / 6 / 103:  52%|███████▊       | 104/200 [11:31:52<10:38:39, 399.16s/it]

[Succeeded / Failed / Skipped / Total] 83 / 15 / 6 / 104:  52%|███████▊       | 104/200 [11:31:52<10:38:39, 399.16s/it]

[Succeeded / Failed / Skipped 



[Succeeded / Failed / Skipped / Total] 117 / 25 / 8 / 150:  76%|███████████▎   | 151/200 [16:57:19<5:30:07, 404.24s/it]

[Succeeded / Failed / Skipped / Total] 118 / 25 / 8 / 151:  76%|███████████▎   | 151/200 [16:57:19<5:30:07, 404.24s/it]

[Succeeded / Failed / Skipped / Total] 118 / 25 / 8 / 151:  76%|███████████▍   | 152/200 [17:07:37<5:24:30, 405.64s/it]

[Succeeded / Failed / Skipped / Total] 119 / 25 / 8 / 152:  76%|███████████▍   | 152/200 [17:07:37<5:24:30, 405.64s/it]

[Succeeded / Failed / Skipped / Total] 119 / 25 / 8 / 152:  76%|███████████▍   | 153/200 [17:17:38<5:18:45, 406.92s/it]

[Succeeded / Failed / Skipped / Total] 120 / 25 / 8 / 153:  76%|███████████▍   | 153/200 [17:17:38<5:18:45, 406.92s/it]

[Succeeded / Failed / Skipped / Total] 120 / 25 / 8 / 153:  77%|███████████▌   | 154/200 [17:20:23<5:10:45, 405.35s/it]

[Succeeded / Failed / Skipped / Total] 121 / 25 / 8 / 154:  77%|███████████▌   | 154/200 [17:20:23<5:10:45, 405.35s/it]

[Succeeded / Failed / Skipped 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 154    |
| Number of failed attacks:     | 35     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 17.5%  |
| Attack success rate:          | 81.48% |
| Average perturbed word %:     | 23.36% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 333.75 |
+-------------------------------+--------+


In [16]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptsim06.csv', target_ds_ag, 0.6, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']



  0%|▍                                                                                 | 1/200 [00:01<05:08,  1.55s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<05:11,  1.57s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                        | 2/200 [05:32<9:08:36, 166.24s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   1%|▎                        | 2/200 [05:32<9:08:37, 166.25s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   2%|▎                       | 3/200 [14:48<16:12:20, 296.14s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:   2%|▎                       | 3/200 [14:48<16:12:21, 296.15s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:   2%|▍                       | 4/200 [20:33<16:47:20, 308.37s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 1 / 4:   2%|▍                       | 4/200 [20:33<16:47:20, 308.37s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 33 / 15 / 2 / 50:  26%|████▌             | 51/200 [4:28:25<13:04:12, 315.79s/it]
[Succeeded / Failed / Skipped / Total] 34 / 15 / 2 / 51:  26%|████▌             | 51/200 [4:28:25<13:04:12, 315.79s/it]
[Succeeded / Failed / Skipped / Total] 34 / 15 / 2 / 51:  26%|████▋             | 52/200 [4:32:55<12:56:48, 314.92s/it]
[Succeeded / Failed / Skipped / Total] 35 / 15 / 2 / 52:  26%|████▋             | 52/200 [4:32:55<12:56:48, 314.92s/it]
[Succeeded / Failed / Skipped / Total] 35 / 15 / 2 / 52:  26%|████▊             | 53/200 [4:39:31<12:55:17, 316.44s/it]
[Succeeded / Failed / Skipped / Total] 36 / 15 / 2 / 53:  26%|████▊             | 53/200 [4:39:31<12:55:17, 316.44s/it]
[Succeeded / Failed / Skipped / Total] 36 / 15 / 2 / 53:  27%|████▊             | 54/200 [4:42:04<12:42:37, 313.41s/it]
[Succeeded / Failed / Skipped / Total] 37 / 15 / 2 / 54:  27%|████▊             | 54/200 [4:42:04<12:42:37, 313.41s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 70 / 24 / 6 / 100:  50%|████████▌        | 101/200 [8:36:06<8:25:53, 306.60s/it]
[Succeeded / Failed / Skipped / Total] 71 / 24 / 6 / 101:  50%|████████▌        | 101/200 [8:36:06<8:25:53, 306.60s/it]
[Succeeded / Failed / Skipped / Total] 71 / 24 / 6 / 101:  51%|████████▋        | 102/200 [8:39:27<8:19:05, 305.56s/it]
[Succeeded / Failed / Skipped / Total] 72 / 24 / 6 / 102:  51%|████████▋        | 102/200 [8:39:27<8:19:05, 305.56s/it]
[Succeeded / Failed / Skipped / Total] 72 / 24 / 6 / 102:  52%|████████▊        | 103/200 [8:43:56<8:13:25, 305.21s/it]
[Succeeded / Failed / Skipped / Total] 73 / 24 / 6 / 103:  52%|████████▊        | 103/200 [8:43:56<8:13:25, 305.21s/it]
[Succeeded / Failed / Skipped / Total] 73 / 24 / 6 / 103:  52%|████████▊        | 104/200 [8:50:57<8:10:07, 306.33s/it]
[Succeeded / Failed / Skipped / Total] 74 / 24 / 6 / 104:  52%|████████▊        | 104/200 [8:50:58<8:10:07, 306.33s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 101 / 41 / 8 / 150:  76%|███████████▎   | 151/200 [12:56:44<4:12:03, 308.64s/it]
[Succeeded / Failed / Skipped / Total] 102 / 41 / 8 / 151:  76%|███████████▎   | 151/200 [12:56:44<4:12:03, 308.64s/it]
[Succeeded / Failed / Skipped / Total] 102 / 41 / 8 / 151:  76%|███████████▍   | 152/200 [13:02:22<4:07:04, 308.83s/it]
[Succeeded / Failed / Skipped / Total] 103 / 41 / 8 / 152:  76%|███████████▍   | 152/200 [13:02:22<4:07:04, 308.83s/it]
[Succeeded / Failed / Skipped / Total] 103 / 41 / 8 / 152:  76%|███████████▍   | 153/200 [13:09:49<4:02:37, 309.74s/it]
[Succeeded / Failed / Skipped / Total] 103 / 42 / 8 / 153:  76%|███████████▍   | 153/200 [13:09:49<4:02:37, 309.74s/it]
[Succeeded / Failed / Skipped / Total] 103 / 42 / 8 / 153:  77%|███████████▌   | 154/200 [13:11:54<3:56:32, 308.54s/it]
[Succeeded / Failed / Skipped / Total] 104 / 42 / 8 / 154:  77%|███████████▌   | 154/200 [13:11:54<3:56:32, 308.54s/it]
[Succeeded / Failed / Skipped / Total] 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 135    |
| Number of failed attacks:     | 54     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 27.0%  |
| Attack success rate:          | 71.43% |
| Average perturbed word %:     | 24.03% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 253.44 |
+-------------------------------+--------+


In [17]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptsim07.csv', target_ds_ag, 0.7, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']



  0%|▍                                                                                 | 1/200 [00:01<04:58,  1.50s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<05:01,  1.52s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                        | 2/200 [03:49<6:19:26, 114.98s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   1%|▎                        | 2/200 [03:49<6:19:27, 114.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:   2%|▍                        | 3/200 [09:02<9:54:15, 180.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:   2%|▍                        | 3/200 [09:02<9:54:16, 181.00s/it]
[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:   2%|▌                        | 4/200 [12:07<9:53:55, 181.82s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 1 / 4:   2%|▌                        | 4/200 [12:07<9:53:57, 181.82s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 23 / 25 / 2 / 50:  26%|████▊              | 51/200 [3:05:16<9:01:16, 217.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 26 / 2 / 51:  26%|████▊              | 51/200 [3:05:16<9:01:16, 217.96s/it]
[Succeeded / Failed / Skipped / Total] 23 / 26 / 2 / 51:  26%|████▉              | 52/200 [3:09:07<8:58:15, 218.21s/it]
[Succeeded / Failed / Skipped / Total] 24 / 26 / 2 / 52:  26%|████▉              | 52/200 [3:09:07<8:58:15, 218.21s/it]
[Succeeded / Failed / Skipped / Total] 24 / 26 / 2 / 52:  26%|█████              | 53/200 [3:13:13<8:55:56, 218.75s/it]
[Succeeded / Failed / Skipped / Total] 24 / 27 / 2 / 53:  26%|█████              | 53/200 [3:13:14<8:55:56, 218.75s/it]
[Succeeded / Failed / Skipped / Total] 24 / 27 / 2 / 53:  27%|█████▏             | 54/200 [3:15:03<8:47:23, 216.73s/it]
[Succeeded / Failed / Skipped / Total] 25 / 27 / 2 / 54:  27%|█████▏             | 54/200 [3:15:03<8:47:23, 216.73s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 55 / 39 / 6 / 100:  50%|████████▌        | 101/200 [5:40:17<5:33:33, 202.16s/it]
[Succeeded / Failed / Skipped / Total] 56 / 39 / 6 / 101:  50%|████████▌        | 101/200 [5:40:17<5:33:33, 202.16s/it]
[Succeeded / Failed / Skipped / Total] 56 / 39 / 6 / 101:  51%|████████▋        | 102/200 [5:42:48<5:29:22, 201.65s/it]
[Succeeded / Failed / Skipped / Total] 57 / 39 / 6 / 102:  51%|████████▋        | 102/200 [5:42:48<5:29:22, 201.65s/it]
[Succeeded / Failed / Skipped / Total] 57 / 39 / 6 / 102:  52%|████████▊        | 103/200 [5:45:47<5:25:38, 201.43s/it]
[Succeeded / Failed / Skipped / Total] 58 / 39 / 6 / 103:  52%|████████▊        | 103/200 [5:45:47<5:25:38, 201.43s/it]
[Succeeded / Failed / Skipped / Total] 58 / 39 / 6 / 103:  52%|████████▊        | 104/200 [5:51:04<5:24:03, 202.54s/it]
[Succeeded / Failed / Skipped / Total] 58 / 40 / 6 / 104:  52%|████████▊        | 104/200 [5:51:04<5:24:03, 202.54s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 81 / 61 / 8 / 150:  76%|████████████▊    | 151/200 [8:19:13<2:42:00, 198.37s/it]
[Succeeded / Failed / Skipped / Total] 82 / 61 / 8 / 151:  76%|████████████▊    | 151/200 [8:19:13<2:42:00, 198.37s/it]
[Succeeded / Failed / Skipped / Total] 82 / 61 / 8 / 151:  76%|████████████▉    | 152/200 [8:24:41<2:39:22, 199.22s/it]
[Succeeded / Failed / Skipped / Total] 82 / 62 / 8 / 152:  76%|████████████▉    | 152/200 [8:24:41<2:39:22, 199.22s/it]
[Succeeded / Failed / Skipped / Total] 82 / 62 / 8 / 152:  76%|█████████████    | 153/200 [8:28:48<2:36:18, 199.54s/it]
[Succeeded / Failed / Skipped / Total] 82 / 63 / 8 / 153:  76%|█████████████    | 153/200 [8:28:48<2:36:18, 199.54s/it]
[Succeeded / Failed / Skipped / Total] 82 / 63 / 8 / 153:  77%|█████████████    | 154/200 [8:30:02<2:32:20, 198.71s/it]
[Succeeded / Failed / Skipped / Total] 83 / 63 / 8 / 154:  77%|█████████████    | 154/200 [8:30:02<2:32:20, 198.71s/it]
[Succeeded / Failed / Skipped / Total] 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 109    |
| Number of failed attacks:     | 80     |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 57.67% |
| Average perturbed word %:     | 23.51% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 174.73 |
+-------------------------------+--------+


In [18]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptsim08.csv', target_ds_ag, 0.8, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']



  0%|▍                                                                                 | 1/200 [00:01<04:55,  1.48s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<04:59,  1.51s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                         | 2/200 [02:20<3:52:25, 70.43s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:   1%|▎                         | 2/200 [02:20<3:52:26, 70.44s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:   2%|▍                         | 3/200 [04:46<5:13:52, 95.60s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:   2%|▍                         | 3/200 [04:46<5:13:53, 95.60s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:   2%|▌                         | 4/200 [06:28<5:17:15, 97.12s/it]
[Succeeded / Failed / Skipped / Total] 0 / 3 / 1 / 4:   2%|▌                         | 4/200 [06:28<5:17:16, 97.12s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 15 / 33 / 2 / 50:  26%|████▊              | 51/200 [1:46:53<5:12:17, 125.75s/it]
[Succeeded / Failed / Skipped / Total] 15 / 34 / 2 / 51:  26%|████▊              | 51/200 [1:46:53<5:12:17, 125.75s/it]
[Succeeded / Failed / Skipped / Total] 15 / 34 / 2 / 51:  26%|████▉              | 52/200 [1:49:26<5:11:28, 126.28s/it]
[Succeeded / Failed / Skipped / Total] 15 / 35 / 2 / 52:  26%|████▉              | 52/200 [1:49:26<5:11:28, 126.28s/it]
[Succeeded / Failed / Skipped / Total] 15 / 35 / 2 / 52:  26%|█████              | 53/200 [1:51:08<5:08:16, 125.83s/it]
[Succeeded / Failed / Skipped / Total] 15 / 36 / 2 / 53:  26%|█████              | 53/200 [1:51:08<5:08:16, 125.83s/it]
[Succeeded / Failed / Skipped / Total] 15 / 36 / 2 / 53:  27%|█████▏             | 54/200 [1:53:17<5:06:18, 125.88s/it]
[Succeeded / Failed / Skipped / Total] 16 / 36 / 2 / 54:  27%|█████▏             | 54/200 [1:53:17<5:06:19, 125.88s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 33 / 61 / 6 / 100:  50%|████████▌        | 101/200 [3:29:30<3:25:21, 124.46s/it]
[Succeeded / Failed / Skipped / Total] 34 / 61 / 6 / 101:  50%|████████▌        | 101/200 [3:29:30<3:25:21, 124.46s/it]
[Succeeded / Failed / Skipped / Total] 34 / 61 / 6 / 101:  51%|████████▋        | 102/200 [3:31:18<3:23:01, 124.30s/it]
[Succeeded / Failed / Skipped / Total] 35 / 61 / 6 / 102:  51%|████████▋        | 102/200 [3:31:18<3:23:01, 124.30s/it]
[Succeeded / Failed / Skipped / Total] 35 / 61 / 6 / 102:  52%|████████▊        | 103/200 [3:34:41<3:22:11, 125.06s/it]
[Succeeded / Failed / Skipped / Total] 35 / 62 / 6 / 103:  52%|████████▊        | 103/200 [3:34:41<3:22:11, 125.06s/it]
[Succeeded / Failed / Skipped / Total] 35 / 62 / 6 / 103:  52%|████████▊        | 104/200 [3:36:17<3:19:39, 124.79s/it]
[Succeeded / Failed / Skipped / Total] 35 / 63 / 6 / 104:  52%|████████▊        | 104/200 [3:36:17<3:19:39, 124.79s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 52 / 90 / 8 / 150:  76%|████████████▊    | 151/200 [5:09:53<1:40:33, 123.14s/it]
[Succeeded / Failed / Skipped / Total] 53 / 90 / 8 / 151:  76%|████████████▊    | 151/200 [5:09:53<1:40:33, 123.14s/it]
[Succeeded / Failed / Skipped / Total] 53 / 90 / 8 / 151:  76%|████████████▉    | 152/200 [5:12:36<1:38:42, 123.40s/it]
[Succeeded / Failed / Skipped / Total] 53 / 91 / 8 / 152:  76%|████████████▉    | 152/200 [5:12:36<1:38:42, 123.40s/it]
[Succeeded / Failed / Skipped / Total] 53 / 91 / 8 / 152:  76%|█████████████    | 153/200 [5:14:59<1:36:45, 123.53s/it]
[Succeeded / Failed / Skipped / Total] 53 / 92 / 8 / 153:  76%|█████████████    | 153/200 [5:14:59<1:36:45, 123.53s/it]
[Succeeded / Failed / Skipped / Total] 53 / 92 / 8 / 153:  77%|█████████████    | 154/200 [5:16:24<1:34:30, 123.28s/it]
[Succeeded / Failed / Skipped / Total] 54 / 92 / 8 / 154:  77%|█████████████    | 154/200 [5:16:25<1:34:30, 123.28s/it]
[Succeeded / Failed / Skipped / Total] 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 71     |
| Number of failed attacks:     | 118    |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 59.0%  |
| Attack success rate:          | 37.57% |
| Average perturbed word %:     | 21.58% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 105.63 |
+-------------------------------+--------+


In [19]:
runAttack(pretrained_wrapper_bert, n_samples, '../attack_results_new/ag/bertagattackptsim09.csv', target_ds_ag, 0.9, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/bertagattackptsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']



  0%|▍                                                                                 | 1/200 [00:01<05:21,  1.62s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   0%|▏                           | 1/200 [00:01<05:25,  1.64s/it]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|▎                         | 2/200 [01:26<2:23:29, 43.48s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:   1%|▎                         | 2/200 [01:26<2:23:31, 43.49s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:   2%|▍                         | 3/200 [02:47<3:03:01, 55.74s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:   2%|▍                         | 3/200 [02:47<3:03:02, 55.75s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:   2%|▌                         | 4/200 [03:57<3:14:00, 59.39s/it]
[Succeeded / Failed / Skipped / Total] 0 / 3 / 1 / 4:   2%|▌                         | 4/200 [03:57<3:14:01, 59.39s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 5 / 43 / 2 / 50:  26%|█████▊                 | 51/200 [57:51<2:49:00, 68.06s/it]
[Succeeded / Failed / Skipped / Total] 5 / 44 / 2 / 51:  26%|█████▊                 | 51/200 [57:51<2:49:00, 68.06s/it]
[Succeeded / Failed / Skipped / Total] 5 / 44 / 2 / 51:  26%|█████▉                 | 52/200 [59:10<2:48:24, 68.27s/it]
[Succeeded / Failed / Skipped / Total] 5 / 45 / 2 / 52:  26%|█████▉                 | 52/200 [59:10<2:48:24, 68.27s/it]
[Succeeded / Failed / Skipped / Total] 5 / 45 / 2 / 52:  26%|█████▌               | 53/200 [1:00:05<2:46:40, 68.03s/it]
[Succeeded / Failed / Skipped / Total] 5 / 46 / 2 / 53:  26%|█████▌               | 53/200 [1:00:05<2:46:41, 68.03s/it]
[Succeeded / Failed / Skipped / Total] 5 / 46 / 2 / 53:  27%|█████▋               | 54/200 [1:01:15<2:45:36, 68.06s/it]
[Succeeded / Failed / Skipped / Total] 5 / 47 / 2 / 54:  27%|█████▋               | 54/200 [1:01:15<2:45:37, 68.06s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 9 / 85 / 6 / 100:  50%|█████████▌         | 101/200 [1:58:43<1:56:22, 70.53s/it]
[Succeeded / Failed / Skipped / Total] 10 / 85 / 6 / 101:  50%|█████████         | 101/200 [1:58:43<1:56:22, 70.53s/it]
[Succeeded / Failed / Skipped / Total] 10 / 85 / 6 / 101:  51%|█████████▏        | 102/200 [1:59:41<1:54:59, 70.40s/it]
[Succeeded / Failed / Skipped / Total] 10 / 86 / 6 / 102:  51%|█████████▏        | 102/200 [1:59:41<1:54:59, 70.40s/it]
[Succeeded / Failed / Skipped / Total] 10 / 86 / 6 / 102:  52%|█████████▎        | 103/200 [2:01:17<1:54:13, 70.65s/it]
[Succeeded / Failed / Skipped / Total] 10 / 87 / 6 / 103:  52%|█████████▎        | 103/200 [2:01:17<1:54:13, 70.65s/it]
[Succeeded / Failed / Skipped / Total] 10 / 87 / 6 / 103:  52%|█████████▎        | 104/200 [2:02:03<1:52:40, 70.42s/it]
[Succeeded / Failed / Skipped / Total] 10 / 88 / 6 / 104:  52%|█████████▎        | 104/200 [2:02:03<1:52:40, 70.42s/it]
[Succeeded / Failed / Skipped / Total] 


[Succeeded / Failed / Skipped / Total] 17 / 125 / 8 / 150:  76%|██████████████▎    | 151/200 [2:57:55<57:44, 70.70s/it]
[Succeeded / Failed / Skipped / Total] 17 / 126 / 8 / 151:  76%|██████████████▎    | 151/200 [2:57:55<57:44, 70.70s/it]
[Succeeded / Failed / Skipped / Total] 17 / 126 / 8 / 151:  76%|██████████████▍    | 152/200 [2:59:18<56:37, 70.78s/it]
[Succeeded / Failed / Skipped / Total] 17 / 127 / 8 / 152:  76%|██████████████▍    | 152/200 [2:59:18<56:37, 70.78s/it]
[Succeeded / Failed / Skipped / Total] 17 / 127 / 8 / 152:  76%|██████████████▌    | 153/200 [3:00:22<55:24, 70.74s/it]
[Succeeded / Failed / Skipped / Total] 17 / 128 / 8 / 153:  76%|██████████████▌    | 153/200 [3:00:22<55:24, 70.74s/it]
[Succeeded / Failed / Skipped / Total] 17 / 128 / 8 / 153:  77%|██████████████▋    | 154/200 [3:01:42<54:16, 70.80s/it]
[Succeeded / Failed / Skipped / Total] 18 / 128 / 8 / 154:  77%|██████████████▋    | 154/200 [3:01:42<54:16, 70.80s/it]
[Succeeded / Failed / Skipped / Total] 





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 168    |
| Number of skipped attacks:    | 11     |
| Original accuracy:            | 94.5%  |
| Accuracy under attack:        | 84.0%  |
| Attack success rate:          | 11.11% |
| Average perturbed word %:     | 12.58% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 59.9   |
+-------------------------------+--------+


### Pre-trained DISBERT

In [25]:
# bert-based-uncased
pretrained_disbert_path = 'textattack/distilbert-base-uncased-ag-news'

pretrained_model_disbert = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_disbert_path)
pretrained_tokenizer_disbert = transformers.AutoTokenizer.from_pretrained(pretrained_disbert_path)
pretrained_wrapper_disbert = HuggingFaceModelWrapper(pretrained_model_disbert, pretrained_tokenizer_disbert)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptep0841.csv', target_ds_ag, 0.5, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 40 / 8 / 2 / 50:  25%|█████               | 50/200 [2:55:54<8:47:44, 211.10s/it]textattack: Saving checkpoint under "checkpoints\1721414842581.ta.chkpt" at 2024-07-20 02:47:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 16 / 2 / 51:  26%|████            | 51/200 [25:20:02<74:00:54, 1788.29s/it]
textattack: CSVLogger exiting without calling flush().
[Succeeded / Failed / Skipped / Total] 83 / 13 / 4 / 100:  50%|████████▌        | 100/200 [5:02:52<5:02:52, 181.72s/it]textattack: Saving checkpoint under "checkpoints\1721422459943.ta.chkpt" at 2024-07-20 04:54:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 121 / 22 / 7 / 150:  75%|████████████    | 150/200 [7:36:00<2:32:00, 182.40s/it]textattack: Saving checkpoint under "checkpoints\1721431648360.ta.chkpt" at 2024-07-20 07:27:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 158 / 30 / 12 / 200: 100%|████████████████| 200/200 [10:12:07<00:00, 183.64s/it]textattack: Saving checkpoint under "checkpoints\1721441014890.ta.chkpt" at 2024-07-20 10:03:34 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 158 / 30 / 12 / 200: 100%|████████████████| 200/200 [10:12:07<00:00, 183.64s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 158    |
| Number of failed attacks:     | 30     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 15.0%  |
| Attack success rate:          | 84.04% |
| Average perturbed word %:     | 25.23% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 318.94 |
+-------------------------------+--------+


In [27]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptep0809.csv', target_ds_ag, 0.5, 1-0.6/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 38 / 10 / 2 / 50:  25%|████▊              | 50/200 [2:58:51<8:56:34, 214.63s/it]textattack: Saving checkpoint under "checkpoints\1721451746784.ta.chkpt" at 2024-07-20 13:02:26 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 14 / 4 / 100:  50%|████████▌        | 100/200 [5:11:58<5:11:58, 187.19s/it]textattack: Saving checkpoint under "checkpoints\1721459734064.ta.chkpt" at 2024-07-20 15:15:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 121 / 22 / 7 / 150:  75%|████████████    | 150/200 [7:51:48<2:37:16, 188.72s/it]textattack: Saving checkpoint under "checkpoints\1721469323986.ta.chkpt" at 2024-07-20 17:55:23 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 159 / 29 / 12 / 200: 100%|████████████████| 200/200 [10:39:56<00:00, 191.98s/it]textattack: Saving checkpoint under "checkpoints\1721479411891.ta.chkpt" at 2024-07-20 20:43:31 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 159 / 29 / 12 / 200: 100%|████████████████| 200/200 [10:39:56<00:00, 191.98s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 159    |
| Number of failed attacks:     | 29     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 14.5%  |
| Attack success rate:          | 84.57% |
| Average perturbed word %:     | 24.68% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 322.38 |
+-------------------------------+--------+


In [28]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptep0873.csv', target_ds_ag, 0.5, 1-0.4/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 34 / 14 / 2 / 50:  25%|████▊              | 50/200 [2:31:03<7:33:11, 181.27s/it]textattack: Saving checkpoint under "checkpoints\1721488475682.ta.chkpt" at 2024-07-20 23:14:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 21 / 4 / 100:  50%|████████▌        | 100/200 [4:36:41<4:36:41, 166.01s/it]textattack: Saving checkpoint under "checkpoints\1721496013297.ta.chkpt" at 2024-07-21 01:20:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 106 / 37 / 7 / 150:  75%|████████████    | 150/200 [7:02:16<2:20:45, 168.91s/it]textattack: Saving checkpoint under "checkpoints\1721504748133.ta.chkpt" at 2024-07-21 03:45:48 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 140 / 48 / 12 / 200: 100%|█████████████████| 200/200 [9:28:54<00:00, 170.67s/it]textattack: Saving checkpoint under "checkpoints\1721513546127.ta.chkpt" at 2024-07-21 06:12:26 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 140 / 48 / 12 / 200: 100%|█████████████████| 200/200 [9:28:54<00:00, 170.67s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 140    |
| Number of failed attacks:     | 48     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 74.47% |
| Average perturbed word %:     | 25.24% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 295.66 |
+-------------------------------+--------+


In [29]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptep09.csv', target_ds_ag, 0.5, 0.9)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 19 / 29 / 2 / 50:  25%|████▊              | 50/200 [1:49:14<5:27:43, 131.09s/it]textattack: Saving checkpoint under "checkpoints\1721520101043.ta.chkpt" at 2024-07-21 08:01:41 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 44 / 4 / 100:  50%|████████▌        | 100/200 [3:31:41<3:31:41, 127.02s/it]textattack: Saving checkpoint under "checkpoints\1721526248054.ta.chkpt" at 2024-07-21 09:44:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 68 / 7 / 150:  75%|████████████▊    | 150/200 [5:31:54<1:50:38, 132.77s/it]textattack: Saving checkpoint under "checkpoints\1721533461237.ta.chkpt" at 2024-07-21 11:44:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 96 / 92 / 12 / 200: 100%|██████████████████| 200/200 [7:34:11<00:00, 136.26s/it]textattack: Saving checkpoint under "checkpoints\1721540797429.ta.chkpt" at 2024-07-21 13:46:37 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 96 / 92 / 12 / 200: 100%|██████████████████| 200/200 [7:34:11<00:00, 136.26s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 96     |
| Number of failed attacks:     | 92     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 46.0%  |
| Attack success rate:          | 51.06% |
| Average perturbed word %:     | 26.95% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 231.86 |
+-------------------------------+--------+


In [30]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptep095.csv', target_ds_ag, 0.5, 0.95)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  25%|██████▎                  | 50/200 [19:35<58:45, 23.50s/it]textattack: Saving checkpoint under "checkpoints\1721541972696.ta.chkpt" at 2024-07-21 14:06:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 96 / 4 / 100:  50%|███████████▌           | 100/200 [41:10<41:10, 24.71s/it]textattack: Saving checkpoint under "checkpoints\1721543268021.ta.chkpt" at 2024-07-21 14:27:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 142 / 7 / 150:  75%|███████████████     | 150/200 [1:03:26<21:08, 25.38s/it]textattack: Saving checkpoint under "checkpoints\1721544603987.ta.chkpt" at 2024-07-21 14:50:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 186 / 12 / 200: 100%|███████████████████| 200/200 [1:23:57<00:00, 25.19s/it]textattack: Saving checkpoint under "checkpoints\1721545835189.ta.chkpt" at 2024-07-21 15:10:35 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 186 / 12 / 200: 100%|███████████████████| 200/200 [1:23:57<00:00, 25.19s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 2     |
| Number of failed attacks:     | 186   |
| Number of skipped attacks:    | 12    |
| Original accuracy:            | 94.0% |
| Accuracy under attack:        | 93.0% |
| Attack success rate:          | 1.06% |
| Average perturbed word %:     | 6.15% |
| Average num. words per input: | 38.66 |
| Avg num queries:              | 37.04 |
+-------------------------------+-------+


In [31]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptsim06.csv', target_ds_ag, 0.6, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 34 / 14 / 2 / 50:  25%|████▊              | 50/200 [2:15:04<6:45:13, 162.09s/it]textattack: Saving checkpoint under "checkpoints\1721553939968.ta.chkpt" at 2024-07-21 17:25:39 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 77 / 19 / 4 / 100:  50%|████████▌        | 100/200 [4:07:15<4:07:15, 148.35s/it]textattack: Saving checkpoint under "checkpoints\1721560670370.ta.chkpt" at 2024-07-21 19:17:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 111 / 32 / 7 / 150:  75%|████████████    | 150/200 [6:09:48<2:03:16, 147.92s/it]textattack: Saving checkpoint under "checkpoints\1721568023980.ta.chkpt" at 2024-07-21 21:20:23 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 144 / 44 / 12 / 200: 100%|█████████████████| 200/200 [8:17:03<00:00, 149.12s/it]textattack: Saving checkpoint under "checkpoints\1721575658354.ta.chkpt" at 2024-07-21 23:27:38 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 144 / 44 / 12 / 200: 100%|█████████████████| 200/200 [8:17:03<00:00, 149.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 144    |
| Number of failed attacks:     | 44     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 22.0%  |
| Attack success rate:          | 76.6%  |
| Average perturbed word %:     | 27.24% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 246.29 |
+-------------------------------+--------+


In [32]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptsim07.csv', target_ds_ag, 0.7, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 21 / 27 / 2 / 50:  25%|████▊              | 50/200 [1:28:07<4:24:21, 105.75s/it]textattack: Saving checkpoint under "checkpoints\1721580945860.ta.chkpt" at 2024-07-22 00:55:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 39 / 4 / 100:  50%|████████▌        | 100/200 [2:52:10<2:52:10, 103.30s/it]textattack: Saving checkpoint under "checkpoints\1721585988641.ta.chkpt" at 2024-07-22 02:19:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 61 / 7 / 150:  75%|████████████▊    | 150/200 [4:15:38<1:25:12, 102.25s/it]textattack: Saving checkpoint under "checkpoints\1721590996692.ta.chkpt" at 2024-07-22 03:43:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 113 / 75 / 12 / 200: 100%|█████████████████| 200/200 [5:39:20<00:00, 101.80s/it]textattack: Saving checkpoint under "checkpoints\1721596019089.ta.chkpt" at 2024-07-22 05:06:59 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 113 / 75 / 12 / 200: 100%|█████████████████| 200/200 [5:39:20<00:00, 101.80s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 113    |
| Number of failed attacks:     | 75     |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 37.5%  |
| Attack success rate:          | 60.11% |
| Average perturbed word %:     | 26.01% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 169.31 |
+-------------------------------+--------+


In [33]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptsim08.csv', target_ds_ag, 0.8, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 13 / 35 / 2 / 50:  25%|█████▌                | 50/200 [53:08<2:39:25, 63.77s/it]textattack: Saving checkpoint under "checkpoints\1721599207524.ta.chkpt" at 2024-07-22 06:00:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 60 / 4 / 100:  50%|█████████         | 100/200 [1:47:54<1:47:54, 64.74s/it]textattack: Saving checkpoint under "checkpoints\1721602493286.ta.chkpt" at 2024-07-22 06:54:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 91 / 7 / 150:  75%|███████████████     | 150/200 [2:40:07<53:22, 64.05s/it]textattack: Saving checkpoint under "checkpoints\1721605626269.ta.chkpt" at 2024-07-22 07:47:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 115 / 12 / 200: 100%|██████████████████| 200/200 [3:32:42<00:00, 63.81s/it]textattack: Saving checkpoint under "checkpoints\1721608781411.ta.chkpt" at 2024-07-22 08:39:41 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 115 / 12 / 200: 100%|██████████████████| 200/200 [3:32:42<00:00, 63.81s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 115    |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 57.5%  |
| Attack success rate:          | 38.83% |
| Average perturbed word %:     | 24.96% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 105.14 |
+-------------------------------+--------+


In [34]:
runAttack(pretrained_wrapper_disbert, n_samples, '../attack_results_new/ag/disbertagattackptsim09.csv', target_ds_ag, 0.9, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/ag/disbertagattackptsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  25%|█████▊                 | 50/200 [29:23<1:28:09, 35.26s/it]textattack: Saving checkpoint under "checkpoints\1721610544513.ta.chkpt" at 2024-07-22 09:09:04 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 83 / 4 / 100:  50%|█████████         | 100/200 [1:01:30<1:01:30, 36.90s/it]textattack: Saving checkpoint under "checkpoints\1721612471811.ta.chkpt" at 2024-07-22 09:41:11 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 125 / 7 / 150:  75%|██████████████▎    | 150/200 [1:31:19<30:26, 36.53s/it]textattack: Saving checkpoint under "checkpoints\1721614261333.ta.chkpt" at 2024-07-22 10:11:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 165 / 12 / 200: 100%|██████████████████| 200/200 [2:01:47<00:00, 36.54s/it]textattack: Saving checkpoint under "checkpoints\1721616088854.ta.chkpt" at 2024-07-22 10:41:28 after 200 attacks.
[Succeeded / Failed / Skipped / Total] 23 / 165 / 12 / 200: 100%|██████████████████| 200/200 [2:01:47<00:00, 36.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 23     |
| Number of failed attacks:     | 165    |
| Number of skipped attacks:    | 12     |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 82.5%  |
| Attack success rate:          | 12.23% |
| Average perturbed word %:     | 14.44% |
| Average num. words per input: | 38.66  |
| Avg num queries:              | 59.28  |
+-------------------------------+--------+
